https://www.rwkgoodman.com/info-hub/what-you-need-to-know-about-the-court-of-protection-and-deputyships/

In [ ]:
# GitHub 

# !git pull
# !git merge main origin/deputyship
#!pip install pip update
#!pip install panda update
#!pip install arrow_pd_parser
#!pip install pydbtools
#!pip install xlsxwriter
#!pip install openpyxl
#!cat ~/.local/share/jupyter/kernels/your_project/kernel.json
# !cd venv/
# !export PYTHONPATH=$PWD
# !source <VENV>/bin/activate
# !pip list
# !pip install pip --upgrade
# !pip install python --upgrade
# !pip install python3 --upgrade
# !pip install pydbtools --upgrade
#!pip install --force numpy==1.26.4


#!pip install pydbtools
#!pip install --force numpy==1.24
import os
import calendar
import shutil
from datetime import datetime,timedelta
import pydbtools
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pandas as pd
import boto3
import numpy as np
import matplotlib.pyplot as plt
import logging
import botocore
import s3fs
from dateutil.relativedelta import relativedelta

In [ ]:


# suppress that specific pandas RuntimeWarning
import warnings
warnings.filterwarnings(
    "ignore",
    category=RuntimeWarning,
    message=".*invalid value encountered in cast.*"
)

# Ensure pydbtools is available
try:
    import pydbtools
except ImportError:
    raise ImportError("The 'pydbtools' package is required. Install it with: pip install pydbtools")

def fetch_cases_for_date(run_date: str) -> pd.DataFrame:
    """
    Fetch all cases & their fee reductions for the given run_date (YYYY-MM-DD)
    using pydbtools.read_sql_query, which returns a pandas DataFrame.
    """
    query = f"""
    WITH active_fee_reductions AS (
      SELECT
        fc.client_id,
        SUBSTRING(fr.type,1,1) || LOWER(SUBSTRING(fr.type,2)) AS type,
        DATE(fr.startdate) AS startdate,
        DATE(fr.enddate)   AS enddate,
        fc.payment_method
      FROM opg_sirius_prod.fee_reduction fr
      JOIN opg_sirius_prod.finance_client fc
        ON fc.id = fr.finance_client_id
       AND fc.glueexporteddate = DATE('{run_date}')
      JOIN (
        SELECT
          MAX(id)           AS id,
          finance_client_id
        FROM opg_sirius_prod.fee_reduction
        WHERE enddate           >= DATE('{run_date}')
          AND startdate         <= DATE('{run_date}')
          AND deleted            = FALSE
          AND glueexporteddate   = DATE('{run_date}')
        GROUP BY finance_client_id
      ) latest ON latest.id = fr.id
      WHERE fr.glueexporteddate = DATE('{run_date}')
    )
    SELECT
      c.glueexporteddate,
      c.caserecnumber            AS casenumber,
      c.uid                      AS siriusid,
      (
        SELECT supervisionlevel
        FROM opg_sirius_prod.supervision_level_log sll
        WHERE sll.order_id         = c.id
          AND sll.glueexporteddate = DATE('{run_date}')
        ORDER BY sll.appliesfrom DESC
        LIMIT 1
      ) AS casesupervisionlevel,
      p.risk_score               AS CREC,
      c.casesubtype              AS orderType,
      c.orderdate                AS ordermadedate,
      c.orderstatus              AS orderStatus,
      afr.type                   AS feereductiontype,
      p.dob,
      CASE
        WHEN FLOOR(DATE_DIFF('day', p.dob, p.createddate) / 365.25) < 0 THEN 0
        ELSE ROUND(DATE_DIFF('day', p.dob, p.createddate) / 365.25)
      END AS age_in_years
    FROM opg_sirius_prod.persons p
    JOIN opg_sirius_prod.cases c
      ON p.id                   = c.client_id
     AND c.glueexporteddate     = DATE('{run_date}')
    LEFT JOIN active_fee_reductions afr
      ON afr.client_id          = p.id
    WHERE c.orderstatus IN ('OPEN','ACTIVE','DUPLICATE')
      AND p.glueexporteddate     = DATE('{run_date}')
    ORDER BY c.orderdate;
    """
    return pydbtools.read_sql_query(query)


def parse_month(month_str: str) -> datetime:
    """Strip quotes/whitespace and parse 'YYYY-MM' → datetime."""
    cleaned = month_str.strip().strip("'\"")
    return datetime.strptime(cleaned, "%Y-%m")


def generate_month_list(start_month: str, end_month: str):
    """
    Return a list of datetime objects for each month-start
    from start_month to end_month inclusive.
    """
    start_dt = parse_month(start_month)
    end_dt = parse_month(end_month)
    if start_dt > end_dt:
        raise ValueError(f"Start month ({start_month}) is after end month ({end_month})")

    months = []
    current = start_dt
    while current <= end_dt:
        months.append(current)
        current += relativedelta(months=1)
    return months

def last_day_of_month(dt: datetime) -> str:
    """Return the last day of dt's month as 'YYYY-MM-DD'."""
    day = calendar.monthrange(dt.year, dt.month)[1]
    return dt.replace(day=day).strftime("%Y-%m-%d")
        
def export_monthly_reports(first_month: str, last_month: str, output_base="output"):
    # Clean inputs
    clean_first = first_month.strip().strip("'\"")
    clean_last = last_month.strip().strip("'\"")

    # Generate all months
    months = generate_month_list(clean_first, clean_last)
    if not months:
        print("No months in range; nothing to do.")
        return

    # Prepare output directory
    os.makedirs(output_base, exist_ok=True)
    excel_filename = f"cases_{clean_first}_to_{clean_last}.xlsx"
    excel_path = os.path.join(output_base, excel_filename)    
    
    # List to accumulate each month's DataFrame
    all_months = []

    # Create Excel workbook and write each month's sheet
    with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
        for dt in months:
            month_tag = dt.strftime("%Y-%m")
            run_date = last_day_of_month(dt)

            # Fetch data for this month-end
            df = fetch_cases_for_date(run_date)

            # Tag the DataFrame with its month, then collect it
            df["month"] = month_tag
            all_months.append(df)

            # Save CSV for this month
            month_folder = os.path.join(output_base, month_tag)
            os.makedirs(month_folder, exist_ok=True)
            csv_path = os.path.join(month_folder, f"cases_{month_tag}.csv")
            df.to_csv(csv_path, index=False)

            # Add to Excel workbook
            df.to_excel(writer, sheet_name=month_tag, index=False)

            print(f"→ Saved CSV for {month_tag}: {csv_path}")

        print(f"→ Combined Excel workbook saved at: {excel_path}")

    # After all sheets are written, concatenate & export one big CSV
    if all_months:
        combined_df = pd.concat(all_months, ignore_index=True)
        combined_csv_path = os.path.join(
            output_base,
            f"all_cases_{clean_first}_to_{clean_last}.csv"
        )
        combined_df.to_csv(combined_csv_path, index=False)
        print(f"→ Combined CSV for all months saved at: {combined_csv_path}")
        

def calculate_monthly_active_cases(first_month: str, last_month: str, output_base="output") -> pd.DataFrame:
    """
    For each month between first_month and last_month (inclusive), fetch the data,
    filter to ACTIVE cases, then aggregate unique casenumber counts by orderType.
    Writes a CSV 'monthly_active_cases_<first>_to_<last>.csv' under output_base,
    and returns the aggregated DataFrame.
    """
    # Clean inputs
    clean_first = first_month.strip().strip("'\"")
    clean_last = last_month.strip().strip("'\"")

    # Generate all months
    months = generate_month_list(clean_first, clean_last)
    if not months:
        print("No months in range; nothing to do.")
        return pd.DataFrame()

    # Prepare output directory
    os.makedirs(output_base, exist_ok=True)

    # List to collect each month's summary
    summaries = []

    for dt in months:
        month_tag = dt.strftime("%Y-%m")
        run_date = last_day_of_month(dt)

        # Fetch data for this month-end
        df = fetch_cases_for_date(run_date)

        # Keep only ACTIVE cases
        df_active = df[df["orderstatus"] == "ACTIVE"].copy()
        if df_active.empty:
            # still record zero counts for completeness
            summaries.append(
                pd.DataFrame([{"month": month_tag, "orderType": None, "active_case_count": 0}])
            )
            continue

        # Tag with month for grouping
        df_active["month"] = month_tag

        # Aggregate unique casenumbers per orderType
        summary = (
            df_active
            .groupby(["month", "ordertype"])["casenumber"]
            .nunique()
            .reset_index(name="active_case_count")
        )
        summaries.append(summary)

        print(f"→ Aggregated ACTIVE cases for {month_tag}")

    # Combine all month summaries
    result_df = pd.concat(summaries, ignore_index=True)

    # Fill any missing orderTypes/months with zeros if you want full matrix:
    # pivot = result_df.pivot_table(index="month", columns="orderType",
    #                               values="active_case_count", fill_value=0).reset_index()

    # Write out CSV
    out_csv = os.path.join(
        output_base,
        f"monthly_active_cases_{clean_first}_to_{clean_last}.csv"
    )
    result_df.to_csv(out_csv, index=False)
    print(f"→ Monthly ACTIVE cases CSV saved at: {out_csv}")

    return result_df

# Forecasting Active Caseloads:

## entered: count of cases newly appearing in the active caseload each month

## exited: count of cases that dropped out since the prior month

def calculate_monthly_flow(first_month: str, last_month: str, output_base="output") -> pd.DataFrame:
    """
    For each month from first_month to last_month (inclusive),
    snapshot the set of active casenumbers, then compare to the prior month
    to count how many entered and exited the active caseload.
    Writes 'monthly_flow_<first>_to_<last>.csv' under output_base,
    and returns the flow DataFrame.
    """
    # Clean inputs
    clean_first = first_month.strip().strip("'\"")
    clean_last  = last_month.strip().strip("'\"")

    # Generate all months
    months = generate_month_list(clean_first, clean_last)
    if not months:
        print("No months in range; nothing to do.")
        return pd.DataFrame()

    # Prepare snapshots dict: { "YYYY-MM": set_of_casenumbers }
    snapshots = {}
    for dt in months:
        month_tag = dt.strftime("%Y-%m")
        run_date  = last_day_of_month(dt)
        df        = fetch_cases_for_date(run_date)
        # Only unique casenumbers
        snapshots[month_tag] = set(df["casenumber"].unique())
        print(f"→ Snapshot for {month_tag}: {len(snapshots[month_tag])} active cases")

    # Build flow records
    flow_records = []
    prev_month = None
    for month in sorted(snapshots):
        current = snapshots[month]
        if prev_month is None:
            # First month: all are "new", none have "exited"
            flow_records.append({
                "month": month,
                "entered": len(current),
                "exited":  0
            })
        else:
            prev = snapshots[prev_month]
            entered = current - prev
            exited  = prev - current
            flow_records.append({
                "month":  month,
                "entered": len(entered),
                "exited":  len(exited)
            })
        prev_month = month

    # Create DataFrame and write CSV
    flow_df = pd.DataFrame(flow_records)
    os.makedirs(output_base, exist_ok=True)
    out_csv = os.path.join(
        output_base,
        f"monthly_flow_{clean_first}_to_{clean_last}.csv"
    )
    flow_df.to_csv(out_csv, index=False)
    print(f"→ Monthly flow CSV saved at: {out_csv}")

    return flow_df




The calculation of the age specific new orders and termination rates probably needs to be more specific ie by single years of age. 
1) age makes quite a big difference, above the age of 60 in particular in terms of the rates that are applied so it is probably better to apply these as single years. 
2) estimating the number of terminations for 2025 based on the 2024 estimated termination rates and active caseload in April 2025 suggest fewer terminations in 2025 despite the active case load being larger in April 2025 than April 2024, and this is because the age distribution is slightly different in 2025 than 2024. 
Thus, these highlights why we need to use more age specific estimates of new orders and termination rates, particularly as any effect of compressing the age bands too much is likely to be cumulative if we eventually try to extend our forecasts (say) upto 5 years, and / or link to cumulative LPA holders etc.

To calculate the new orders for 2025 I think the attached analysis needs to be repeated but also looking at the change between April 2023 and April 2024 and / or rolling 12 months in between to try to understand how the volume of new orders is changing, and then add this to the April 2025 active caseload and subtract the estimated number of terminations .


# forecasting model for active caseloads:

# Calculating Deputyship In Flows ( entered as New Deputyships) and outflows (exited as Terminations). 

- For example, identify entered cases from those cases that appear in the April 2025 Active Caseload but not in the April 2024 Active Caseload.  
- And identify exited cases who appear on the April 2024 Active Caseload but not on April 2025 Caseload. 
- Also estimate age specific Terminations rates by comparing total Terminations by age group with the volume of cases in the Active Caseload.

# calculate_yearly_deputyship_flows
Compares two snapshots (e.g. April 2024 vs April 2025) and writes out both the “new deputyships” (inflows) and “terminations” (outflows) CSVs.

# estimate_age_specific_termination_rates
Takes the termination set from above, buckets by age, compares against the earlier month’s active caseload age‐distribution, and writes a CSV of age‐specific termination rates.


# change Sql code above to do age specific New order and terminated

In [ ]:

def calculate_yearonyear_flows_and_age_rates(
    first_month: str,
    last_month: str,
    output_base: str = "output",
    redistribute_unknown_age=False
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Loops every month between first_month and last_month (inclusive),
    compares snapshot at month vs snapshot 12 months earlier,
    computes:
      - entered (new deputyships)
      - exited  (terminations)
      - age-specific termination & entry rates using 1-year age bins (0-119)
      
      - redistribute_unknown_age=False (the default) will simply ignore 
        any cases whose age_in_years is blank — they fall into no bin, 
        so they don’t contribute to any age‐specific counts or rates.

      - redistribute_unknown_age=True will take all of those “blank‐age” entries and 
        split them proportionally across your one-year age bins, for inflows, terminations, 
        and the baseline, before computing counts and rates.
        
    Writes:
      yearonyear_flows_<first>_to_<last>.csv
      termination_and_entry_rates_by_age_<first>_to_<last>.csv
    Returns: (flows_df, age_rates_df)
    """
    # create output directory
    os.makedirs(output_base, exist_ok=True)

    # generate list of month-start datetimes
    months = generate_month_list(first_month, last_month)
    flow_records = []
    age_rate_records = []

    # define one-year age bins 0–120 and labels '0'...'119'
    age_bins = list(range(0, 121))
    age_labels = [str(a) for a in age_bins[:-1]]

    # Load data for the period (pseudo-code, replace with real loading)
    # df = load_data(first_month, last_month)
    #df  = calculate_monthly_flow(first_month, last_month)
    # Example DataFrame must have columns: ['year', 'age', 'case_no', 'population_estimate']
    #print(df.head())
    
#     # Assign each record to a single-year age bin
#     df['age_group'] = pd.cut(
#         df['age'],
#         bins=age_bins,
#         labels=age_labels,
#         right=False,       # so bin '0' covers [0,1), '1' covers [1,2), etc.
#         include_lowest=True
#     )

#     # Aggregate flows (case counts) and population by year & age_group
#     rates = (
#         df
#         .groupby(['year', 'age_group'], observed=True)
#         .agg(
#             flows=('case_no', 'count'),
#             population=('population_estimate', 'sum')
#         )
#         .reset_index()
#     )

#     # Compute rate per 1,000 population
#     rates['rate_per_1000'] = (rates['flows'] / rates['population']) * 1000
#     print(f"rates['rate_per_1000']: {rates['rate_per_1000']}")
    

    for dt in months:
        prev_dt = dt - relativedelta(years=1)
        if prev_dt < parse_month(first_month):
            continue

        tag_cur = dt.strftime("%Y-%m")
        tag_prev = prev_dt.strftime("%Y-%m")

        # fetch snapshots
        df_cur = fetch_cases_for_date(last_day_of_month(dt))
        df_prev = fetch_cases_for_date(last_day_of_month(prev_dt))
        set_cur = set(df_cur["casenumber"])
        set_prev = set(df_prev["casenumber"])
        entered_set = set_cur - set_prev
        exited_set = set_prev - set_cur

        # record flows
        flow_records.append({
            "month": tag_cur,
            "active_count": len(set_cur),
            "pre_active_count": len(set_prev),
            "entered": len(entered_set),
            "exited": len(exited_set)
        })

        # subset for age grouping
        df_term = df_prev[df_prev["casenumber"].isin(exited_set)].copy()
        df_inflow = df_cur[df_cur["casenumber"].isin(entered_set)].copy()
        
    
        # after compute df_inflow and age_labels as before
        # 1) Count blanks
        entered_unknown = df_inflow['age_group'].isna().sum()
        print(f"Count blanks (entered_unknown): {entered_unknown}")
        
        # 2) Count known‐age inflows per age
        entry_counts = (
            df_inflow
              .dropna(subset=['age_group'])
              .groupby('age_group')['casenumber']
              .nunique()
              .reindex(age_labels, fill_value=0)
        )
        print(f"Count known‐age inflows per age: {entry_countsentry_counts}")
        
        # 3) Compute proportions (avoid divide-by-zero)
        total_known = entry_counts.sum()
        if total_known > 0:
            proportions = entry_counts / total_known
        else:
            # if literally *no* known ages, fall back to equal distribution
            proportions = pd.Series(1/len(age_labels), index=age_labels)
        print(f"proportions: {proportions}")
        
        # 4) Allocate unknowns
        allocated_unknowns = (proportions * entered_unknown).round().astype(int)

        # Now add allocated_unknowns into entry_counts for your reporting
        entry_counts += allocated_unknowns
        # Then use this adjusted entry_counts when you build age_rate_records
        
        
        df_base = df_prev.copy()

        for df_ in (df_term, df_inflow, df_base):
            df_["age_group"] = pd.cut(
                df_["age_in_years"],
                bins=age_bins,
                labels=age_labels,
                right=False,
                include_lowest=True
            )

        # group counts
        term_counts = df_term.groupby("age_group")["casenumber"].nunique()
        
        if redistribute_unknown_age:
            entry_counts = df_inflow.groupby("age_group")["casenumber"]\
                            .nunique()\
                            .reindex(age_labels, fill_value=0)
        else: 
            # commented below as decided to redistribute_unknown_age
            entry_counts = df_inflow.groupby("age_group")["casenumber"].nunique()

        
        base_counts = df_base.groupby("age_group")["casenumber"].nunique()

        # compile age-rate records
        for age in age_labels:
            active_cnt = int(base_counts.get(age, 0))
            term_cnt = int(term_counts.get(age, 0))
            entry_cnt = int(entry_counts.get(age, 0))
            rate = round(term_cnt / active_cnt, 4) if active_cnt else 0.0
            age_rate_records.append({
                "month": tag_cur,
                "age_group": age,
                "active_count": active_cnt,
                "entered": entry_cnt,
                "terminations": term_cnt,
                "termination_rate": rate
            })

    flows_df = pd.DataFrame(flow_records)
    age_rates_df = pd.DataFrame(age_rate_records)

    # write outputs
    flows_path = os.path.join(
        output_base,
        f"yearonyear_flows_{first_month}_to_{last_month}.csv"
    )
    rates_path = os.path.join(
        output_base,
        f"termination_and_entry_rates_by_age_{first_month}_to_{last_month}.csv"
    )
    flows_df.to_csv(flows_path, index=False)
    age_rates_df.to_csv(rates_path, index=False)

    print(f"→ Year-on-year flows saved to: {flows_path}")
    print(f"→ Age-specific termination & entry rates saved to: {rates_path}")

    return flows_df, age_rates_df


In [ ]:
rates1['month'] = rates1['month'].astype(str)
r1 = rates1.query("year-month == '2023-04' and age_group == '30'")['entered'].sum()



In [ ]:
export_monthly_reports("2023-04","2023-05")

active_summary = calculate_monthly_active_cases("2023-04","2023-05")
print(active_summary.head())

flow_summary  = calculate_monthly_flow("2023-04","2023-05")
print(flow_summary.head())

In [ ]:
rates1

In [ ]:
# 1) Run without redistribution
flows1, rates1 = calculate_yearonyear_flows_and_age_rates("2023-04","2023-05", "out1")

# 2) Run with redistribution
flows2, rates2 = calculate_yearonyear_flows_and_age_rates("2023-04","2023-05", "out2", redistribute_unknown_age=True)

# Compare total 'entered' counts in month '2018-06' for Financial bin:
r1 = rates1.query("month=='2023-04' and age_group=='30'")["entered"].sum()
r2 = rates2.query("month=='2023-04' and age_group=='30'")["entered"].sum()
print(r1, r2)  # r2 should be r1 plus part of the ~blank-age cases


In [ ]:
import statsmodels.api as sm

# annualise 'entered' flows_df
yearly = (
    flows_df
      .assign(year=lambda df: df['month'].str[:4].astype(int))
      .groupby('year')['entered']
      .sum()
      .reset_index()
)

# OLS trend
X = sm.add_constant(yearly['year'])
y = yearly['entered']
model = sm.OLS(y, X).fit()

# forecast 2024 & 2025
future = pd.DataFrame({'year':[2024,2025]})
future_X = sm.add_constant(future['year'])
future['entered_pred'] = model.predict(future_X)

print("Forecasted new orders:")
print(future)


In [ ]:
# def calculate_yearonyear_flows_and_age_rates(first_month: str,
#                                               last_month: str,
#                                               output_base="output",
#                                               age_bins=(0,1 30, 45, 60, 120),
#                                               age_labels=("0-29","30-44","45-59","60+")):
#     """
#     Loops every month between first_month and last_month (inclusive),
#     compares snapshot at month vs snapshot 12 months earlier,
#     computes:
#       - entered (new deputyships)
#       - exited  (terminations)
#       - age-specific termination rates
#       - age-specific entered counts
#     Writes:
#       yearonyear_flows_<first>_to_<last>.csv
#       termination_and_entry_rates_by_age_<first>_to_<last>.csv
#     Returns: (flows_df, age_rates_df)
#     """
#     os.makedirs(output_base, exist_ok=True)

#     months = generate_month_list(first_month, last_month)
#     flow_records = []
#     age_rate_records = []

#     for dt in months:
#         prev_dt = dt - relativedelta(years=1)
#         if prev_dt < parse_month(first_month):
#             continue

#         tag_cur, tag_prev = dt.strftime("%Y-%m"), prev_dt.strftime("%Y-%m")

#         df_cur     = fetch_cases_for_date(last_day_of_month(dt))
#         df_prev    = fetch_cases_for_date(last_day_of_month(prev_dt))
#         set_cur    = set(df_cur["casenumber"])
#         set_prev   = set(df_prev["casenumber"])
#         entered_set = set_cur - set_prev
#         exited_set  = set_prev - set_cur

#         flow_records.append({
#             "month":  tag_cur,
#             "active_count": len(set_cur),
#             "pre_active_count": len(set_prev),
#             "entered": len(entered_set),
#             "exited":  len(exited_set)
#         })

#         df_term     = df_prev[df_prev["casenumber"].isin(exited_set)].copy()
#         df_inflows  = df_cur [df_cur ["casenumber"].isin(entered_set)].copy()
#         df_baseline = df_prev.copy()

#         for df_ in (df_term, df_inflows, df_baseline):
#             df_["age_group"] = pd.cut(
#                 df_["age_in_years"],
#                 bins=age_bins,
#                 labels=age_labels,
#                 right=False
#             )

#         # <-- here we explicitly pass observed=False -->
#         ter_counts  = df_term    .groupby("age_group", observed=False)["casenumber"].nunique()
#         in_counts   = df_inflows .groupby("age_group", observed=False)["casenumber"].nunique()
#         base_counts = df_baseline.groupby("age_group", observed=False)["casenumber"].nunique()

#         for grp in age_labels:
#             active_cnt = int(base_counts.get(grp, 0))
#             term_cnt   = int(ter_counts .get(grp, 0))
#             entry_cnt  = int(in_counts  .get(grp, 0))
#             rate       = round(term_cnt / active_cnt, 4) if active_cnt else 0.0

#             age_rate_records.append({
#                 "month":            tag_cur,
#                 "age_group":        grp,
#                 "active_count":     active_cnt,
#                 "entered":          entry_cnt,
#                 "terminations":     term_cnt,
#                 "termination_rate": rate
#             })

#     flows_df     = pd.DataFrame(flow_records)
#     age_rates_df = pd.DataFrame(age_rate_records)

#     flows_path = os.path.join(
#         output_base,
#         f"yearonyear_flows_{first_month}_to_{last_month}.csv"
#     )
#     rates_path = os.path.join(
#         output_base,
#         f"termination_and_entry_rates_by_age_{first_month}_to_{last_month}.csv"
#     )

#     flows_df.to_csv(flows_path, index=False)
#     age_rates_df.to_csv(rates_path, index=False)

#     print(f"→ Year-on-year flows saved to: {flows_path}")
#     print(f"→ Age‐specific termination & entry rates saved to: {rates_path}")

#     return flows_df, age_rates_df



In [ ]:

# def calculate_yearonyear_flows_and_age_rates(
#     first_month: str,
#     last_month: str,
#     output_base: str = "output"
# ):
#     """
#     Calculate year-on-year flows and age-specific rates using one-year bins from age 0 to 120.

#     :param first_month: Starting period in 'YYYY-MM' format
#     :param last_month:  Ending period in 'YYYY-MM' format
#     :param output_base: Base path for output files
#     :returns: DataFrame with columns [year, age, flows, population, rate_per_1000]
#     """
#     # 1) Define one-year age bins 0–120 and labels '0','1',…,'119'
#     age_bins   = list(range(0, 121))          # [0,1,2,…,120]
#     age_labels = [str(a) for a in age_bins[:-1]]  # ['0','1',…,'119']

#     # 2) Load your data for the period (pseudo-code, replace with real loading)
#     # df = load_data(first_month, last_month)
#     # Example DataFrame must have columns: ['year', 'age', 'case_no', 'population_estimate']

#     # 3) Assign each record to a single-year age bin
#     df['age_group'] = pd.cut(
#         df['age'],
#         bins=age_bins,
#         labels=age_labels,
#         right=False,       # so bin '0' covers [0,1), '1' covers [1,2), etc.
#         include_lowest=True
#     )

#     # 4) Aggregate flows (case counts) and population by year & age_group
#     rates = (
#         df
#         .groupby(['year', 'age_group'], observed=True)
#         .agg(
#             flows=('case_no', 'count'),
#             population=('population_estimate', 'sum')
#         )
#         .reset_index()
#     )

#     # 5) Compute rate per 1,000 population
#     rates['rate_per_1000'] = (rates['flows'] / rates['population']) * 1000

#     # 6) Persist or return
#     out_path = f"{output_base}_yearonyear_age_rates.csv"
#     rates.to_csv(out_path, index=False)
#     print(f"Saved age‐specific rates to {out_path}")
#     return rates

In [ ]:
flow_summary  = calculate_monthly_flow(first_month, last_month)
flow_summary

In [ ]:
yoy_flows, age_rates = calculate_yearonyear_flows_and_age_rates(first_month, last_month)
yoy_flows, age_rates

# Is there any flow increasing/decrerasing trend in new deputyships and terminations?

# Can we use new deputyships and terminations and a couple of years data on active caseloads to forecast 12-month ahead?

# How do we reflect trends in New Deputyships ?

# How do we reflect trends in age specific termination rates ? 


In [ ]:
if __name__ == "__main__":
    first_month = input("Enter first month (YYYY-MM): ")
    last_month  = input("Enter last month  (YYYY-MM): ")

    export_monthly_reports(first_month, last_month)
    active_summary = calculate_monthly_active_cases(first_month, last_month)
    flow_summary  = calculate_monthly_flow(first_month, last_month)

    # now do month‐to‐month style year‐on‐year deputyship flows:
    
    # compute year-on-year flows + age rates across all months
    yoy_flows, age_rates = calculate_yearonyear_flows_and_age_rates(first_month, last_month)

    # after export_monthly_reports("2023-05", "2025-05")
    print("\nYear-on-year deputyship flows:")
    print(yoy_flows)
    # for each month and age‐group, the number of entered (new deputyships) alongside terminations and termination rates:
    print("\nAge-specific termination rates over time:")
    print(age_rates)


In [ ]:

# Time Series Forecasting
# For forecasting purposes what we need to know is the number of cases actually flowing onto the active caseload each month 
# and the numbers actually leaving. I need to know ACTIVE cases going in and out of the system 
# and to draw by logical conclusion what we mean by the active caseload so that 
# we can deduce from this new cases that have been added (each month) and those that have left. 
# The simplest way to do that, without having to separately define inputs cases and outputs (terminations) 
# is to take a snapshot of the active cases each month (by using glueexporteddate as monthly active caseloads, 
#                                                       which is already implemented in the code below) 
# and compare it with the previous month so that we can see who has flowed on and who has left.


# a simple 12-month projection of how many cases expected to enter and leave the active caseload each month.

# Indexing: Converts month strings into a DatetimeIndex at month-start (asfreq('MS')).
# Model: Uses ExponentialSmoothing with additive trend+seasonality (period=12).
# Forecast: model.forecast(12) gives the next 12 monthly points.
# Output: A DataFrame with columns month, entered_forecast, exited_forecast, saved to CSV.



def forecast_monthly_flow(flow_df: pd.DataFrame,
                          first_month: str,
                          last_month: str,
                          output_base="output") -> pd.DataFrame:
    """
    Given a flow_df with columns ['month','entered','exited'],
    fit Holt–Winters models (additive trend+seasonality, period=12)
    separately on 'entered' and 'exited', then forecast the next 12 months.
    Writes 'forecast_flow_<first>_to_<last>_next12.csv' under output_base
    and returns the forecast DataFrame.
    """
    # Clean inputs
    clean_first = first_month.strip().strip("'\"")
    clean_last  = last_month.strip().strip("'\"")
    os.makedirs(output_base, exist_ok=True)

    # Prepare the time index
    df = flow_df.copy()
    #df['month_dt'] = pd.to_datetime(df['month'] + '-01')
    df['month_dt'] = pd.to_datetime(df['month'], format='%Y-%m')
    df = df.set_index('month_dt').asfreq('MS')  # monthly start frequency

    # our two series
    entered = df['entered']
    exited  = df['exited']

    # fit Holt–Winters (additive trend+season, period=12)
    hw_entered = ExponentialSmoothing(entered,
                                      trend='add',
                                      seasonal='mul',
                                      seasonal_periods=12).fit()
    
    hw_exited  = ExponentialSmoothing(exited,
                                      trend='add',
                                      seasonal='add',
                                      seasonal_periods=12).fit()

    # forecast next 12
    f_entered = hw_entered.forecast(12)
    f_exited  = hw_exited.forecast(12)

    # assemble forecast DataFrame
    forecast_months = f_entered.index.strftime('%Y-%m')
    # convert to posivitive array
    f_entered = np.abs(f_entered)
    forecast_df = pd.DataFrame({
        'month':            forecast_months,
        'entered_forecast': f_entered.values.astype(int),
        'exited_forecast':  f_exited.values.astype(int),
    })

    # write to CSV
    out_csv = os.path.join(
        output_base,
        f"forecast_flow_{clean_first}_to_{clean_last}_next12.csv"
    )
    forecast_df.to_csv(out_csv, index=False)
    print(f"→ Forecast CSV for next 12 months saved at: {out_csv}")

    return forecast_df


# forecast next 12 months
forecast_summary = forecast_monthly_flow(flow_summary,
                                         first_month,
                                         last_month)
print("\nForecasted flow for next 12 months:")
print(forecast_summary)
    
flow_sum = forecast_summary
# Convert 'month' from string → datetime
flow_sum['month'] = pd.to_datetime(flow_sum['month'], format='%Y-%m')
    
# Convert both 'month' columns to datetime at the very start
flow_sum['month']          = pd.to_datetime(flow_sum['month'],          format='%Y-%m')
active_summary['month']    = pd.to_datetime(active_summary['month'],    format='%Y-%m')

    
# Compute “same month last year” for each forecast row
flow_sum['lookup_month'] = flow_sum['month'] - pd.DateOffset(years=1)

# Aggregate active_summary by month (if you have multiple rows per month)
active_agg = (
    active_summary
    .groupby('month', as_index=False)['active_case_count']
    .sum()
)
    
# Merge (left‐join) the previous‐year active counts in
flow_with_active = (
    flow_sum
    .merge(
        active_agg,
        left_on='lookup_month',
        right_on='month',
        how='left',
        suffixes=('','_prev')
    )
)
    
# Tidy up
flow_with_active = (
    flow_with_active
    .rename(columns={
        'month': 'month',                      # forecast month
        'active_case_count': 'active_case_count_prev_year'
    })
    .drop(columns=['month_prev','lookup_month'])
)

# add another column to the resulted table to calculate for each year-month 
# the forecasted active caseloads = active_case_count_prev_year + entered_forecast  - exited_forecast
flow_with_active['forecasted_active_caseload'] = (
    flow_with_active['active_case_count_prev_year']
    + flow_with_active['entered_forecast']
    - flow_with_active['exited_forecast']
)


print("\nForecasted flow with active cases from the previous year for next 12 months:")
print(flow_with_active)
    
    


In [ ]:
flow_with_active

## Purpose  of the Deputyship Modelling / Forecasting: 
The purpose of the Deputyships modelling is twofold: 
- 1) to forecast income derived from the active caseload (so we are only interested, for our purposes at least, in active cases where clients are actually being charged or there is a full remission or exemption.
- 2) for policy purposes, e.g., equalising general and minimal fees, or changing fee levels, so again we are only interested in active cases where the fee payable is known (including R&Es). 

## Definitions of active caseload, new orders, and terminations:
- New active caseload: is todays active caseload plus new orders minus closed cases, ie new active caseload = old active caseload + new orders – closed cases.
    - We need to agree a clear definition of the active caseload that we can compare at defined periods (say each month) and from this derive how many “new orders” and been added and how many “cases closed”.
    - In practice we might not want to do this every day, it is more likely that we would want to do this quarterly or at least annually but the same principle should apply. 
    - as long as we have a clear definition of the active caseload that the simplest way to achieve the above is to compare the active caseload over a defined period (say each month) to identify how many new cases (people) have been added and how many have been closed. 
    - this may be difficult to do historically as it would require recreating the active caseload at fixed points in the past in order to compare. 
    - it would be better to have a time series, but if the above is possible and gives us consistent / logical outputs for modelling purposes then we could work with that and begin to build up a time series each month. 
    - Ideally what we would also want to do in comparing the active caseloads over time is to link this to the "Ps" age so that we can both make longer term age specific assumptions about "terminations" and link assumptions about "new orders" to the LPA forecast. 
    - Any other information about "P" including fee type would be really useful.
    
- fees/clients being charged - are those a per-client thing? e.g. if a person has multiple orders they aren't being charged multiple sets of fees? so do they ONLY need to know active CLIENTS going in and out of the system and not counting active ORDERS going in and out?
    - i.e. we only consider clients where every order they've ever had has come in since the migration (HAVING COUNT(*) = COUNT(c.madeactivedate)).
    - but for eligible Sirus-only clients we track their earliest madeactivedate for any order they've had in Sirius, and likewise for people going 'out' we track the latest closedondate for clients who have regained-capacity or death-confirmed (p.clientstatus IN ('DEATH_CONFIRMED', 'REGAINED_CAPACITY')).
    - need to check the question of whether "P" is charged only once if they have separate orders for H&W and F&P, or for each order.
    - If "P" can be charged multiple times it would be useful to get Mandy's input at this point about how we define the size of active caseload ie if we are counting people ie "P" or all chargeable orders.
    - draw by logical conclusion what we mean by the active caseload so that we can deduce from this new orders / people that have been added (each month) and those that have left.
    - new orders and terminations don't really work for (our) forecasting needs , though that doesn't mean that they aren't useful for other users.
    - For instance , my understanding is that the current definition of new orders received by OPG includes cases that might not ever be added to the active caseload or delays mean the case remains open but not active for quite some while; this means that on current definitions we can't simply add new orders / people to the active caseload each month (say) to calculate the change in the size of the active caseload, and similarly for terminations.
    - For forecasting purposes what we need to know is the number of orders / people actually flowing onto the active caseload each month and the numbers actually leaving.
    - The simplest way to do that, without having to separately define inputs (orders/ people) and outputs (terminations) is to take a snapshot of the active caseload (once we have agreed how to define this) each month and compare it with the previous month so that we can see who has flowed on and who has left.
    - it would still be very very useful to see an historic trend for new orders (using the earliest date that the order was made) using the current definitions to get a sense of the overall trajectory and which would help inform what we think actual flows onto the active caseload might be over time (especially as to start with we would not have a lot of historic data on actual flows if we are using snapshots of the active caseload each month).
    
To count number of , expand on the code above to also 

counts the cases who have regained-capacity or death-confirmed:
```sql
SELECT 
        c.client_id,
        MAX(DATE(c.closedondate)) AS closed_date
    FROM opg_sirius_prod.cases c
    INNER JOIN opg_sirius_prod.persons p 
        ON p.id = c.client_id
       AND p.glueexporteddate = DATE('2025-04-09')
    WHERE c.glueexporteddate = DATE('2025-04-09')
      AND c.casetype = 'ORDER'
      AND p.clientstatus IN ('DEATH_CONFIRMED', 'REGAINED_CAPACITY')
    GROUP BY c.client_id, p.clientstatus, c.caserecnumber
    HAVING MAX(c.closedondate) IS NOT NULL) 
```

                                                            
    

# focusing on defining the "active caseload" based on actual client status (rather than order events), which makes sense for forecasting demand, income, and resourcing.

## Definition of "active caseload" (for forecasting):

- A person is in the active caseload if:

    - They have a supervision level (i.e., active order), and

    - They are currently alive (dateofdeath IS NULL), and

    - Their status is 'ACTIVE' or NULL.

## New clients: 
- First time a person enters the caseload (based on earliest appliesfrom date from supervision_level_log).

## Clients who leave: 
- Drop off in the snapshot comparison year-to-year.

WITH sll_per_order AS (
    SELECT *
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY appliesfrom ASC) rn
        FROM opg_sirius_prod.supervision_level_log
        WHERE glueexporteddate = DATE('2024-09-01') AND supervisionlevel IS NOT NULL
    ) x
    WHERE rn = 1
),
first_supervision_per_client AS (
    SELECT c.client_id, MIN(s.appliesfrom) AS appliesfrom
    FROM opg_sirius_prod.cases c
    LEFT JOIN sll_per_order s ON s.order_id = c.id
    WHERE c.glueexporteddate = DATE('2024-09-01')
    GROUP BY c.client_id
),
filtered_clients AS (
    SELECT 
        p.id AS client_id,
        p.caserecnumber AS court_number,
        p.clientstatus,
        p.dateofdeath,
        fspc.appliesfrom
    FROM opg_sirius_prod.persons p
    JOIN first_supervision_per_client fspc ON fspc.client_id = p.id
    WHERE p.glueexporteddate = DATE('2024-09-01')
      AND p.type = 'actor_client'
      AND fspc.appliesfrom IS NOT NULL
),
valid_clients AS (
    SELECT 
        client_id, 
        appliesfrom AS start_date
    FROM filtered_clients
    WHERE start_date BETWEEN DATE('2022-01-01') AND DATE('2024-09-01')
),
yearly_new_clients AS (
    SELECT 
        EXTRACT(YEAR FROM start_date) AS year,
        COUNT(DISTINCT client_id) AS new_clients_count
    FROM valid_clients
    GROUP BY EXTRACT(YEAR FROM start_date)
),
yearly_active_snapshot AS (
    SELECT 
        EXTRACT(YEAR FROM glueexporteddate) AS year,
        glueexporteddate AS snapshot_date,
        COUNT(DISTINCT id) AS active_clients
    FROM opg_sirius_prod.persons
    WHERE type = 'actor_client'
      AND (clientstatus = 'ACTIVE' OR clientstatus IS NULL)
      AND dateofdeath IS NULL
      AND glueexporteddate <= DATE('2024-09-01')
    GROUP BY glueexporteddate
),
yearly_losses_estimate AS (
    SELECT 
        y.year,
        (y.active_clients - COALESCE(LAG(y.active_clients) OVER (ORDER BY y.year), 0)) + COALESCE(nc.new_clients_count, 0) AS estimated_leavers
    FROM yearly_active_snapshot y
    LEFT JOIN yearly_new_clients nc ON y.year = nc.year
)
SELECT 
    yac.year,
    yac.snapshot_date,
    nc.new_clients_count,
    yac.active_clients,
    yl.estimated_leavers
FROM yearly_active_snapshot yac
LEFT JOIN yearly_new_clients nc ON yac.year = nc.year
LEFT JOIN yearly_losses_estimate yl ON yac.year = yl.year
ORDER BY yac.year DESC;


## Adding a daily net flow table to track clients entering and exiting the active caseload more precisely, based on:

- madeactivedate from cases → actual flows onto the active caseload (more reliable than orderdate)

- closedondate for clients whose orders are fully closed due to DEATH_CONFIRMED or REGAINED_CAPACITY → flows out of the caseload

### Annual snapshots (forecast-aligned)

### Daily inflows and outflows (actual net flows)
- comment/uncomment the last SELECT depending on whether you want daily flow as a separate result.

- The daily_net_flow CTE helps validate assumptions used in the yearly snapshot (estimated_leavers).

- aggregate daily_net_flow monthly for trend analysis.

WITH sll_per_order AS (
    SELECT *
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY appliesfrom ASC) rn
        FROM opg_sirius_prod.supervision_level_log
        WHERE glueexporteddate = DATE('2024-09-01') AND supervisionlevel IS NOT NULL
    ) x
    WHERE rn = 1
),
first_supervision_per_client AS (
    SELECT c.client_id, MIN(s.appliesfrom) AS appliesfrom
    FROM opg_sirius_prod.cases c
    LEFT JOIN sll_per_order s ON s.order_id = c.id
    WHERE c.glueexporteddate = DATE('2024-09-01')
    GROUP BY c.client_id
),
filtered_clients AS (
    SELECT 
        p.id AS client_id,
        p.caserecnumber AS court_number,
        p.clientstatus,
        p.dateofdeath,
        fspc.appliesfrom
    FROM opg_sirius_prod.persons p
    JOIN first_supervision_per_client fspc ON fspc.client_id = p.id
    WHERE p.glueexporteddate = DATE('2024-09-01')
      AND p.type = 'actor_client'
      AND fspc.appliesfrom IS NOT NULL
),
valid_clients AS (
    SELECT 
        client_id, 
        appliesfrom AS start_date
    FROM filtered_clients
    WHERE start_date BETWEEN DATE('2022-01-01') AND DATE('2024-09-01')
),
yearly_new_clients AS (
    SELECT 
        EXTRACT(YEAR FROM start_date) AS year,
        COUNT(DISTINCT client_id) AS new_clients_count
    FROM valid_clients
    GROUP BY EXTRACT(YEAR FROM start_date)
),
yearly_active_snapshot AS (
    SELECT 
        EXTRACT(YEAR FROM glueexporteddate) AS year,
        glueexporteddate AS snapshot_date,
        COUNT(DISTINCT id) AS active_clients
    FROM opg_sirius_prod.persons
    WHERE type = 'actor_client'
      AND (clientstatus = 'ACTIVE' OR clientstatus IS NULL)
      AND dateofdeath IS NULL
      AND glueexporteddate <= DATE('2024-09-01')
    GROUP BY glueexporteddate
),
yearly_losses_estimate AS (
    SELECT 
        y.year,
        (y.active_clients - COALESCE(LAG(y.active_clients) OVER (ORDER BY y.year), 0)) + COALESCE(nc.new_clients_count, 0) AS estimated_leavers
    FROM yearly_active_snapshot y
    LEFT JOIN yearly_new_clients nc ON y.year = nc.year
),

-- NEW: Net flow by actual movement dates
earliest_activations AS (
    SELECT
        c.client_id,
        MIN(DATE(c.madeactivedate)) AS first_active_date
    FROM opg_sirius_prod.cases c
    WHERE c.glueexporteddate = DATE('2025-04-09')
      AND c.casetype = 'ORDER'
    GROUP BY c.client_id
    HAVING COUNT(*) = COUNT(c.madeactivedate)
),
all_orders_closed AS (
    SELECT
        c.client_id,
        MAX(DATE(c.closedondate)) AS closed_date
    FROM opg_sirius_prod.cases c
    INNER JOIN opg_sirius_prod.persons p 
        ON p.id = c.client_id
       AND p.glueexporteddate = DATE('2025-04-09')
    WHERE c.glueexporteddate = DATE('2025-04-09')
      AND c.casetype = 'ORDER'
      AND p.clientstatus IN ('DEATH_CONFIRMED', 'REGAINED_CAPACITY')
    GROUP BY c.client_id, p.clientstatus, c.caserecnumber
    HAVING MAX(c.closedondate) IS NOT NULL
),
start_counts AS (
    SELECT first_active_date AS event_date, COUNT(*) AS in_count
    FROM earliest_activations
    GROUP BY first_active_date
),
end_counts AS (
    SELECT closed_date AS event_date, COUNT(*) AS out_count
    FROM all_orders_closed
    GROUP BY closed_date
),
all_dates AS (
    SELECT event_date FROM start_counts
    UNION
    SELECT event_date FROM end_counts
),
daily_net_flow AS (
    SELECT
        DATE(d.event_date) AS date,
        COALESCE(s.in_count, 0) AS inflow,
        COALESCE(e.out_count, 0) AS outflow
    FROM all_dates d
    LEFT JOIN start_counts s ON d.event_date = s.event_date
    LEFT JOIN end_counts e ON d.event_date = e.event_date
)

-- Final SELECTs (2 parts)
-- 1. Annual Summary
SELECT 
    yac.year,
    yac.snapshot_date,
    nc.new_clients_count,
    yac.active_clients,
    yl.estimated_leavers
FROM yearly_active_snapshot yac
LEFT JOIN yearly_new_clients nc ON yac.year = nc.year
LEFT JOIN yearly_losses_estimate yl ON yac.year = yl.year
ORDER BY yac.year DESC;

-- 2. Daily Flow (optional if needed separately)
-- SELECT * FROM daily_net_flow ORDER BY date;


# Demands --> Total demands in each year without age
# Age specific demands --> Aged demands ratio per year 
# Apply ML / Predictive modelling (SARIMA) to Total demands in each year without age historical         
# Multiply the forecasted by Aged demands ratio per year 

- active: order earliest made date before 2017 or during 2017,
    - any cases that had a start date before the given date and either no termination date, 
    - or a termination date that occurred after the given date should have been an active case. 
# For forecasting the active caseloads:
Active(t+1,age) = Active(t,age) + New(t+1,age) - Losses(t,age)/Active(t)

# OPG (Greg) code:

```sql
--ACTIVE CASELOAD BY DAY
SELECT dates.record_date, p.active_clients 
FROM (
    SELECT glueexporteddate, COUNT(*) AS Active_Clients 
    FROM persons 
    WHERE clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') 
    AND STARTS_WITH(caserecnumber, '6') = FALSE 
    GROUP BY glueexporteddate
    ) p
RIGHT JOIN (
    SELECT * 
    FROM UNNEST(sequence(DATE '2024-01-01', CURRENT_DATE, INTERVAL '1' DAY)) AS t(record_date)
    ) dates 
ON dates.record_date = p.glueexporteddate
ORDER BY dates.record_date;


--NEW cases
SELECT a.EarliestOrderAllocation, COUNT(*) AS New_Cases
FROM (
    SELECT p.caserecnumber, c.EarliestOrderAllocation 
    FROM persons p 
    LEFT JOIN
        (SELECT caserecnumber, MIN(madeactivedate) AS EarliestOrderAllocation 
        FROM cases c 
        WHERE c.glueexporteddate = current_date 
        GROUP BY caserecnumber
        ) c ON c.caserecnumber = p.caserecnumber
    WHERE p.glueexporteddate = current_date 
    AND p.type = 'actor_client'
    AND STARTS_WITH(p.caserecnumber, '6') = FALSE
    ) a
WHERE a.EarliestOrderAllocation >= DATE('2024-01-01')
GROUP BY a.EarliestOrderAllocation
ORDER BY a.EarliestOrderAllocation;


--CLOSED CASES
SELECT p.statusdate, COUNT(*) FROM persons p
WHERE p.glueexporteddate = current_date 
AND p.clientstatus IN ('DEATH_CONFIRMED', 'REGAINED_CAPACITY')
AND p.statusdate >= DATE('2024-01-01')
GROUP BY p.statusdate
ORDER BY p.statusdate;


--CLOSED CASES AGAIN
SELECT a.LatestOrderClosure, COUNT(*) FROM (
    SELECT p.caserecnumber, DATE(c.LatestOrderClosure) AS LatestOrderClosure FROM persons p 
    LEFT JOIN
        (SELECT caserecnumber, MAX(closedondate) AS LatestOrderClosure FROM cases c 
        WHERE c.glueexporteddate = current_date AND c.orderstatus = 'CLOSED'
        GROUP BY caserecnumber
        ) c ON c.caserecnumber = p.caserecnumber
    WHERE p.glueexporteddate = current_date 
    AND p.type = 'actor_client'
    AND p.clientstatus IN ('DEATH_CONFIRMED', 'REGAINED_CAPACITY')
    AND STARTS_WITH(p.caserecnumber, '6') = FALSE
    ) a
WHERE a.LatestOrderClosure >= DATE('2024-01-01')
GROUP BY a.LatestOrderClosure
ORDER BY a.LatestOrderClosure;


SELECT '2023-01-01', COUNT(*), COUNT(DISTINCT p.caserecnumber) FROM persons p 
WHERE p.clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') AND p.glueexporteddate = DATE('2023-01-01')
UNION ALL
SELECT '2024-01-01', COUNT(*), COUNT(DISTINCT p.caserecnumber) FROM persons p 
WHERE p.clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') AND p.glueexporteddate = DATE('2024-01-01');



--CLOSURES/NEW CASES BETWEEN DATES (Doesn't count cases where both events happened)
SELECT 
  (SELECT COUNT(*) 
   FROM (
   SELECT caserecnumber 
   FROM persons 
   WHERE clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') 
   AND STARTS_WITH(caserecnumber, '6') = FALSE 
   AND glueexporteddate = DATE('2024-12-19')) t1 --START DATE
   LEFT JOIN (
   SELECT caserecnumber 
   FROM persons 
   WHERE clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') 
   AND STARTS_WITH(caserecnumber, '6') = FALSE 
   AND glueexporteddate = DATE('2024-12-20')) t2 --END DATE
     ON t1.caserecnumber = t2.caserecnumber
   WHERE t2.caserecnumber IS NULL) AS closures,
  (SELECT COUNT(*) 
   FROM (SELECT caserecnumber 
   FROM persons 
   WHERE clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') 
   AND STARTS_WITH(caserecnumber, '6') = FALSE 
   AND glueexporteddate = DATE('2024-12-20')) t1 --END DATE
   LEFT JOIN (SELECT caserecnumber FROM persons 
   WHERE clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') 
   AND STARTS_WITH(caserecnumber, '6') = FALSE 
   AND glueexporteddate = DATE('2024-12-19')) t2 --START DATE
     ON t1.caserecnumber = t2.caserecnumber
   WHERE t2.caserecnumber IS NULL) AS new_cases;

```

In [ ]:
# This SQL query consolidates the logic from your Python functions into a single query that:

# Retrieves the relevant supervision level logs.
# Determines the earliest order and receipt dates for each client.
# Filters clients based on applicable date constraints.
# Aggregates the new client count per quarter.

deputyship_clients = pydbtools.read_sql_query(f"""
WITH 
    sll_per_order AS (
        SELECT *
        FROM (
            SELECT *, ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY appliesfrom ASC) rn
            FROM opg_sirius_prod.supervision_level_log
            WHERE glueexporteddate = DATE('2024-09-01')
              AND supervisionlevel IS NOT NULL
        ) x
        WHERE x.rn = 1
    ),
    sll_per_client AS (
        SELECT c.client_id, MIN(s.appliesfrom) AS appliesfrom
        FROM opg_sirius_prod.cases c
        LEFT JOIN sll_per_order s ON s.order_id = c.id
        WHERE c.glueexporteddate = DATE('2024-09-01')
        GROUP BY c.client_id
    ),
    earliest_orderdate AS (
        SELECT MIN(orderdate) AS orderdate, client_id
        FROM opg_sirius_prod.cases
        WHERE glueexporteddate = DATE('2024-09-01')
        GROUP BY client_id
    ),
    earliest_receiptdate AS (
        SELECT MIN(receiptdate) AS receiptdate, client_id
        FROM opg_sirius_prod.cases
        WHERE glueexporteddate = DATE('2024-09-01')
        GROUP BY client_id
    ),
    filtered_clients AS (
        SELECT 
            client.caserecnumber AS court_number,
            client.id AS client_id, 
            client.clientstatus,
            client.dateofdeath,
            eo.orderdate,
            er.receiptdate,
            sl.appliesfrom
        FROM opg_sirius_prod.persons client
        LEFT JOIN earliest_receiptdate er ON er.client_id = client.id
        LEFT JOIN earliest_orderdate eo ON eo.client_id = client.id
        LEFT JOIN sll_per_client sl ON sl.client_id = client.id
        WHERE client.glueexporteddate = DATE('2024-09-01')
          AND client.type = 'actor_client'
    ),
    valid_clients AS (
        SELECT 
            court_number, client_id, appliesfrom AS date
        FROM filtered_clients
        WHERE appliesfrom IS NOT NULL
          AND appliesfrom BETWEEN DATE('2022-01-01') AND DATE('2024-09-01')
        --   AND appliesfrom >= DATE('2015-01-01')
    ),
    quarterly_clients AS (
        SELECT 
            YEAR(date) AS year, 
            QUARTER(date) AS quarter, 
            COUNT(DISTINCT client_id) AS new_clients_count_quarter
        FROM valid_clients
        GROUP BY YEAR(date), QUARTER(date)
        ORDER BY year DESC, quarter DESC
    ),        
    yearly_clients AS (
        SELECT 
            YEAR(date) AS year, 
            COUNT(DISTINCT client_id) AS new_clients_count_year
        FROM valid_clients
        GROUP BY YEAR(date)
        ORDER BY year DESC
    ),
    active_caseloads AS (
        SELECT 
            DATE_FORMAT(p.glueexporteddate, '%Y') AS year,
            p.glueexporteddate AS year_end_date, 
            COUNT(*) AS active_clients_count_year
        FROM 
            opg_sirius_prod.persons p
        JOIN 
            (SELECT 
                MAX(glueexporteddate) AS MaxDate 
            FROM 
                opg_sirius_prod.persons 
            WHERE type = 'actor_client' 
            AND (clientstatus = 'ACTIVE' OR clientstatus IS NULL)
            AND glueexporteddate <= DATE('2024-09-01')
            -- AND YEAR(createddate) >= YEAR(glueexporteddate)
            GROUP BY 
                DATE_FORMAT(glueexporteddate, '%Y')
            ) max_dates 
        ON 
            p.glueexporteddate = max_dates.MaxDate
        WHERE 
            p.type = 'actor_client' 
            AND (p.clientstatus = 'ACTIVE' OR p.clientstatus IS NULL)
            AND p.glueexporteddate <= DATE('2024-09-01')
            -- AND YEAR(p.createddate) >= YEAR(p.glueexporteddate)
        GROUP BY 
            p.glueexporteddate
    ),
    closed_caseloads AS (
        SELECT 
            DATE_FORMAT(p.glueexporteddate, '%Y') AS year,
            p.glueexporteddate AS year_end_date, 
            COUNT(*) AS closed_clients_count_year
        FROM 
            opg_sirius_prod.persons p
        JOIN 
            (SELECT 
                MAX(glueexporteddate) AS MaxDate 
            FROM 
                opg_sirius_prod.persons 
            WHERE type = 'actor_client' 
            AND clientstatus = 'CLOSED'
            AND glueexporteddate <= DATE('2024-09-01')
            AND STARTS_WITH(caserecnumber, '6') = FALSE
            -- AND YEAR(createddate) >= YEAR(glueexporteddate)
            GROUP BY 
                DATE_FORMAT(glueexporteddate, '%Y')
            ) max_dates 
        ON 
            p.glueexporteddate = max_dates.MaxDate
        WHERE 
            p.type = 'actor_client' 
            AND p.clientstatus = 'CLOSED'
            AND p.glueexporteddate <= DATE('2024-09-01')
            AND STARTS_WITH(p.caserecnumber, '6') = FALSE
            -- AND YEAR(p.createddate) >= YEAR(p.glueexporteddate)
        GROUP BY 
            p.glueexporteddate
    ),
    /* calculation for losses per year by subtracting the active clients count of the previous year and the new clients count from the current year.*/
    losses_per_year AS (
        SELECT 
            yc.year,
            COALESCE(ac.active_clients_count_year, 0) - COALESCE(LAG(ac.active_clients_count_year) OVER (ORDER BY yc.year), ac.active_clients_count_year)
            + COALESCE(yc.new_clients_count_year, 0) AS losses
        FROM yearly_clients yc
        LEFT JOIN active_caseloads ac
        ON yc.year = CAST(ac.year AS INT)
    )
SELECT 
    yc.year, 
    ac.year_end_date,
    yc.new_clients_count_year,
    ac.active_clients_count_year,
    lp.losses,
    cc.closed_clients_count_year
FROM yearly_clients yc
LEFT JOIN active_caseloads ac
ON yc.year = CAST(ac.year AS INT)
LEFT JOIN losses_per_year lp
ON yc.year = lp.year
LEFT JOIN closed_caseloads cc
ON yc.year = CAST(cc.year AS INT)
ORDER BY yc.year DESC
""")

deputyship_clients.to_csv('deputyship_clients.csv', index=False)
deputyship_clients

In [ ]:
# Greg Quesries
# The attached workbook shows the 'active' caseload every day since 1/1/2024 alongside various ways of counting new and closed cases. 
# I've also included the queries used to produce the figures in each column - I calculated them separately, I haven't got SQL skills 
# as impressive as you guys! As Peter alluded to in the call, this data frustratingly doesn't seem suitable for reliably calculating 
# the number of new or closed cases over any given time period. The figures in the last two columns are probably quite accurate for income forecasting though 
# as we probably wouldn't want to count cases opened and closed on the same day as we hopefully wouldn't be invoicing them.

--ACTIVE CASELOAD BY DAY
SELECT dates.record_date, p.active_clients FROM (
    SELECT glueexporteddate, COUNT(*) AS active_clients FROM persons WHERE clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') AND STARTS_WITH(caserecnumber, '6') = FALSE GROUP BY glueexporteddate
    ) p
RIGHT JOIN (
    SELECT * 
    FROM UNNEST(sequence(DATE '2024-01-01', CURRENT_DATE, INTERVAL '1' DAY)) AS t(record_date)
    ) dates ON dates.record_date = p.glueexporteddate
ORDER BY dates.record_date


--NEW cases
SELECT a.EarliestOrderAllocation, COUNT(*) FROM (
    SELECT p.caserecnumber, c.EarliestOrderAllocation FROM persons p 
    LEFT JOIN
        (SELECT caserecnumber, MIN(madeactivedate) AS EarliestOrderAllocation FROM cases c 
        WHERE c.glueexporteddate = current_date GROUP BY caserecnumber
        ) c ON c.caserecnumber = p.caserecnumber
    WHERE p.glueexporteddate = current_date 
    AND p.type = 'actor_client'
    AND STARTS_WITH(p.caserecnumber, '6') = FALSE
    ) a
WHERE a.EarliestOrderAllocation >= DATE('2024-01-01')
GROUP BY a.EarliestOrderAllocation
ORDER BY a.EarliestOrderAllocation


--CLOSED CASES
SELECT p.statusdate, COUNT(*) FROM persons p
WHERE p.glueexporteddate = current_date 
AND p.clientstatus IN ('DEATH_CONFIRMED', 'REGAINED_CAPACITY')
AND p.statusdate >= DATE('2024-01-01')
GROUP BY p.statusdate
ORDER BY p.statusdate


--CLOSED CASES AGAIN
SELECT a.LatestOrderClosure, COUNT(*) FROM (
    SELECT p.caserecnumber, DATE(c.LatestOrderClosure) AS LatestOrderClosure FROM persons p 
    LEFT JOIN
        (SELECT caserecnumber, MAX(closedondate) AS LatestOrderClosure FROM cases c 
        WHERE c.glueexporteddate = current_date AND c.orderstatus = 'CLOSED'
        GROUP BY caserecnumber
        ) c ON c.caserecnumber = p.caserecnumber
    WHERE p.glueexporteddate = current_date 
    AND p.type = 'actor_client'
    AND p.clientstatus IN ('DEATH_CONFIRMED', 'REGAINED_CAPACITY')
    AND STARTS_WITH(p.caserecnumber, '6') = FALSE
    ) a
WHERE a.LatestOrderClosure >= DATE('2024-01-01')
GROUP BY a.LatestOrderClosure
ORDER BY a.LatestOrderClosure


SELECT '2023-01-01', COUNT(*), COUNT(DISTINCT p.caserecnumber) FROM persons p 
WHERE p.clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') AND p.glueexporteddate = DATE('2023-01-01')
UNION ALL
SELECT '2024-01-01', COUNT(*), COUNT(DISTINCT p.caserecnumber) FROM persons p 
WHERE p.clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') AND p.glueexporteddate = DATE('2024-01-01')


--CLOSURES/NEW CASES BETWEEN DATES (Doesn't count cases where both events happened)
SELECT 
  (SELECT COUNT(*) 
   FROM (SELECT caserecnumber FROM persons WHERE clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') AND STARTS_WITH(caserecnumber, '6') = FALSE AND glueexporteddate = DATE('2024-12-19')) t1 --START DATE
   LEFT JOIN (SELECT caserecnumber FROM persons WHERE clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') AND STARTS_WITH(caserecnumber, '6') = FALSE AND glueexporteddate = DATE('2024-12-20')) t2 --END DATE
     ON t1.caserecnumber = t2.caserecnumber
   WHERE t2.caserecnumber IS NULL) AS closures,
  (SELECT COUNT(*) 
   FROM (SELECT caserecnumber FROM persons WHERE clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') AND STARTS_WITH(caserecnumber, '6') = FALSE AND glueexporteddate = DATE('2024-12-20')) t1 --END DATE
   LEFT JOIN (SELECT caserecnumber FROM persons WHERE clientstatus IN ('ACTIVE', 'INACTIVE', 'DEATH_NOTIFIED') AND STARTS_WITH(caserecnumber, '6') = FALSE AND glueexporteddate = DATE('2024-12-19')) t2 --START DATE
     ON t1.caserecnumber = t2.caserecnumber
   WHERE t2.caserecnumber IS NULL) AS new_cases


In [ ]:
# # Queries from Athena:

# SELECT glueexporteddate AS Date, COUNT(*) AS Losses FROM persons
# WHERE type = 'actor_client' 
# AND ((clientstatus = 'CLOSED') 
# OR (clientstatus = 'DEATH_CONFIRMED')) 
# --OR (clientstatus = 'DEATH_NOTIFIED') 
# --OR (clientstatus = 'REGAINED_CAPACITY') 
# --OR (clientstatus = 'INACTIVE') 
# --OR (YEAR(dateofdeath) >= YEAR(glueexporteddate)))
# GROUP BY glueexporteddate
# ORDER BY glueexporteddate;

# SELECT 
#     DATE_FORMAT(p.glueexporteddate, '%Y') AS Year,
#     p.glueexporteddate AS YearEndDate, 
#     COUNT(*) AS NewClients 
# FROM 
#     persons p
# JOIN 
#     (SELECT 
#         MAX(glueexporteddate) AS MaxDate 
#     FROM 
#         persons 
#     WHERE type = 'actor_client' 
#     AND (clientstatus = 'ACTIVE' OR clientstatus IS NULL)
#     AND YEAR(createddate) >= YEAR(glueexporteddate)
#     GROUP BY 
#         DATE_FORMAT(glueexporteddate, '%Y')
#     ) max_dates 
# ON 
#     p.glueexporteddate = max_dates.MaxDate
# WHERE 
#     p.type = 'actor_client' 
#     AND (p.clientstatus = 'ACTIVE' OR p.clientstatus IS NULL)
#     AND YEAR(p.createddate) >= YEAR(p.glueexporteddate)
# GROUP BY 
#     p.glueexporteddate;
    
# SELECT glueexporteddate AS Date, COUNT(*) AS ActiveClients FROM persons p
# WHERE type = 'actor_client' AND (clientstatus = 'ACTIVE' OR clientstatus IS NULL)
# GROUP BY glueexporteddate
# ORDER BY glueexporteddate;

# SELECT 
#     DATE_FORMAT(p.glueexporteddate, '%Y') AS Year,
#     p.glueexporteddate AS YearEndDate, 
#     COUNT(*) AS NewClients 
# FROM 
#     persons p
# JOIN 
#     (SELECT 
#         MAX(glueexporteddate) AS MaxDate 
#     FROM 
#         persons 
#     WHERE type = 'actor_client' 
#     AND (clientstatus = 'ACTIVE' OR clientstatus IS NULL)
#     AND createddate IS NULL
#     GROUP BY 
#         DATE_FORMAT(glueexporteddate, '%Y')
#     ) max_dates 
# ON 
#     p.glueexporteddate = max_dates.MaxDate
# WHERE 
#     p.type = 'actor_client' 
#     AND (p.clientstatus = 'ACTIVE' OR p.clientstatus IS NULL)
#     AND p.createddate IS NULL
# GROUP BY 
#     p.glueexporteddate;
    
# WITH
# active_cases AS
#     (SELECT 
#         p.createddate,
#         CASE 
#             WHEN FLOOR(DATE_DIFF('day', p.dob, p.createddate) / 365.25) < 0 THEN 0
#             ELSE ROUND(DATE_DIFF('day', p.dob, p.createddate) / 365.25)
#         END AS age_in_years
#     FROM persons p
#     WHERE glueexporteddate = DATE('2024-12-31')
#         AND p.type = 'actor_client' 
#         AND (p.clientstatus = 'ACTIVE' OR p.clientstatus IS NULL)
#     GROUP BY p.createddate, p.dob
#     ORDER BY p.createddate, p.dob DESC
#     ),
# aggregated_active_caseloads AS (
#     SELECT 
#         YEAR(createddate) AS year,
#         age_in_years,
#         COUNT(*) AS ActiveClients 
#     FROM active_cases
#     GROUP BY YEAR(createddate), age_in_years
#     ORDER BY YEAR(createddate), age_in_years
# ),
# total_active_cases AS (
#     SELECT 
#         a.year, 
#         SUM(a.ActiveClients) AS TotalActiveCases
#     FROM aggregated_active_caseloads a
#     GROUP BY a.year
#     ORDER BY a.year ASC
# )
# SELECT 
#     a.year, 
#     a.age_in_years,
#     a.ActiveClients,
#     b.TotalActiveCases
# FROM aggregated_active_caseloads a
# INNER JOIN total_active_cases b
#     ON a.year = b.year;
    
# WITH  
#     years AS (
#         SELECT year
#         FROM UNNEST(SEQUENCE(2022, 2024)) AS t(year)
#     ),
#     ages AS (
#         SELECT age_in_years
#         FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
#     ),
#     all_years_ages AS (
#         SELECT y.year,
#             a.age_in_years
#         FROM years y
#         CROSS JOIN ages a
#     ),
#     active_cases AS (
#         SELECT 
#             YEAR(eo.earliest_order_date) AS year,
#             CASE 
#                 WHEN FLOOR(DATE_DIFF('day', a.dob, eo.earliest_order_date) / 365.25) < 0 THEN 0
#                 ELSE ROUND(DATE_DIFF('day', a.dob, eo.earliest_order_date) / 365.25)
#             END AS age_in_years,
#             a.id
#         FROM (
#             SELECT * FROM opg_sirius_prod.persons p
#             WHERE p.glueexporteddate = DATE('2025-03-03')
#                 AND p.type = 'actor_client'
#                 AND (p.clientstatus = 'ACTIVE' OR p.clientstatus IS NULL)
#         ) a
#         LEFT JOIN (
#             SELECT 
#                 c.caserecnumber,
#                 MIN(c.orderdate) AS earliest_order_date,
#                 c.client_id
#             FROM opg_sirius_prod.cases c
#             WHERE c.glueexporteddate = DATE('2025-03-03')
#                 AND c.type = 'order'
#             GROUP BY c.caserecnumber, c.client_id
#         ) eo  
#         ON eo.client_id = a.id
#         ORDER BY eo.earliest_order_date, a.dob
#     ),
#     aggregated_active_caseloads AS (
#         SELECT 
#             year,
#             age_in_years,
#             COUNT(*) AS ActiveClients
#         FROM active_cases
#         GROUP BY year, age_in_years
#         ORDER BY year, age_in_years
#     ),
#     filled_aggregated_active_caseloads AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             COALESCE(ad.ActiveClients, 0) AS ActiveClients
#         FROM all_years_ages a
#         LEFT JOIN aggregated_active_caseloads ad 
#             ON a.year = ad.year  -- Ensure the join includes both year and age
#             AND a.age_in_years = ad.age_in_years
#         ORDER BY a.year, a.age_in_years
#     ),
#     total_active_cases_aged AS (
#         SELECT a.year, a.age_in_years, SUM(a.ActiveClients) AS TotalActiveCasesAged
#         FROM filled_aggregated_active_caseloads a
#         GROUP BY a.year, a.age_in_years
#         ORDER BY a.year ASC, a.age_in_years ASC
#     ),
#     total_active_cases AS (
#         SELECT a.year, SUM(a.ActiveClients) AS TotalActiveCases
#         FROM filled_aggregated_active_caseloads a
#         GROUP BY a.year
#         ORDER BY a.year ASC
#     )
# SELECT 
#     a.year, a.age_in_years,
#     a.TotalActiveCasesAged,
#     b.TotalActiveCases
# FROM total_active_cases_aged a
# INNER JOIN total_active_cases b
#     ON a.year = b.year
# ORDER BY a.year ASC, a.age_in_years ASC;

# SELECT * FROM sirius_derived.opg_lpa_dashboard 
# WHERE glueexporteddate = DATE('2025-03-12');



# WITH 
#     years AS (
#         SELECT year
#         FROM UNNEST(SEQUENCE(2022, 2022)) AS t(year)
#     ),
#     ages AS (
#     	SELECT age_in_years
#     	FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
#     ),
#     all_years_ages AS (
#     	SELECT y.year,
#     		a.age_in_years
#     	FROM years y
#     		CROSS JOIN ages a
#     ),
#     processed_age_loss AS (
#     	SELECT YEAR(earliest_order_date) AS year,
#     		GREATEST(0, ROUND(raw_age)) AS age_in_years,
#     		termination_date
#     	FROM (
#     			SELECT eo.earliest_order_date,
#     				DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
#     				CASE
#     					WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#     					WHEN p.clientstatus != 'ACTIVE'
#     					AND rc.statusdate IS NOT NULL THEN rc.statusdate
#     					WHEN p.dateofdeath IS NULL
#     					AND dn.datenotified IS NOT NULL THEN dn.datenotified ELSE NULL
#     				END AS termination_date --COALESCE(p.dateofdeath, NULLIF(rc.statusdate, p.clientstatus = 'ACTIVE'), dn.datenotified) AS termination_date
#     			FROM opg_sirius_prod.persons p
#     				LEFT JOIN opg_sirius_prod.death_notifications dn ON dn.person_id = p.id
#     				AND dn.glueexporteddate = CURRENT_DATE
#     				LEFT JOIN (
#     					SELECT client_id,
#     						MIN(orderdate) AS earliest_order_date
#     					FROM opg_sirius_prod.cases
#     					WHERE glueexporteddate = CURRENT_DATE
#     						AND type = 'order'
#     					GROUP BY client_id
#     				) eo ON eo.client_id = p.id
#     				LEFT JOIN (
#     					SELECT client_id,
#     						statusdate
#     					FROM opg_sirius_prod.cases
#     					WHERE glueexporteddate = CURRENT_DATE
#     						AND orderclosurereason = 'CLIENT REGAINED CAPACITY'
#     				) rc ON rc.client_id = p.id
#     			WHERE p.glueexporteddate = CURRENT_DATE
#     				AND p.type = 'actor_client'
#     		)
#     ),
#     agg_loss_count AS (
#     	SELECT year,
#     		age_in_years,
#     		COUNT(termination_date) AS num_losses
#     	FROM processed_age_loss
#     	GROUP BY year,
#     		age_in_years
#     ),
#     total_yearly_losses AS (
#     	SELECT year,
#     		SUM(num_losses) AS total_losses
#     	FROM agg_loss_count
#     	GROUP BY year
#     ),
#     final_agg AS (
#     	SELECT a.year,
#     		a.age_in_years,
#     		COALESCE(lc.num_losses, 0) AS num_losses
#     	FROM all_years_ages a
#     		LEFT JOIN agg_loss_count lc ON a.year = lc.year
#     		AND a.age_in_years = lc.age_in_years
#     ),
#     processed_age_data AS (
#     	SELECT YEAR(orderdate) AS year,
#     		GREATEST(0, ROUND(raw_age)) AS age_in_years
#     	FROM (
#     			SELECT c.caserecnumber,
#     				MIN(c.orderdate) AS orderdate,
#     				DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
#     			FROM opg_sirius_prod.persons p
#     				INNER JOIN opg_sirius_prod.cases c ON c.client_id = p.id
#     				AND c.glueexporteddate = CURRENT_DATE
#     			WHERE c.type = 'order'
#     			GROUP BY c.caserecnumber,
#     				p.dob
#     		)
#     ),
#     aggregated_data AS (
#     	SELECT year,
#     		age_in_years,
#     		COUNT(*) AS num_active_caseloads
#     	FROM processed_age_data
#     	GROUP BY year,
#     		age_in_years
#     ),
#     filled_data AS (
#     	SELECT a.year,
#     		a.age_in_years,
#     		COALESCE(ag.num_active_caseloads, 0) AS num_active_caseloads
#     	FROM all_years_ages a
#     		LEFT JOIN aggregated_data ag ON a.year = ag.year
#     		AND a.age_in_years = ag.age_in_years
#     ),
#     total_yearly_active_caseloads AS (
#     	SELECT year,
#     		SUM(num_active_caseloads) AS total_active_caseloads
#     	FROM filled_data
#     	GROUP BY year
#     ),
#     age_data_new AS (
#         SELECT 
#             c.caserecnumber,
#             MIN(c.orderdate) AS earliestorderdate,
#             c.orderdate AS madedate,
#             DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age,
#             CASE
#                 WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                 WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                 WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                 ELSE NULL
#             END AS termination_date
#         FROM opg_sirius_prod.persons p
#         LEFT JOIN opg_sirius_prod.death_notifications dn 
#             ON dn.person_id = p.id
#             AND dn.glueexporteddate = DATE('2025-02-17')
#         LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     c.statusdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                     AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#             ) rc
#                 ON rc.client_id = p.id
#         INNER JOIN opg_sirius_prod.cases c 
#             ON c.client_id = p.id
#         WHERE c.glueexporteddate = DATE('2025-02-17')
#             AND c.type = 'order'
#             AND c.orderstatus = 'ACTIVE'
#             AND p.type = 'actor_client'
#         GROUP BY 
#             c.caserecnumber, 
#             p.dob,
#             c.orderdate,
#             p.dateofdeath,
#             p.clientstatus,
#             rc.statusdate,
#             dn.datenotified
#         --HAVING c.orderdate >= MIN(c.orderdate)
#         HAVING YEAR(c.orderdate) >= YEAR(MIN(c.orderdate))
#     ),
#     processed_age_data_new AS (
#         SELECT
#             YEAR(madedate) AS year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age)
#             END AS age_in_years,
#             caserecnumber
#         FROM age_data_new
#         WHERE termination_date IS NULL
#     ),
#     aggregated_data_new AS (
#         SELECT 
#             year,
#             age_in_years,
#             COUNT(*) AS new_orders
#         FROM processed_age_data_new
#         GROUP BY year, age_in_years
#     ),
#     filled_data_new AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             COALESCE(ag.new_orders, 0) AS num_new_orders
#         FROM all_years_ages a
#         LEFT JOIN aggregated_data_new ag 
#             ON a.year = ag.year AND a.age_in_years = ag.age_in_years
#         ORDER BY a.year, a.age_in_years
#     ),
#     processed_age_loss_closed AS (
#         SELECT
#             year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age)
#             END AS age_in_years,
#             termination_date
#         FROM (
#             SELECT
#                 YEAR(eo.earliest_order_date) AS year,
#                 p.clientstatus,
#                 DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
#                 CASE
#                     WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                     WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                     WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                     ELSE NULL
#                 END AS termination_date
#             FROM opg_sirius_prod.persons p
#             LEFT JOIN opg_sirius_prod.death_notifications dn 
#                 ON dn.person_id = p.id
#                 AND dn.glueexporteddate = DATE('2025-02-17')
#             LEFT JOIN (
#                 SELECT 
#                     c.caserecnumber,
#                     MIN(c.orderdate) AS earliest_order_date,
#                     c.client_id
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.type = 'order'
#                 GROUP BY c.caserecnumber, c.client_id
#             ) eo  
#                 ON eo.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     MIN(c.receiptdate) AS receiptdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.receiptdate IS NOT NULL
#                   AND c.type = 'order'
#                 GROUP BY c.client_id
#             ) er 
#                 ON er.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     c.statusdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#             ) rc 
#                 ON rc.client_id = p.id
#             WHERE p.glueexporteddate = DATE('2025-02-17')
#               AND p.type = 'actor_client'
#               AND p.clientstatus = 'CLOSED'
#               AND CASE
#                     WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                     WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                     WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                     ELSE NULL
#                 END IS NULL
#                 )
#     ),
#     agg_loss_count_closed AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             --a.termination_date
#             COUNT(a.termination_date IS NULL) AS num_losses_closed
#         FROM processed_age_loss_closed a
#         GROUP BY a.year, a.age_in_years
#         ORDER BY a.year, a.age_in_years
#     ),
#     total_yearly_losses_closed AS (
#         SELECT 
#             year,
#             SUM(num_losses_closed) AS total_losses_closed
#         FROM agg_loss_count_closed
#         GROUP BY year
#     ),
#     final_agg_closed AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             COALESCE(lc.num_losses_closed, 0) AS num_losses_closed
#         FROM all_years_ages a
#         LEFT JOIN agg_loss_count_closed lc
#             ON a.year = lc.year
#             AND a.age_in_years = lc.age_in_years
#         ORDER BY a.year, a.age_in_years
#     ),
#     total_yearly_new_orders AS (
#         SELECT 
#             year,
#             SUM(num_new_orders) AS total_new_orders
#         FROM filled_data_new
#         GROUP BY year
#     ),
#     processed_active_caseloads AS (
#         SELECT
#             YEAR(earliest_order_date) AS year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age)
#             END AS age_in_years,
#             termination_date
#         FROM (
#             SELECT
#                 eo.earliest_order_date,
#                 DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
#                 CASE
#                     WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                     WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                     WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                     ELSE NULL
#                 END AS termination_date
#             FROM opg_sirius_prod.persons p
#             LEFT JOIN opg_sirius_prod.death_notifications dn 
#                 ON dn.person_id = p.id
#                 AND dn.glueexporteddate = DATE('2025-02-17')
#             LEFT JOIN (
#                 SELECT 
#                     c.caserecnumber,
#                     MIN(c.orderdate) AS earliest_order_date,
#                     c.client_id
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.type = 'order'
#                   --AND c.orderdate >= DATE('2024-01-01')
#                 GROUP BY c.caserecnumber, c.client_id
#             ) eo  
#                 ON eo.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     MIN(c.receiptdate) AS receiptdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.receiptdate IS NOT NULL
#                 GROUP BY c.client_id
#             ) er 
#                 ON er.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     c.statusdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#             ) rc 
#                 ON rc.client_id = p.id
#             WHERE p.glueexporteddate = DATE('2025-02-17')
#               AND p.type = 'actor_client'
#         )
#         WHERE ((termination_date IS NULL) OR (YEAR(termination_date) > YEAR(earliest_order_date)))
#         --WHERE YEAR(orderdate) < YEAR(CURRENT_DATE) -- Orders that started before the given year
#         --AND (termination_date IS NULL OR termination_date >= DATE(CONCAT(YEAR(orderdate), '-01-01')))
#     ),
#     aggregated_active_caseloads AS (
#         SELECT 
#             eo.earliest_order_date,
#             DATE_DIFF('day', a.dob, eo.earliest_order_date) / 365.25 AS raw_age,
#             COUNT(*) AS ActiveClients 
#         FROM (opg_sirius_prod.persons p
#         WHERE p.glueexporteddate = DATE('2025-02-17')
#             AND p.type = 'actor_client'
#             AND (p.clientstatus = 'ACTIVE' OR p.clientstatus IS NULL)) a
#         LEFT JOIN (
#             SELECT 
#                 c.caserecnumber,
#                 MIN(c.orderdate) AS earliest_order_date,
#                 c.client_id
#             FROM opg_sirius_prod.cases c
#             WHERE c.glueexporteddate = DATE('2025-02-17')
#                 AND c.type = 'order'
#             GROUP BY c.caserecnumber, c.client_id
#                 ) eo  
#             ON eo.client_id = a.id
#         GROUP BY p.year, age_in_years
#         ORDER BY glueexporteddate
#     ), 
#     filled_aggregated_active_caseloads AS (
#         SELECT 
#             YEAR(earliest_order_date) AS year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age),
#             a.year,
#             a.age_in_years,
#             COALESCE(ad.num_active_caseloads_historical, 0
#                 --LAG(ad.num_active_caseloads_historical) OVER (ORDER BY a.age_in_years), 
#                 --LEAD(ad.num_active_caseloads_historical) OVER (ORDER BY a.age_in_years)
#             ) AS num_active_caseloads_historical
#         FROM all_years_ages a
#         LEFT JOIN aggregated_active_caseloads ad 
#             ON a.age_in_years = ad.age_in_years
#         ORDER BY a.year, a.age_in_years
#     )
#     SELECT 
#         a.year,
#         a.age_in_years,
#         b.num_active_caseloads,
#         d.num_new_orders,
#         a.num_losses,
#         e.num_losses_closed,
#         CAST((a.num_losses + e.num_losses_closed) AS DOUBLE) AS sum_losses,
#         CAST((b.num_active_caseloads - d.num_new_orders) AS DOUBLE) AS num_closed,
#         CAST(a.num_losses AS DOUBLE) / NULLIF((b.num_active_caseloads), 0) AS mortality_rate,
#         (CAST(a.num_losses AS DOUBLE) + CAST(e.num_losses_closed AS DOUBLE)) / NULLIF((b.num_active_caseloads), 0) AS sum_mortality_rate,
#         g.total_new_orders,
#         l.total_active_caseloads,
#         f.total_losses_closed,
#         c.total_losses,
#         CAST((c.total_losses + f.total_losses_closed) AS DOUBLE) AS total_losses_closed_terminated
#         --CAST(a.num_losses AS DOUBLE) / NULLIF(c.total_losses, 0) AS mortality_rate_aged,
#         --CAST(e.num_losses_closed AS DOUBLE) / NULLIF(f.total_losses_closed, 0) AS mortality_rate_aged_closed
#     FROM final_agg a
#     INNER JOIN filled_data b
#         ON a.year = b.year AND a.age_in_years = b.age_in_years
#     LEFT JOIN total_yearly_losses c
#         ON a.year = c.year
#     LEFT JOIN filled_data_new d
#         ON a.year = d.year AND a.age_in_years = d.age_in_years
#     LEFT JOIN final_agg_closed e
#         ON a.year = e.year AND a.age_in_years = e.age_in_years
#     LEFT JOIN total_yearly_losses_closed f
#         ON a.year = f.year
#     LEFT JOIN total_yearly_new_orders g
#         ON a.year = g.year
#     LEFT JOIN total_yearly_active_caseloads l
#         ON a.year = l.year
#     ORDER BY 
#         a.year ASC, 
#         a.age_in_years ASC;

In [ ]:
# https://github.com/moj-analytical-services/opg_fc_stats/blob/main/moj-fcstats-quarterly/run.py
# fcs: https://www.gov.uk/government/collections/family-court-statistics-quarterly
# run the code on the glueexporteddates that the fcs code is ran
date_min = '2015-01-01'
date_max = '2024-09-01'
glueexporteddate = '2024-09-01'


!pip install lovely_logger

import os
import sys
import shutil
import psutil
import string
import pydbtools
import pyarrow
import boto3
import uuid
import numpy
import pandas
import datetime
from datetime import date
import uuid
import lovely_logger as log
from arrow_pd_parser import reader, writer
import re
import git

def supervision_casrec_dates_sql(glueexporteddate): 
    
    '''
    Recover all possible interpretations of start dates associated with
    each supervision court number (i.e. client)
    '''
    
    q = (f"""
    WITH 
    
    sll_per_order as (
        SELECT *
        FROM (
            SELECT  *, ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY appliesfrom ASC) rn
            FROM opg_sirius_prod.supervision_level_log sll_mr
            WHERE sll_mr.glueexporteddate = DATE('{glueexporteddate}')
            AND sll_mr.supervisionlevel IS NOT NULL
        ) x
    WHERE x.rn = 1),

    sll_per_client as (
        SELECT
            c.client_id,
            MIN(s.appliesfrom) as appliesfrom
        FROM opg_sirius_prod.cases c
        LEFT JOIN sll_per_order s on s.order_id = c.id
        WHERE c.glueexporteddate = DATE('{glueexporteddate}')
        GROUP BY c.client_id),

    earliest_orderdate AS (
        SELECT 
            MIN(c.orderdate) as orderdate,
            c.client_id
        FROM opg_sirius_prod.cases c
        WHERE c.glueexporteddate = DATE('{glueexporteddate}')
        GROUP BY c.client_id),

    earliest_receiptdate AS (
        SELECT 
            MIN(c.receiptdate) as receiptdate,
            c.client_id
        FROM opg_sirius_prod.cases c
        WHERE c.glueexporteddate = DATE('{glueexporteddate}')
        GROUP BY c.client_id
    )

    SELECT
        client.caserecnumber as court_number,
        client.id as client_id, 
        client.clientstatus,
        client.dateofdeath,
        eo.orderdate,
        er.receiptdate,
        sl.appliesfrom
    FROM opg_sirius_prod.persons client
    LEFT JOIN earliest_receiptdate er ON er.client_id = client.id
    LEFT JOIN earliest_orderdate eo ON eo.client_id = client.id
    LEFT JOIN sll_per_client sl ON sl.client_id = client.id
    
    WHERE client.glueexporteddate = DATE('{glueexporteddate}')
    AND client.type = 'actor_client'
    """)
    
    q = ' '.join(q.split())
    
    log.info(f"supervision_casrec_dates_sql(): our query is {q}")
    
    return q


# def supervision_casrec_dates(glueexporteddate): 
    
#     '''
#     In case you want to check all the dates from the SQL above
#     '''
    
#     q = supervision_casrec_dates_sql(glueexporteddate)
#     df = pydbtools.read_sql_query(q)
    
#     return(df)

def supervision_casrec_date(glueexporteddate, date_min, date_max): 
    '''
    This is the function for deciding which dates and logic will be used
    to map deputyship cases to dates for the purposes of reporting new cases. 
    Should return something agnostic of which dates and logic were used 
    for downstream functions ... return "court_number" vs "date". 
    '''
    # Let date max/min be strings or dates
    date_min, date_max = [datetime.datetime.strptime(date, '%Y-%m-%d').date() if(type(date) == str) else date for date in [date_min, date_max]]
    log.info(f"supervision_casrec_date(): mapping supervision cases to dates")
    df = pydbtools.read_sql_query(supervision_casrec_dates_sql(glueexporteddate))
    # We only want one date
    df = df[['court_number','client_id','appliesfrom']]
    df = df.rename(columns = {'appliesfrom':'date'})
    # Stats don't care about cases with unknown dates
    idx = df.loc[df.date.isnull()].index
    log.warning(f"supervision_casrec_date(): {len(idx)} supervision cases had null dates, dropping")
    df = df.drop(idx)
    # Stats don't want data from outside quarters to be published
    log.info(f"supervision_casrec_date(): windowing supervision case dates")
    df = df.loc[df.date <= date_max]
    df = df.loc[df.date >= date_min]
    # Deputyship data before 2015 is not reliable. At time of migration, cases closed
    # more than 7 years prior were not migrated. Do not publish data from this period.
    log.info(f"supervision_casrec_date(): dropping pre-2015 supervision cases")
    df = df.loc[df.date >= datetime.date(2015,1,1)]
    df['year'] = pandas.DatetimeIndex(df['date']).year
    df['quarter'] = pandas.PeriodIndex(df.date, freq= 'Q')
    df = df.sort_values(by='date', ascending = False)
    return(df)

deputyship_new_clients = supervision_casrec_date(glueexporteddate, date_min, date_max)
print(deputyship_new_clients)

# Count the number of new clients in each year and quarter
new_clients_count_quarter = deputyship_new_clients.groupby(['year', 'quarter'])['client_id'].nunique()
print(f"new_clients_count_quarter: {new_clients_count_quarter}")

# Count the number of new clients in each year
new_clients_count_yearly = deputyship_new_clients.groupby(['year'])['client_id'].nunique()
print(f"new_clients_count_yearly: {new_clients_count_yearly}")


In [ ]:
# find new clients who have their first order received on a given month

# Choose the most recent available source data
target_date = pydbtools.read_sql_query(f"""
    SELECT max(glueexporteddate) AS glueexporteddate 
    FROM opg_sirius_prod.cases""")['glueexporteddate'][0]

print(target_date)

from datetime import datetime as dt
st_dt = '2017-01-01'
en_dt = '2024-12-31' #'2017-11-14'
# The final date the data is extracted
final_date = str(target_date)
final_date = pd.to_datetime(final_date, format ='%Y-%m-%d', errors='coerce') #.dt.date
final_date = final_date.strftime('%Y-%m-%d')
print(f"final_date_extracted: {final_date}")

# The start date of the analysis
start_date = pd.to_datetime(st_dt, format ='%Y-%m-%d', errors='coerce').strftime('%Y-%m-%d')
print(f"start_date: {start_date}")

# The end date of the analysis
end_date = pd.to_datetime(en_dt, format ='%Y-%m-%d', errors='coerce').strftime('%Y-%m-%d')
print(f"end_date: {end_date}")

# The start year of the analysis
start_year = pd.to_datetime(start_date, errors='coerce').year
print(f"start_year: {start_year}")

# The end year of the analysis
end_year = pd.to_datetime(end_date, errors='coerce').year
print(f"end_year: {end_year}")

# The final year of the analysis
final_year = pd.to_datetime(final_date, errors='coerce').year
print(f"final_year: {final_year}")

# Demands --> Total demands in each year without age
# Age specific demands --> Aged demands ratio per year 
# Apply ML / Predictive modelling (SARIMA) to Total demands in each year without age historical         
# Multiply the forecasted by Aged demands ratio per year 
#
#
# active caseloads: order earliest made date before a specific year or during that year

# num_total = num_active_caseload + num_losses
#
# num_losses = num_total - num_active_caseload

# Active(t+1,age) = (Active(t,age) + New(t+1,age)) - (Active(t,age) + New(t+1,age)) * Losses(t,age)/Active(t)

- If records with the same "court_number" are more than one ("earliest_order_made_date" for all those records are the same)
    - If there is 2 records with the same "court_number" that "earliest_order_made_date" for all those records are the same
        - If for the first record "order_status" IS "ACTIVE" and "order_made_date" = "earliest_order_made_date" AND termination_date IS NULL
        - AND If for the second record "order_status" IS "CLOSED" and "order_made_date" > "earliest_order_made_date" AND termination_date IS NULL
            - Then INCREMENT BY ONE the number of active caseloads (num_active_caseload) for all years starting from year("earliest_order_made_date") to the following years upto and including the current year (e.g., 2024).

- ElseIf there is only on record with a unique "court_number":
    - If (("order_status" IS "ACTIVE") AND (termination_date IS NULL)) 
        - Then ADD one to the number of new orders (num_new_orders)- Then INCREMENT BY ONE the number of new orders (num_new_orders) for that specific year("earliest_order_made_date").
        - AND Then INCREMENT BY ONE the number of active caseloads (num_active_caseload) for all years starting from year("earliest_order_made_date") to the following years upto and including the current year (e.g., 2024).
    - ElseIF ("order_status" IS "CLOSED") AND (termination_date ISNOT NULL) 
        - Then INCREMENT BY ONE the number of losses (num_losses) for that specific year("earliest_order_made_date").
    

<!-- # alternatively, instead of 'receiptdate' of the order, you could use the earliest date that the order was given a supervision-level in OPG
# i.e. basically the date where OPG actually kind of 'take control' of an order from CoP

# greg said: (That spike in November 2023 is genuine – we received a huge dump of older orders.)
# thats the spike in the receiptdate data
# but supervision-level date will not show that peak, because thats the date when OPG actually get around to doing their bit 
# - i.e. they cant just work a huge spike of 2000 received orders from OPG, that work gets spread over the next few months
# and, again, this is new CLIENTS/ new CASERECNUMBERS (Court numbers), not new orders
# count the new orders: df2.groupby(['month_year_earliest_receipt_date'])['client_id'].nunique().reset_index(name='count')  (WHERE c.glueexporteddate = DATE('2024-08-15'))
# (WHERE c.glueexporteddate = DATE('2024-08-15') OR {target_date} )

# age_data: Computes raw ages and identifies order dates.
# processed_age_data: Rounds ages to integers and handles negative ages by setting them to 0.
# all_years_ages: Generates all combinations of years and ages (0–106).
# aggregated_data: Counts the number of orders for each age in each year.
# filled_data: Joins aggregated_data with all_years_ages and imputes missing num_orders using the average for that year.
# Final Selection: Outputs the required columns in ascending order of year and age.

# Replaced generator: Used SEQUENCE(0, 106) and UNNEST to generate age values from 0 to 106. This works in Athena.
# Cross-Join all_years_ages: Combines distinct years with all possible age values.
# Aggregate and Join: Ensures missing ages are included for every year, and missing num_orders values are replaced with averages.     -->

In [ ]:
st_year = 2022
en_year = 2024
st_age = 0
en_age = 106

- num_losses is the number of people whos earliest_order_date is a given year and they are a given age at the time of that earliest order 
- then you want to count of that year-age group which ones are no longer active clients anymore, 
- then count their person clientstatus right, which can be CLOSED even without a termination date
- sum of COUNT(col), the CLOSED cases without the termination date, and COUNT(termination_date), for the ACTIVE cases with termination date

Our figures come from the opg_sirius_prod database too but there are a few intricacies to the data. The definition of 'deputyship' and 'case' within OPG are very vague. A client can have one or both of PFA and HW order types - some people would say a client with both has two deputyships but in the caseload figures I produce, I'm effectively counting the number of clients, regardless of the number of orders but if you take your count of cases from opg_sirius_prod.cases, a 'case' is an order.
 
You might find it easier to ignore HW orders to start with - there are only ~2600 active ones and around half of them have a PFA order too, so you'd only be excluding 1300 or so clients
 
This query'll give you daily active client counts going back a couple of years but it won't exactly match the caseload figures above as they include clients where we've been notified of the client's death but it's not been confirmed and don't exclude clients where the deputy is under investigation (tiny numbers - usually 40ish).
SELECT glueexporteddate AS Date, COUNT(*) AS ActiveClients FROM persons p
WHERE p.type = 'actor_client' AND (p.clientstatus = 'ACTIVE' OR p.clientstatus IS NULL)
GROUP BY glueexporteddate
ORDER BY glueexporteddate
 

In [ ]:
import pydbtools

glueexporteddate = '2024-12-31'

deputyship_aggregated = pydbtools.read_sql_query(f"""
WITH  
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE(2022, 2024)) AS t(year)
    ),
    ages AS (
        SELECT age_in_years
        FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
    ),
    all_years_ages AS (
        SELECT y.year,
            a.age_in_years
        FROM years y
        CROSS JOIN ages a
    ),
    active_cases AS (
        SELECT 
            YEAR(eo.earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(DATE_DIFF('day', a.dob, eo.earliest_order_date) / 365.25) < 0 THEN 0
                ELSE ROUND(DATE_DIFF('day', a.dob, eo.earliest_order_date) / 365.25)
            END AS age_in_years,
            a.id
        FROM (
            SELECT * FROM opg_sirius_prod.persons p
            WHERE p.glueexporteddate = DATE('{glueexporteddate}')
                AND p.type = 'actor_client'
                AND (p.clientstatus = 'ACTIVE' OR p.clientstatus IS NULL)
        ) a
        LEFT JOIN (
            SELECT 
                c.caserecnumber,
                MIN(c.orderdate) AS earliest_order_date,
                c.client_id
            FROM opg_sirius_prod.cases c
            WHERE c.glueexporteddate = DATE('{glueexporteddate}')
                AND c.type = 'order'
            GROUP BY c.caserecnumber, c.client_id
        ) eo  
        ON eo.client_id = a.id
        ORDER BY eo.earliest_order_date, a.dob
    ),
    aggregated_active_caseloads AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS ActiveClients
        FROM active_cases
        GROUP BY year, age_in_years
        ORDER BY year, age_in_years
    ),
    filled_aggregated_active_caseloads AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ad.ActiveClients, 0) AS ActiveClients
        FROM all_years_ages a
        LEFT JOIN aggregated_active_caseloads ad 
            ON a.year = ad.year  -- Ensure the join includes both year and age
            AND a.age_in_years = ad.age_in_years
        ORDER BY a.year, a.age_in_years
    ),
    total_active_cases_aged AS (
        SELECT a.year, a.age_in_years, SUM(a.ActiveClients) AS TotalActiveCasesAged
        FROM filled_aggregated_active_caseloads a
        GROUP BY a.year, a.age_in_years
        ORDER BY a.year ASC, a.age_in_years ASC
    ),
    total_active_cases AS (
        SELECT a.year, SUM(a.ActiveClients) AS TotalActiveCases
        FROM filled_aggregated_active_caseloads a
        GROUP BY a.year
        ORDER BY a.year ASC
    ),
    processed_age_loss AS (
    	SELECT YEAR(earliest_order_date) AS year,
    		GREATEST(0, ROUND(raw_age)) AS age_in_years,
    		termination_date
    	FROM (
    			SELECT eo.earliest_order_date,
    				DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
    				CASE
    					WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
    					WHEN p.clientstatus != 'ACTIVE'
    					AND rc.statusdate IS NOT NULL THEN rc.statusdate
    					WHEN p.dateofdeath IS NULL
    					AND dn.datenotified IS NOT NULL THEN dn.datenotified ELSE NULL
    				END AS termination_date --COALESCE(p.dateofdeath, NULLIF(rc.statusdate, p.clientstatus = 'ACTIVE'), dn.datenotified) AS termination_date
    			FROM opg_sirius_prod.persons p
    				LEFT JOIN opg_sirius_prod.death_notifications dn ON dn.person_id = p.id
    				AND dn.glueexporteddate = DATE('{glueexporteddate}')
    				LEFT JOIN (
    					SELECT client_id,
    						MIN(orderdate) AS earliest_order_date
    					FROM opg_sirius_prod.cases
    					WHERE glueexporteddate = DATE('{glueexporteddate}')
    						AND type = 'order'
    					GROUP BY client_id
    				) eo ON eo.client_id = p.id
    				LEFT JOIN (
    					SELECT client_id,
    						statusdate
    					FROM opg_sirius_prod.cases
    					WHERE glueexporteddate = DATE('{glueexporteddate}')
    						AND orderclosurereason = 'CLIENT REGAINED CAPACITY'
    				) rc ON rc.client_id = p.id
    			WHERE p.glueexporteddate = DATE('{glueexporteddate}')
    				AND p.type = 'actor_client'
    		)
    ),
    agg_loss_count AS (
    	SELECT year,
    		age_in_years,
    		COUNT(termination_date) AS num_losses
    	FROM processed_age_loss
    	GROUP BY year,
    		age_in_years
    ),
    total_yearly_losses AS (
    	SELECT year,
    		SUM(num_losses) AS total_losses
    	FROM agg_loss_count
    	GROUP BY year
    ),
    final_agg_losses AS (
    	SELECT a.year,
    		a.age_in_years,
    		COALESCE(lc.num_losses, 0) AS num_losses
    	FROM all_years_ages a
    		LEFT JOIN agg_loss_count lc ON a.year = lc.year
    		AND a.age_in_years = lc.age_in_years
    ),
    total_losses AS (
        SELECT a.year, SUM(a.num_losses) AS TotalLosses
        FROM final_agg_losses a
        GROUP BY a.year
        ORDER BY a.year ASC
    ),
    age_data_new AS (
        SELECT 
            c.caserecnumber,
            MIN(c.orderdate) AS earliestorderdate,
            c.orderdate AS madedate,
            DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age,
            CASE
                WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                ELSE NULL
            END AS termination_date
        FROM opg_sirius_prod.persons p
        LEFT JOIN opg_sirius_prod.death_notifications dn 
            ON dn.person_id = p.id
            AND dn.glueexporteddate = DATE('{glueexporteddate}')
        LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.statusdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{glueexporteddate}')
                    AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
            ) rc
                ON rc.client_id = p.id
        INNER JOIN opg_sirius_prod.cases c 
            ON c.client_id = p.id
        WHERE c.glueexporteddate = DATE('{glueexporteddate}')
            AND c.type = 'order'
            AND c.orderstatus = 'ACTIVE'
            AND p.type = 'actor_client'
        GROUP BY 
            c.caserecnumber, 
            p.dob,
            c.orderdate,
            p.dateofdeath,
            p.clientstatus,
            rc.statusdate,
            dn.datenotified
        --HAVING c.orderdate >= MIN(c.orderdate)
        HAVING YEAR(c.orderdate) >= YEAR(MIN(c.orderdate))
    ),
    processed_age_data_new AS (
        SELECT
            YEAR(madedate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            caserecnumber
        FROM age_data_new
        WHERE termination_date IS NULL
    ),
    aggregated_data_new AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS new_orders
        FROM processed_age_data_new
        GROUP BY year, age_in_years
    ),
    filled_data_new AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.new_orders, 0) AS num_new_orders
        FROM all_years_ages a
        LEFT JOIN aggregated_data_new ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
        ORDER BY a.year, a.age_in_years
    ),
    total_new_cases AS (
        SELECT a.year, SUM(a.num_new_orders) AS TotalNewCases
        FROM filled_data_new a
        GROUP BY a.year
        ORDER BY a.year ASC
    )
SELECT 
    a.year, 
    a.age_in_years,
    a.TotalActiveCasesAged,
    c.num_losses,
    d.num_new_orders,
    b.TotalActiveCases,
    e.TotalNewCases,
    f.TotalLosses
FROM total_active_cases_aged a
INNER JOIN total_active_cases b
    ON a.year = b.year
LEFT JOIN final_agg_losses c
    ON a.year = c.year
    AND a.age_in_years = c.age_in_years
LEFT JOIN filled_data_new d
    ON a.year = d.year
    AND a.age_in_years = d.age_in_years
LEFT JOIN total_new_cases e
    ON a.year = e.year
LEFT JOIN total_losses f 
    ON a.year = f.year
ORDER BY a.year ASC, a.age_in_years ASC
""")

deputyship_aggregated

In [ ]:
active_cases_historical = pydbtools.read_sql_query(f"""
WITH 
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE({st_year}, {en_year})) AS t(year)
    ),
    ages AS (
    	SELECT age_in_years
    	FROM UNNEST(SEQUENCE({st_age}, {en_age})) AS t(age_in_years)
    ),
    all_years_ages AS (
    	SELECT y.year,
    		a.age_in_years
    	FROM years y
    		CROSS JOIN ages a
    ),
    active_cases AS (
        SELECT 
        YEAR(eo.earliest_order_date) AS year,
        CASE 
            WHEN FLOOR(DATE_DIFF('day', a.dob, eo.earliest_order_date) / 365.25) < 0 THEN 0
            ELSE ROUND(DATE_DIFF('day', a.dob, eo.earliest_order_date) / 365.25)
        END AS age_in_years,
        a.id
        FROM (
        SELECT * FROM opg_sirius_prod.persons p
        WHERE p.glueexporteddate = DATE('2025-02-17')
            AND p.type = 'actor_client'
            AND (p.clientstatus = 'ACTIVE' OR p.clientstatus IS NULL)) a
        LEFT JOIN (
            SELECT 
                c.caserecnumber,
                MIN(c.orderdate) AS earliest_order_date,
                c.client_id
            FROM opg_sirius_prod.cases c
            WHERE c.glueexporteddate = DATE('2025-02-17')
                AND c.type = 'order'
                -- AND ignore HW orders to start with
            GROUP BY c.caserecnumber, c.client_id
                ) eo  
            ON eo.client_id = a.id
        --GROUP BY eo.earliest_order_date, a.dob
        ORDER BY eo.earliest_order_date, a.dob
    ),
    aggregated_active_caseloads AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS ActiveClients
        FROM active_cases
        GROUP BY year, age_in_years
        ORDER BY year, age_in_years
    ),
    filled_aggregated_active_caseloads AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ActiveClients, 0) AS ActiveClients
        FROM all_years_ages a
        LEFT JOIN aggregated_active_caseloads ad 
            ON a.age_in_years = ad.age_in_years
        ORDER BY a.year, a.age_in_years
    ),
    total_active_cases_aged AS (
        SELECT a.year, a.age_in_years, SUM(a.ActiveClients) AS TotalActiveCasesAged
        FROM filled_aggregated_active_caseloads a
        GROUP BY a.year, a.age_in_years
        ORDER BY a.year ASC, a.age_in_years ASC
    ),
    total_active_cases AS (
        SELECT a.year, SUM(a.ActiveClients) AS TotalActiveCases
        FROM filled_aggregated_active_caseloads a
        GROUP BY a.year
        ORDER BY a.year ASC
    )
    SELECT 
        a.year, a.age_in_years,
        a.TotalActiveCasesAged,
        b.TotalActiveCases
    FROM total_active_cases_aged a
    INNER JOIN total_active_cases b
        ON a.year = b.year
""")
active_cases_historical

In [ ]:
deputyship_num_active_cases_historical = active_cases_historical
#deputyship_num_active_cases_historical['mortality_rate'] = deputyship_num_active_cases_historical['mortality_rate'].fillna(0)
#deputyship_num_active_cases_historical['sum_mortality_rate'] = deputyship_num_active_cases_historical['sum_mortality_rate'].fillna(0)
deputyship_num_active_cases_historical['totalactivecasesaged'] = deputyship_num_active_cases_historical['totalactivecasesaged'].fillna(0)
deputyship_num_active_cases_historical.to_csv('deputyship_num_active_cases_historical.csv', index=False)

# Extract Active Cases Based on Start Date and Termination Conditions:
The query now includes num_active_caseloads_historical, which accounts for 
all active caseloads that started before the given year and were still active during that year.

- Identify cases that started before the given year.
- Ensure that these cases have no termination date or a termination date that occurred after the given year.
- Aggregate Active Caseloads per Year:
    - Count these cases per year and per age group.
    


In [ ]:
# num_active_cases_historical = pydbtools.read_sql_query(f"""
# WITH 
#     years AS (
#         SELECT year
#         FROM UNNEST(SEQUENCE({st_year}, {en_year})) AS t(year)
#     ),
#     ages AS (
#     	SELECT age_in_years
#     	FROM UNNEST(SEQUENCE({st_age}, {en_age})) AS t(age_in_years)
#     ),
#     all_years_ages AS (
#     	SELECT y.year,
#     		a.age_in_years
#     	FROM years y
#     		CROSS JOIN ages a
#     ),
#     processed_age_loss AS (
#     	SELECT YEAR(earliest_order_date) AS year,
#     		GREATEST(0, ROUND(raw_age)) AS age_in_years,
#     		termination_date
#     	FROM (
#     			SELECT eo.earliest_order_date,
#     				DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
#     				CASE
#     					WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#     					WHEN p.clientstatus != 'ACTIVE'
#     					AND rc.statusdate IS NOT NULL THEN rc.statusdate
#     					WHEN p.dateofdeath IS NULL
#     					AND dn.datenotified IS NOT NULL THEN dn.datenotified ELSE NULL
#     				END AS termination_date --COALESCE(p.dateofdeath, NULLIF(rc.statusdate, p.clientstatus = 'ACTIVE'), dn.datenotified) AS termination_date
#     			FROM opg_sirius_prod.persons p
#     				LEFT JOIN opg_sirius_prod.death_notifications dn ON dn.person_id = p.id
#     				AND dn.glueexporteddate = CURRENT_DATE
#     				LEFT JOIN (
#     					SELECT client_id,
#     						MIN(orderdate) AS earliest_order_date
#     					FROM opg_sirius_prod.cases
#     					WHERE glueexporteddate = CURRENT_DATE
#     						AND type = 'order'
#     					GROUP BY client_id
#     				) eo ON eo.client_id = p.id
#     				LEFT JOIN (
#     					SELECT client_id,
#     						statusdate
#     					FROM opg_sirius_prod.cases
#     					WHERE glueexporteddate = CURRENT_DATE
#     						AND orderclosurereason = 'CLIENT REGAINED CAPACITY'
#     				) rc ON rc.client_id = p.id
#     			WHERE p.glueexporteddate = CURRENT_DATE
#     				AND p.type = 'actor_client'
#     		)
#     ),
#     agg_loss_count AS (
#     	SELECT year,
#     		age_in_years,
#     		COUNT(termination_date) AS num_losses
#     	FROM processed_age_loss
#     	GROUP BY year,
#     		age_in_years
#     ),
#     total_yearly_losses AS (
#     	SELECT year,
#     		SUM(num_losses) AS total_losses
#     	FROM agg_loss_count
#     	GROUP BY year
#     ),
#     final_agg AS (
#     	SELECT a.year,
#     		a.age_in_years,
#     		COALESCE(lc.num_losses, 0) AS num_losses
#     	FROM all_years_ages a
#     		LEFT JOIN agg_loss_count lc ON a.year = lc.year
#     		AND a.age_in_years = lc.age_in_years
#     ),
#     processed_age_data AS (
#     	SELECT YEAR(orderdate) AS year,
#     		GREATEST(0, ROUND(raw_age)) AS age_in_years
#     	FROM (
#     			SELECT c.caserecnumber,
#     				MIN(c.orderdate) AS orderdate,
#     				DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
#     			FROM opg_sirius_prod.persons p
#     				INNER JOIN opg_sirius_prod.cases c ON c.client_id = p.id
#     				AND c.glueexporteddate = CURRENT_DATE
#     			WHERE c.type = 'order'
#     			GROUP BY c.caserecnumber,
#     				p.dob
#     		)
#     ),
#     aggregated_data AS (
#     	SELECT year,
#     		age_in_years,
#     		COUNT(*) AS num_active_caseloads
#     	FROM processed_age_data
#     	GROUP BY year,
#     		age_in_years
#     ),
#     filled_data AS (
#     	SELECT a.year,
#     		a.age_in_years,
#     		COALESCE(ag.num_active_caseloads, 0) AS num_active_caseloads
#     	FROM all_years_ages a
#     		LEFT JOIN aggregated_data ag ON a.year = ag.year
#     		AND a.age_in_years = ag.age_in_years
#     ),
#     total_yearly_active_caseloads AS (
#     	SELECT year,
#     		SUM(num_active_caseloads) AS total_active_caseloads
#     	FROM filled_data
#     	GROUP BY year
#     ),
#     age_data_new AS (
#         SELECT 
#             c.caserecnumber,
#             MIN(c.orderdate) AS earliestorderdate,
#             c.orderdate AS madedate,
#             DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age,
#             CASE
#                 WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                 WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                 WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                 ELSE NULL
#             END AS termination_date
#         FROM opg_sirius_prod.persons p
#         LEFT JOIN opg_sirius_prod.death_notifications dn 
#             ON dn.person_id = p.id
#             AND dn.glueexporteddate = DATE('2025-02-17')
#         LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     c.statusdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                     AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#             ) rc
#                 ON rc.client_id = p.id
#         INNER JOIN opg_sirius_prod.cases c 
#             ON c.client_id = p.id
#         WHERE c.glueexporteddate = DATE('2025-02-17')
#             AND c.type = 'order'
#             AND c.orderstatus = 'ACTIVE'
#             AND p.type = 'actor_client'
#         GROUP BY 
#             c.caserecnumber, 
#             p.dob,
#             c.orderdate,
#             p.dateofdeath,
#             p.clientstatus,
#             rc.statusdate,
#             dn.datenotified
#         --HAVING c.orderdate >= MIN(c.orderdate)
#         HAVING YEAR(c.orderdate) >= YEAR(MIN(c.orderdate))
#     ),
#     processed_age_data_new AS (
#         SELECT
#             YEAR(madedate) AS year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age)
#             END AS age_in_years,
#             caserecnumber
#         FROM age_data_new
#         WHERE termination_date IS NULL
#     ),
#     aggregated_data_new AS (
#         SELECT 
#             year,
#             age_in_years,
#             COUNT(*) AS new_orders
#         FROM processed_age_data_new
#         GROUP BY year, age_in_years
#     ),
#     filled_data_new AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             COALESCE(ag.new_orders, 0) AS num_new_orders
#         FROM all_years_ages a
#         LEFT JOIN aggregated_data_new ag 
#             ON a.year = ag.year AND a.age_in_years = ag.age_in_years
#         ORDER BY a.year, a.age_in_years
#     ),
#     processed_age_loss_closed AS (
#         SELECT
#             year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age)
#             END AS age_in_years,
#             termination_date
#         FROM (
#             SELECT
#                 YEAR(eo.earliest_order_date) AS year,
#                 p.clientstatus,
#                 DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
#                 CASE
#                     WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                     WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                     WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                     ELSE NULL
#                 END AS termination_date
#             FROM opg_sirius_prod.persons p
#             LEFT JOIN opg_sirius_prod.death_notifications dn 
#                 ON dn.person_id = p.id
#                 AND dn.glueexporteddate = DATE('2025-02-17')
#             LEFT JOIN (
#                 SELECT 
#                     c.caserecnumber,
#                     MIN(c.orderdate) AS earliest_order_date,
#                     c.client_id
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.type = 'order'
#                 GROUP BY c.caserecnumber, c.client_id
#             ) eo  
#                 ON eo.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     MIN(c.receiptdate) AS receiptdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.receiptdate IS NOT NULL
#                   AND c.type = 'order'
#                 GROUP BY c.client_id
#             ) er 
#                 ON er.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     c.statusdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#             ) rc 
#                 ON rc.client_id = p.id
#             WHERE p.glueexporteddate = DATE('2025-02-17')
#               AND p.type = 'actor_client'
#               AND p.clientstatus = 'CLOSED'
#               AND CASE
#                     WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                     WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                     WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                     ELSE NULL
#                 END IS NULL
#                 )
#     ),
#     agg_loss_count_closed AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             --a.termination_date
#             COUNT(a.termination_date IS NULL) AS num_losses_closed
#         FROM processed_age_loss_closed a
#         GROUP BY a.year, a.age_in_years
#         ORDER BY a.year, a.age_in_years
#     ),
#     total_yearly_losses_closed AS (
#         SELECT 
#             year,
#             SUM(num_losses_closed) AS total_losses_closed
#         FROM agg_loss_count_closed
#         GROUP BY year
#     ),
#     final_agg_closed AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             COALESCE(lc.num_losses_closed, 0) AS num_losses_closed
#         FROM all_years_ages a
#         LEFT JOIN agg_loss_count_closed lc
#             ON a.year = lc.year
#             AND a.age_in_years = lc.age_in_years
#         ORDER BY a.year, a.age_in_years
#     ),
#     total_yearly_new_orders AS (
#         SELECT 
#             year,
#             SUM(num_new_orders) AS total_new_orders
#         FROM filled_data_new
#         GROUP BY year
#     )
#     SELECT 
#         a.year,
#         a.age_in_years,
#         b.num_active_caseloads,
#         d.num_new_orders,
#         a.num_losses,
#         e.num_losses_closed,
#         CAST((a.num_losses + e.num_losses_closed) AS DOUBLE) AS sum_losses,
#         CAST((b.num_active_caseloads - d.num_new_orders) AS DOUBLE) AS num_closed,
#         CAST(a.num_losses AS DOUBLE) / NULLIF((b.num_active_caseloads), 0) AS mortality_rate,
#         (CAST(a.num_losses AS DOUBLE) + CAST(e.num_losses_closed AS DOUBLE)) / NULLIF((b.num_active_caseloads), 0) AS sum_mortality_rate,
#         g.total_new_orders,
#         l.total_active_caseloads,
#         f.total_losses_closed,
#         c.total_losses,
#         CAST((c.total_losses + f.total_losses_closed) AS DOUBLE) AS total_losses_closed_terminated
#         --CAST(a.num_losses AS DOUBLE) / NULLIF(c.total_losses, 0) AS mortality_rate_aged,
#         --CAST(e.num_losses_closed AS DOUBLE) / NULLIF(f.total_losses_closed, 0) AS mortality_rate_aged_closed
#     FROM final_agg a
#     INNER JOIN filled_data b
#         ON a.year = b.year AND a.age_in_years = b.age_in_years
#     LEFT JOIN total_yearly_losses c
#         ON a.year = c.year
#     LEFT JOIN filled_data_new d
#         ON a.year = d.year AND a.age_in_years = d.age_in_years
#     LEFT JOIN final_agg_closed e
#         ON a.year = e.year AND a.age_in_years = e.age_in_years
#     LEFT JOIN total_yearly_losses_closed f
#         ON a.year = f.year
#     LEFT JOIN total_yearly_new_orders g
#         ON a.year = g.year
#     LEFT JOIN total_yearly_active_caseloads l
#         ON a.year = l.year
#     ORDER BY 
#         a.year ASC, 
#         a.age_in_years ASC
# """)
# num_active_cases_historical

In [ ]:
# Forecasting using active_cases_historical and mortality_rate:
last_forecast_year = en_year+1
ages = range(st_age, en_age+1)
years = range(st_year, en_year+1)

dep_df = deputyship_num_active_cases_historical[[
    'year',
    'age_in_years',
    'num_active_caseloads',
    'num_new_orders',
    'mortality_rate',
    'total_new_orders',
    'total_active_caseloads',
    'total_losses_closed_terminated',
    'total_losses_closed',
    'total_losses'
]]

dep_df = dep_df.rename(columns={'age_in_years': 'age'})
dep_df = pd.DataFrame(dep_df)
dep_df
#for age, year in enumerate(years, ages):
    #num_active_caseload(year+1, age) = (num_active_caseload(year,age) + num_new_orders(year+1,age)) * mortality_rate(year,age)


# Initialisation
actives_dict = {}
new_orders_dict = {}
mort_rate_dict = {}
actives_df = pd.DataFrame()
orders_df = pd.DataFrame()
mort_rate_df = pd.DataFrame()


for year in list(years):
    aged_data = []
    for age in list(ages):
        
        active_caseloads = dep_df.loc[(dep_df['year'] == year) & (
                        dep_df['age'] == age),'num_active_caseloads'].values[0] 
        mortality_rates = dep_df.loc[(dep_df['year'] == year) & (
                        dep_df['age'] == age),'mortality_rate'].values[0]
        
        if year > last_forecast_year:
            new_orders = dep_df.loc[(dep_df['year'] == year+1) & (
                        dep_df['age'] == age),'num_new_orders'].values[0]
        else:
            new_orders = dep_df.loc[(dep_df['year'] == year) & (
                        dep_df['age'] == age),'num_new_orders'].values[0]
            
        forecasted_active_caseloads = round((active_caseloads+new_orders) - ((active_caseloads+new_orders)*mortality_rates))
        
        # Append the data for the current age to the yearly data list
        aged_data.append({'year': year+1, 
                            'age': age, 
                            'forecasted_active_caseloads': forecasted_active_caseloads})
    # Convert yearly data to a DataFrame and set multi-index for easy access
    actives_df = pd.DataFrame(aged_data).set_index(['year', 'age'])
    actives_df['total_losses'] = dep_df.loc[(dep_df['year'] == year
                                                              ) ,'total_losses'].values[0]
    actives_df = round(actives_df.fillna(0), 2)
    actives_df = actives_df.reset_index()
    actives_dict[year] = actives_df[['year', 'age', 'forecasted_active_caseloads', 'total_losses']]
            
        
forecasted_active_caseloads = pd.concat(actives_dict.values(), axis=0)

print(f"Deputyship forecasted values for Active Caseloads: {forecasted_active_caseloads}")
forecasted_active_caseloads.to_csv('forecasted_active_caseloads.csv', index=True)
#forecasted_active_caseloads_df = pd.concat(actives_dict.values(), axis=1)

#actives_dict[2025].reset_index()
# forecasted_active_caseloads.loc[(forecasted_active_caseloads['year'] == 2025
#                                 ) ,'total_losses_closed_terminated'].values[0]
# forecasted_active_caseloads.loc[(forecasted_active_caseloads['year'] == 2025
#                                 ) ,'total_losses_closed_terminated']
annual_forecasted_active_caseloads = forecasted_active_caseloads[['year','forecasted_active_caseloads']]
annual_forecasted_active_caseloads.groupby(['year']).sum(['forecasted_active_caseloads'])

# Next 
I am wondering whether we should simply start with this , though it would be useful to ask Phil 
what additional variables are additional available from Sirius that could be added to this . 
I am also wondering that if we cannot get termination rates from the Sirius data on the AP that we might impute these . 
If we request another cut of the active caseload directly from Sirius, we might compare this with the previous caseload 
we could identify who has left. If we also know the time period between the two we can scale the number of terminations 
to represent a one year period. This might be problematic initially as it seems that the attached caseload was downloaded 
in April 2023 so we cannot be sure of the age which someone actually left eg if they left in 2023 we won’t actually know 
if that was in 2023 , 2024 or more recently in 2025.    The only way that I can think of to get around that would be to 
use age categories rather than single years . This also wouldn’t be perfect but better than single years. 
Please can you have a think about this and whether it makes sense to simply use what seems easily available ? 
It may be that we can combine what we can derive from the Deputyships data on the AP with that which can directly be downloaded from Sirius . 
The alternative would be to simply use the termination rates previously used in the model as a temporary fix and improve these as 
we can take regular cuts of the caseload data directly from Sirius to infer the termination rates. 

In [ ]:
# num_active_cases_historical = pydbtools.read_sql_query(f"""
# WITH    
#     years AS (
#         SELECT year
#         FROM UNNEST(SEQUENCE(2022, 2024)) AS t(year)
#     ),
#     ages AS (
#         SELECT age_in_years
#         FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
#     ),
#     all_years_ages AS (
#         SELECT 
#             y.year, 
#             a.age_in_years
#         FROM years y
#         CROSS JOIN ages a
#     ),
#     processed_age_loss AS (
#         SELECT
#             YEAR(earliest_order_date) AS year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age)
#             END AS age_in_years,
#             termination_date
#         FROM (
#             SELECT
#                 eo.earliest_order_date,
#                 DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
#                 CASE
#                     WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                     WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                     WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                     ELSE NULL
#                 END AS termination_date
#             FROM opg_sirius_prod.persons p
#             LEFT JOIN opg_sirius_prod.death_notifications dn 
#                 ON dn.person_id = p.id
#                 AND dn.glueexporteddate = DATE('2025-02-17')
#             LEFT JOIN (
#                 SELECT 
#                     c.caserecnumber,
#                     MIN(c.orderdate) AS earliest_order_date,
#                     c.client_id
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.type = 'order'
#                   AND c.orderdate >= DATE('2024-01-01')
#                 GROUP BY c.caserecnumber, c.client_id
#             ) eo  
#                 ON eo.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     MIN(c.receiptdate) AS receiptdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.receiptdate IS NOT NULL
#                 GROUP BY c.client_id
#             ) er 
#                 ON er.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     c.statusdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#             ) rc 
#                 ON rc.client_id = p.id
#             WHERE p.glueexporteddate = DATE('2025-02-17')
#               AND p.type = 'actor_client'
#         )
#     ),
#     agg_loss_count AS (
#         SELECT 
#             year,
#             age_in_years,
#             COUNT(termination_date) AS num_losses
#         FROM processed_age_loss
#         GROUP BY year, age_in_years
#     ),
#     total_yearly_losses AS (
#         SELECT 
#             year,
#             SUM(num_losses) AS total_losses
#         FROM agg_loss_count
#         GROUP BY year
#     ),
#     final_agg AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             COALESCE(lc.num_losses, 0) AS num_losses
#         FROM all_years_ages a
#         LEFT JOIN agg_loss_count lc
#             ON a.year = lc.year
#             AND a.age_in_years = lc.age_in_years
#     ),
#     age_data AS (
#         SELECT 
#             c.caserecnumber,
#             MIN(c.orderdate) AS orderdate,
#             DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
#         FROM opg_sirius_prod.persons p
#         INNER JOIN opg_sirius_prod.cases c 
#             ON c.client_id = p.id
#             AND c.glueexporteddate = DATE('2025-02-17')
#         WHERE c.glueexporteddate = DATE('2025-02-17')  
#             AND c.type = 'order'
#         GROUP BY 
#             c.caserecnumber, 
#             p.dob
#     ),
#     processed_age_data AS (
#         SELECT
#             YEAR(a.orderdate) AS year,
#             CASE 
#                 WHEN FLOOR(a.raw_age) < 0 THEN 0
#                 ELSE ROUND(a.raw_age)
#             END AS age_in_years,
#             a.orderdate,
#             i.termination_date
#         FROM age_data a
#         LEFT JOIN processed_age_loss i
#             ON YEAR(a.orderdate) = i.year
#             AND a.raw_age = i.age_in_years
#         WHERE ((i.termination_date IS NULL) OR (YEAR(termination_date) > YEAR(earliest_order_date)))
#         --WHERE YEAR(orderdate) < YEAR(CURRENT_DATE) -- Orders that started before the given year
#         --AND (termination_date IS NULL OR termination_date >= DATE(CONCAT(YEAR(orderdate), '-01-01')))
#     ),
#     all_years_ages_orders AS (
#         SELECT 
#             y.year, 
#             a.age AS age_in_years
#         FROM 
#             (SELECT DISTINCT year FROM processed_age_data) y
#         CROSS JOIN 
#             (SELECT CAST(n AS INT) AS age 
#             FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
#     ),
#     aggregated_data AS (
#         SELECT 
#             year,
#             age_in_years,
#             COUNT(*) AS num_active_caseloads
#         FROM processed_age_data
#         GROUP BY year, age_in_years
#     ),
#     filled_data AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             COALESCE(ag.num_active_caseloads, 0) AS num_active_caseloads
#         FROM all_years_ages_orders a
#         LEFT JOIN aggregated_data ag 
#             ON a.year = ag.year AND a.age_in_years = ag.age_in_years
#     ),
#     age_data_new AS (
#         SELECT 
#             c.caserecnumber,
#             MIN(c.orderdate) AS earliestorderdate,
#             c.orderdate AS madedate,
#             DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age,
#             CASE
#                 WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                 WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                 WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                 ELSE NULL
#             END AS termination_date
#         FROM opg_sirius_prod.persons p
#         LEFT JOIN opg_sirius_prod.death_notifications dn 
#             ON dn.person_id = p.id
#             AND dn.glueexporteddate = DATE('2025-02-17')
#         LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     c.statusdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                     AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#             ) rc
#                 ON rc.client_id = p.id
#         INNER JOIN opg_sirius_prod.cases c 
#             ON c.client_id = p.id
#         WHERE c.glueexporteddate = DATE('2025-02-17')
#             AND c.type = 'order'
#             AND c.orderstatus = 'ACTIVE'
#             AND p.type = 'actor_client'
#         GROUP BY 
#             c.caserecnumber, 
#             p.dob,
#             c.orderdate,
#             p.dateofdeath,
#             p.clientstatus,
#             rc.statusdate,
#             dn.datenotified
#         HAVING c.orderdate >= MIN(c.orderdate)
#     ),
#     processed_age_data_new AS (
#         SELECT
#             YEAR(madedate) AS year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age)
#             END AS age_in_years,
#             caserecnumber
#         FROM age_data_new
#         WHERE termination_date IS NULL
#     ),
#     all_years_ages_orders_new AS (
#         SELECT 
#             y.year, 
#             a.age AS age_in_years
#         FROM 
#             (SELECT DISTINCT year FROM processed_age_data_new) y
#         CROSS JOIN 
#             (SELECT CAST(n AS INT) AS age 
#             FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
#     ),
#     aggregated_data_new AS (
#         SELECT 
#             year,
#             age_in_years,
#             COUNT(*) AS new_orders
#         FROM all_years_ages_orders_new
#         GROUP BY year, age_in_years
#     ),
#     filled_data_new AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             COALESCE(ag.new_orders, 0) AS num_new_orders
#         FROM all_years_ages_orders_new a
#         LEFT JOIN aggregated_data_new ag 
#             ON a.year = ag.year AND a.age_in_years = ag.age_in_years
#     ),
#     processed_age_loss_closed AS (
#         SELECT
#             year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age)
#             END AS age_in_years,
#             termination_date
#         FROM (
#             SELECT
#                 YEAR(eo.earliest_order_date) AS year,
#                 p.clientstatus,
#                 DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
#                 CASE
#                     WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                     WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                     WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                     ELSE NULL
#                 END AS termination_date
#             FROM opg_sirius_prod.persons p
#             LEFT JOIN opg_sirius_prod.death_notifications dn 
#                 ON dn.person_id = p.id
#                 AND dn.glueexporteddate = DATE('2025-02-17')
#             LEFT JOIN (
#                 SELECT 
#                     c.caserecnumber,
#                     MIN(c.orderdate) AS earliest_order_date,
#                     c.client_id
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.type = 'order'
#                 GROUP BY c.caserecnumber, c.client_id
#             ) eo  
#                 ON eo.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     MIN(c.receiptdate) AS receiptdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.receiptdate IS NOT NULL
#                   AND c.type = 'order'
#                 GROUP BY c.client_id
#             ) er 
#                 ON er.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     c.statusdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#             ) rc 
#                 ON rc.client_id = p.id
#             WHERE p.glueexporteddate = DATE('2025-02-17')
#               AND p.type = 'actor_client'
#               AND p.clientstatus = 'CLOSED'
#               AND CASE
#                     WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                     WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                     WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                     ELSE NULL
#                 END IS NULL
#                 )
#     ),
#     agg_loss_count_closed AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             --a.termination_date
#             COUNT(a.termination_date IS NULL) AS num_losses_closed
#         FROM processed_age_loss_closed a
#         GROUP BY a.year, a.age_in_years
#         ORDER BY a.year, a.age_in_years
#     ),
#     total_yearly_losses_closed AS (
#         SELECT 
#             year,
#             SUM(num_losses_closed) AS total_losses_closed
#         FROM agg_loss_count_closed
#         GROUP BY year
#     ),
#     final_agg_closed AS (
#         SELECT 
#             a.year,
#             a.age_in_years,
#             COALESCE(lc.num_losses_closed, 0) AS num_losses_closed
#         FROM all_years_ages a
#         LEFT JOIN agg_loss_count_closed lc
#             ON a.year = lc.year
#             AND a.age_in_years = lc.age_in_years
#         ORDER BY a.year, a.age_in_years
#     ),
#     total_yearly_active_caseloads AS (
#         SELECT 
#             year,
#             SUM(num_active_caseloads) AS total_active_caseloads
#         FROM filled_data
#         GROUP BY year
#     ),
#     total_yearly_new_orders AS (
#         SELECT 
#             year,
#             SUM(num_new_orders) AS total_new_orders
#         FROM filled_data_new
#         GROUP BY year
#     ),
#     processed_active_caseloads AS (
#         SELECT
#             YEAR(earliest_order_date) AS year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age)
#             END AS age_in_years,
#             termination_date
#         FROM (
#             SELECT
#                 eo.earliest_order_date,
#                 DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
#                 CASE
#                     WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                     WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                     WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                     ELSE NULL
#                 END AS termination_date
#             FROM opg_sirius_prod.persons p
#             LEFT JOIN opg_sirius_prod.death_notifications dn 
#                 ON dn.person_id = p.id
#                 AND dn.glueexporteddate = DATE('2025-02-17')
#             LEFT JOIN (
#                 SELECT 
#                     c.caserecnumber,
#                     MIN(c.orderdate) AS earliest_order_date,
#                     c.client_id
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.type = 'order'
#                   --AND c.orderdate >= DATE('2024-01-01')
#                 GROUP BY c.caserecnumber, c.client_id
#             ) eo  
#                 ON eo.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     MIN(c.receiptdate) AS receiptdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.receiptdate IS NOT NULL
#                 GROUP BY c.client_id
#             ) er 
#                 ON er.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     c.statusdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('2025-02-17')
#                   AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#             ) rc 
#                 ON rc.client_id = p.id
#             WHERE p.glueexporteddate = DATE('2025-02-17')
#               AND p.type = 'actor_client'
#         )
#         WHERE ((termination_date IS NULL) OR (YEAR(termination_date) > YEAR(earliest_order_date)))
#         --WHERE YEAR(orderdate) < YEAR(CURRENT_DATE) -- Orders that started before the given year
#         --AND (termination_date IS NULL OR termination_date >= DATE(CONCAT(YEAR(orderdate), '-01-01')))
#     ),
#     all_years_ages_active_caseloads_historical AS (
#         SELECT 
#             y.year, 
#             a.age AS age_in_years
#         FROM 
#             (SELECT DISTINCT year FROM processed_active_caseloads) y
#         CROSS JOIN 
#             (SELECT CAST(n AS INT) AS age 
#             FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
#     ),
#     aggregated_active_caseloads AS (
#         SELECT 
#             year,
#             age_in_years,
#             COUNT(*) AS num_active_caseloads_historical
#         FROM processed_active_caseloads
#         GROUP BY year, age_in_years
#     ), 
#     all_ages AS (
#         SELECT age
#         FROM UNNEST(SEQUENCE(0, 106)) AS t(age)  -- Ensuring all ages from 0 to 106 are covered
#     ), 
#     filled_aggregated_active_caseloads AS (
#         SELECT 
#             ad.year,
#             a.age AS age_in_years,
#             COALESCE(ad.num_active_caseloads_historical, 
#                 LAG(ad.num_active_caseloads_historical) OVER (ORDER BY a.age), 
#                 LEAD(ad.num_active_caseloads_historical) OVER (ORDER BY a.age)
#             ) AS num_active_caseloads_historical
#         FROM all_ages a
#         LEFT JOIN aggregated_active_caseloads ad 
#             ON a.age = ad.age_in_years
#     )
#     SELECT 
#         a.year,
#         a.age_in_years,
#         b.num_active_caseloads,
#         h.num_active_caseloads_historical,  -- Newly added column
#         d.num_new_orders,
#         a.num_losses,
#         e.num_losses_closed,
#         CAST((a.num_losses + e.num_losses_closed) AS DOUBLE) AS sum_losses,
#         CAST((b.num_active_caseloads - d.num_new_orders) AS DOUBLE) AS num_closed,
#         CAST(a.num_losses AS DOUBLE) / NULLIF((b.num_active_caseloads), 0) AS mortality_rate,
#         (CAST(a.num_losses AS DOUBLE) + CAST(e.num_losses_closed AS DOUBLE)) / NULLIF((b.num_active_caseloads), 0) AS sum_mortality_rate,
#         g.total_new_orders,
#         l.total_active_caseloads,
#         f.total_losses_closed,
#         c.total_losses,
#         CAST((c.total_losses + f.total_losses_closed) AS DOUBLE) AS total_losses_closed_terminated
#         --CAST(a.num_losses AS DOUBLE) / NULLIF(c.total_losses, 0) AS mortality_rate_aged,
#         --CAST(e.num_losses_closed AS DOUBLE) / NULLIF(f.total_losses_closed, 0) AS mortality_rate_aged_closed
#     FROM final_agg a
#     INNER JOIN filled_data b
#         ON a.year = b.year AND a.age_in_years = b.age_in_years
#     LEFT JOIN total_yearly_losses c
#         ON a.year = c.year
#     LEFT JOIN filled_data_new d
#         ON a.year = d.year AND a.age_in_years = d.age_in_years
#     LEFT JOIN final_agg_closed e
#         ON a.year = e.year AND a.age_in_years = e.age_in_years
#     LEFT JOIN total_yearly_losses_closed f
#         ON a.year = f.year
#     LEFT JOIN total_yearly_new_orders g
#         ON a.year = g.year
#     LEFT JOIN total_yearly_active_caseloads l
#         ON a.year = l.year
#     LEFT JOIN filled_aggregated_active_caseloads h
#         ON a.year = h.year AND a.age_in_years = h.age_in_years
#     ORDER BY 
#         a.year ASC, 
#         a.age_in_years ASC
# """)
# num_active_cases_historical

In [ ]:
QA_mortality_rate_num_new_orders_active = pydbtools.read_sql_query(f"""
WITH    
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE({st_year}, {en_year})) AS t(year)
    ),
    ages AS (
        SELECT age_in_years
        FROM UNNEST(SEQUENCE({st_age}, {en_age})) AS t(age_in_years)
    ),
    all_years_ages AS (
        SELECT 
            y.year, 
            a.age_in_years
        FROM years y
        CROSS JOIN ages a
    ),
    processed_age_loss AS (
        SELECT
            YEAR(earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date
        FROM (
            SELECT
                eo.earliest_order_date,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date
            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('{end_date}')
            LEFT JOIN (
                SELECT 
                    c.caserecnumber,
                    MIN(c.orderdate) AS earliest_order_date,
                    c.client_id
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.type = 'order'
                  AND c.orderdate >= DATE('{start_date}')
                GROUP BY c.caserecnumber, c.client_id
            ) eo  
                ON eo.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    MIN(c.receiptdate) AS receiptdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.receiptdate IS NOT NULL
                GROUP BY c.client_id
            ) er 
                ON er.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.statusdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
            ) rc 
                ON rc.client_id = p.id
            WHERE p.glueexporteddate = DATE('{end_date}')
              AND p.type = 'actor_client'
        )
    ),
    agg_loss_count AS (
        SELECT 
            year,
            age_in_years,
            COUNT(termination_date) AS num_losses
        FROM processed_age_loss
        GROUP BY year, age_in_years
    ),
    total_yearly_losses AS (
        SELECT 
            year,
            SUM(num_losses) AS total_losses
        FROM agg_loss_count
        GROUP BY year
    ),
    final_agg AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(lc.num_losses, 0) AS num_losses
        FROM all_years_ages a
        LEFT JOIN agg_loss_count lc
            ON a.year = lc.year
            AND a.age_in_years = lc.age_in_years
    ),
        age_data AS (
        SELECT 
            c.caserecnumber,
            MIN(c.orderdate) AS orderdate,
            DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
        FROM opg_sirius_prod.persons p
        INNER JOIN opg_sirius_prod.cases c 
            ON c.client_id = p.id
            AND c.glueexporteddate = DATE('{end_date}')
        WHERE c.glueexporteddate = DATE('{end_date}')  
            AND c.type = 'order'
        GROUP BY 
            c.caserecnumber, 
            p.dob
    ),
    processed_age_data AS (
        SELECT
            YEAR(orderdate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            orderdate
        FROM age_data
    ),
    all_years_ages_orders AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    aggregated_data AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS num_active_caseloads
        FROM processed_age_data
        GROUP BY year, age_in_years
    ),
    filled_data AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.num_active_caseloads, 0) AS num_active_caseloads
        FROM all_years_ages_orders a
        LEFT JOIN aggregated_data ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    ),
    age_data_new AS (
        SELECT 
        c.caserecnumber,
        MIN(c.orderdate) AS earliestorderdate,
        c.orderdate AS madedate,
        DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age,
        CASE
            WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
            WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
            WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
            ELSE NULL
        END AS termination_date
    FROM opg_sirius_prod.persons p
    LEFT JOIN opg_sirius_prod.death_notifications dn 
        ON dn.person_id = p.id
        AND dn.glueexporteddate = DATE('{end_date}')
    LEFT JOIN (
            SELECT 
                c.client_id,
                c.statusdate
            FROM opg_sirius_prod.cases c
            WHERE c.glueexporteddate = DATE('{end_date}')
                AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
        ) rc
            ON rc.client_id = p.id
    INNER JOIN opg_sirius_prod.cases c 
        ON c.client_id = p.id
    WHERE c.glueexporteddate = DATE('{end_date}')
        AND c.type = 'order'
        AND c.orderstatus = 'ACTIVE'
        AND p.type = 'actor_client'
    GROUP BY 
        c.caserecnumber, 
        p.dob,
        c.orderdate,
        p.dateofdeath,
        p.clientstatus,
        rc.statusdate,
        dn.datenotified
    HAVING c.orderdate >= MIN(c.orderdate)
    ),
    processed_age_data_new AS (
        SELECT
            YEAR(madedate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            caserecnumber
        FROM age_data_new
        WHERE termination_date IS NULL
    ),
    all_years_ages_orders_new AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data_new) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE({st_age}, {en_age})) AS t(n)) a
    ),
    aggregated_data_new AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS new_orders
        FROM processed_age_data_new
        GROUP BY year, age_in_years
    ),
    filled_data_new AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.new_orders, 0) AS num_new_orders
        FROM all_years_ages_orders_new a
        LEFT JOIN aggregated_data_new ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    ),
    processed_age_loss_closed AS (
        SELECT
            year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date
        FROM (
            SELECT
                YEAR(eo.earliest_order_date) AS year,
                p.clientstatus,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date
            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('{end_date}')
            LEFT JOIN (
                SELECT 
                    c.caserecnumber,
                    MIN(c.orderdate) AS earliest_order_date,
                    c.client_id
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.type = 'order'
                GROUP BY c.caserecnumber, c.client_id
            ) eo  
                ON eo.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    MIN(c.receiptdate) AS receiptdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.receiptdate IS NOT NULL
                  AND c.type = 'order'
                GROUP BY c.client_id
            ) er 
                ON er.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.statusdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
            ) rc 
                ON rc.client_id = p.id
            WHERE p.glueexporteddate = DATE('{end_date}')
              AND p.type = 'actor_client'
              AND p.clientstatus = 'CLOSED'
              AND CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END IS NULL
                )
    ),
    agg_loss_count_closed AS (
        SELECT 
            a.year,
            a.age_in_years,
            --a.termination_date
            COUNT(a.termination_date IS NULL) AS num_losses_closed
        FROM processed_age_loss_closed a
        GROUP BY a.year, a.age_in_years
        ORDER BY a.year, a.age_in_years
    ),
    total_yearly_losses_closed AS (
        SELECT 
            year,
            SUM(num_losses_closed) AS total_losses_closed
        FROM agg_loss_count_closed
        GROUP BY year
    ),
    final_agg_closed AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(lc.num_losses_closed, 0) AS num_losses_closed
        FROM all_years_ages a
        LEFT JOIN agg_loss_count_closed lc
            ON a.year = lc.year
            AND a.age_in_years = lc.age_in_years
        ORDER BY a.year, a.age_in_years
    ),
    total_yearly_active_caseloads AS (
        SELECT 
            year,
            SUM(num_active_caseloads) AS total_active_caseloads
        FROM filled_data
        GROUP BY year
    ),
    total_yearly_new_orders AS (
        SELECT 
            year,
            SUM(num_new_orders) AS total_new_orders
        FROM filled_data_new
        GROUP BY year
    )
    SELECT 
        a.year,
        a.age_in_years,
        b.num_active_caseloads,
        d.num_new_orders,
        a.num_losses,
        e.num_losses_closed,
        CAST((a.num_losses + e.num_losses_closed) AS DOUBLE) AS sum_losses,
        CAST((b.num_active_caseloads - d.num_new_orders) AS DOUBLE) AS num_closed,
        CAST(a.num_losses AS DOUBLE) / NULLIF((b.num_active_caseloads), 0) AS mortality_rate,
        (CAST(a.num_losses AS DOUBLE) + CAST(e.num_losses_closed AS DOUBLE)) / NULLIF((b.num_active_caseloads), 0) AS sum_mortality_rate,
        g.total_new_orders,
        l.total_active_caseloads,
        f.total_losses_closed,
        c.total_losses,
        CAST((c.total_losses + f.total_losses_closed) AS DOUBLE) AS total_losses_closed_terminated
        --CAST(a.num_losses AS DOUBLE) / NULLIF(c.total_losses, 0) AS mortality_rate_aged,
        --CAST(e.num_losses_closed AS DOUBLE) / NULLIF(f.total_losses_closed, 0) AS mortality_rate_aged_closed
    FROM final_agg a
    INNER JOIN filled_data b
        ON a.year = b.year AND a.age_in_years = b.age_in_years
    LEFT JOIN total_yearly_losses c
        ON a.year = c.year
    LEFT JOIN filled_data_new d
        ON a.year = d.year AND a.age_in_years = d.age_in_years
    LEFT JOIN final_agg_closed e
        ON a.year = e.year AND a.age_in_years = e.age_in_years
    LEFT JOIN total_yearly_losses_closed f
        ON a.year = f.year
    LEFT JOIN total_yearly_new_orders g
        ON a.year = g.year
    LEFT JOIN total_yearly_active_caseloads l
        ON a.year = l.year
    ORDER BY 
        a.year ASC, 
        a.age_in_years ASC
""")

QA_mortality_rate_num_new_orders_active.head(60)


In [ ]:
deputyship_mortality_rate_num_new_orders_active = QA_mortality_rate_num_new_orders_active
deputyship_mortality_rate_num_new_orders_active['mortality_rate'] = deputyship_mortality_rate_num_new_orders_active['mortality_rate'].fillna(0)
deputyship_mortality_rate_num_new_orders_active['sum_mortality_rate'] = deputyship_mortality_rate_num_new_orders_active['sum_mortality_rate'].fillna(0)
deputyship_mortality_rate_num_new_orders_active.to_csv('deputyship_mortality_rate_num_new_orders_active.csv', index=False)

# For forecasting the active caseloads:

## Active(t+1,age) = (Active(t,age) + New(t+1,age)) - (Active(t,age) + New(t+1,age)) * Losses(t,age)/Active(t)
- For each year:
    num_active_caseload(year+1, age) = num_active_caseload(year,age) + num_new_orders(year+1,age) - mortality_rate(year,age)
    

In [ ]:
# Forecasting using mortality_rate:
last_forecast_year = en_year+1
ages = range(st_age, en_age+1)
years = range(st_year, en_year+1)

dep_df = deputyship_mortality_rate_num_new_orders_active[[
    'year',
    'age_in_years',
    'num_active_caseloads',
    'num_new_orders',
    'mortality_rate',
    'total_new_orders',
    'total_active_caseloads',
    'total_losses_closed_terminated',
    'total_losses_closed',
    'total_losses'
]]

dep_df = dep_df.rename(columns={'age_in_years': 'age'})
dep_df = pd.DataFrame(dep_df)
dep_df
#for age, year in enumerate(years, ages):
    #num_active_caseload(year+1, age) = (num_active_caseload(year,age) + num_new_orders(year+1,age)) * mortality_rate(year,age)


# Initialisation
actives_dict = {}
new_orders_dict = {}
mort_rate_dict = {}
actives_df = pd.DataFrame()
orders_df = pd.DataFrame()
mort_rate_df = pd.DataFrame()


for year in list(years):
    aged_data = []
    for age in list(ages):
        
        active_caseloads = dep_df.loc[(dep_df['year'] == year) & (
                        dep_df['age'] == age),'num_active_caseloads'].values[0] 
        mortality_rates = dep_df.loc[(dep_df['year'] == year) & (
                        dep_df['age'] == age),'mortality_rate'].values[0]
        
        if year > last_forecast_year:
            new_orders = dep_df.loc[(dep_df['year'] == year+1) & (
                        dep_df['age'] == age),'num_new_orders'].values[0]
        else:
            new_orders = dep_df.loc[(dep_df['year'] == year) & (
                        dep_df['age'] == age),'num_new_orders'].values[0]
            
        forecasted_active_caseloads = round((active_caseloads+new_orders) - ((active_caseloads+new_orders)*mortality_rates))
        
        # Append the data for the current age to the yearly data list
        aged_data.append({'year': year+1, 
                            'age': age, 
                            'forecasted_active_caseloads': forecasted_active_caseloads})
    # Convert yearly data to a DataFrame and set multi-index for easy access
    actives_df = pd.DataFrame(aged_data).set_index(['year', 'age'])
    actives_df['total_losses'] = dep_df.loc[(dep_df['year'] == year
                                                              ) ,'total_losses'].values[0]
    actives_df = round(actives_df.fillna(0), 2)
    actives_df = actives_df.reset_index()
    actives_dict[year] = actives_df[['year', 'age', 'forecasted_active_caseloads', 'total_losses']]
            
        
forecasted_active_caseloads = pd.concat(actives_dict.values(), axis=0)

print(f"Deputyship forecasted values for Active Caseloads: {forecasted_active_caseloads}")
forecasted_active_caseloads.to_csv('forecasted_active_caseloads.csv', index=True)
#forecasted_active_caseloads_df = pd.concat(actives_dict.values(), axis=1)

#actives_dict[2025].reset_index()
# forecasted_active_caseloads.loc[(forecasted_active_caseloads['year'] == 2025
#                                 ) ,'total_losses_closed_terminated'].values[0]
# forecasted_active_caseloads.loc[(forecasted_active_caseloads['year'] == 2025
#                                 ) ,'total_losses_closed_terminated']
annual_forecasted_active_caseloads = forecasted_active_caseloads[['year','forecasted_active_caseloads']]
annual_forecasted_active_caseloads.groupby(['year']).sum(['forecasted_active_caseloads'])

In [ ]:
# Forecasting using sum_mortality_rate:
last_forecast_year = en_year+1
ages = range(st_age, en_age+1)
years = range(st_year, en_year+1)

dep_df = deputyship_mortality_rate_num_new_orders_active[[
    'year',
    'age_in_years',
    'num_active_caseloads',
    'num_new_orders',
    'sum_mortality_rate',
    'total_new_orders',
    'total_active_caseloads',
    'total_losses_closed_terminated',
    'total_losses_closed',
    'total_losses'
]]

dep_df = dep_df.rename(columns={'age_in_years': 'age'})
dep_df = pd.DataFrame(dep_df)
dep_df
#for age, year in enumerate(years, ages):
    #num_active_caseload(year+1, age) = (num_active_caseload(year,age) + num_new_orders(year+1,age)) * mortality_rate(year,age)


# Initialisation
actives_dict = {}
new_orders_dict = {}
mort_rate_dict = {}
actives_df = pd.DataFrame()
orders_df = pd.DataFrame()
mort_rate_df = pd.DataFrame()


for year in list(years):
    aged_data = []
    for age in list(ages):
        
        active_caseloads = dep_df.loc[(dep_df['year'] == year) & (
                        dep_df['age'] == age),'num_active_caseloads'].values[0] 
        mortality_rates = dep_df.loc[(dep_df['year'] == year) & (
                        dep_df['age'] == age),'sum_mortality_rate'].values[0]
        
        if year > last_forecast_year:
            new_orders = dep_df.loc[(dep_df['year'] == year+1) & (
                        dep_df['age'] == age),'num_new_orders'].values[0]
        else:
            new_orders = dep_df.loc[(dep_df['year'] == year) & (
                        dep_df['age'] == age),'num_new_orders'].values[0]
            
        forecasted_active_caseloads = round((active_caseloads+new_orders) - ((active_caseloads+new_orders)*mortality_rates))
        
        # Append the data for the current age to the yearly data list
        aged_data.append({'year': year+1, 
                            'age': age, 
                            'forecasted_active_caseloads': forecasted_active_caseloads})
    # Convert yearly data to a DataFrame and set multi-index for easy access
    actives_df = pd.DataFrame(aged_data).set_index(['year', 'age'])
    actives_df['total_losses_closed_terminated'] = dep_df.loc[(dep_df['year'] == year
                                                              ) ,'total_losses_closed_terminated'].values[0]
    actives_df = round(actives_df.fillna(0), 2)
    actives_df = actives_df.reset_index()
    actives_dict[year] = actives_df[['year', 'age', 'forecasted_active_caseloads', 'total_losses_closed_terminated']]
            
        
forecasted_active_caseloads = pd.concat(actives_dict.values(), axis=0)

print(f"Deputyship forecasted values for Active Caseloads: {forecasted_active_caseloads}")
forecasted_active_caseloads.to_csv('forecasted_active_caseloads.csv', index=True)
#forecasted_active_caseloads_df = pd.concat(actives_dict.values(), axis=1)

#actives_dict[2025].reset_index()
# forecasted_active_caseloads.loc[(forecasted_active_caseloads['year'] == 2025
#                                 ) ,'total_losses_closed_terminated'].values[0]
# forecasted_active_caseloads.loc[(forecasted_active_caseloads['year'] == 2025
#                                 ) ,'total_losses_closed_terminated']
annual_forecasted_active_caseloads = forecasted_active_caseloads[['year','forecasted_active_caseloads']]
annual_forecasted_active_caseloads.groupby(['year']).sum(['forecasted_active_caseloads'])

# total number of Active caseloads under 16 years old

In [ ]:
df = dep_df
# Step 1: Compute total active caseloads per year
total_caseloads = df.groupby("year")["num_active_caseloads"].sum().reset_index()
total_caseloads.rename(columns={"num_active_caseloads": "total_cases"}, inplace=True)

# Step 2: Compute active caseloads for age < 16
under_16_caseloads = df[df["age"] < 16].groupby("year")["num_active_caseloads"].sum().reset_index()
under_16_caseloads.rename(columns={"num_active_caseloads": "under_16_cases"}, inplace=True)

# Step 3: Merge both dataframes
merged_df = pd.merge(total_caseloads, under_16_caseloads, on="year", how="left").fillna(0)

# Step 4: Compute percentage of under-16 caseloads
merged_df["percentage_under_16"] = (merged_df["under_16_cases"] / merged_df["total_cases"]) * 100

# Display result
print(merged_df)

# total number of new orders under 16 years old

In [ ]:
df = dep_df
# Step 1: Compute total active caseloads per year
total_caseloads = df.groupby("year")["num_new_orders"].sum().reset_index()
total_caseloads.rename(columns={"num_new_orders": "total_cases"}, inplace=True)

# Step 2: Compute active caseloads for age < 16
under_16_caseloads = df[df["age"] < 16].groupby("year")["num_new_orders"].sum().reset_index()
under_16_caseloads.rename(columns={"num_new_orders": "under_16_cases"}, inplace=True)

# Step 3: Merge both dataframes
merged_df = pd.merge(total_caseloads, under_16_caseloads, on="year", how="left").fillna(0)

# Step 4: Compute percentage of under-16 caseloads
merged_df["percentage_under_16"] = (merged_df["under_16_cases"] / merged_df["total_cases"]) * 100

# Display result
print(merged_df)

# average deputyship length
This query computes the average deputyship length by calculating the time difference between the earliest made date and the termination date for each case, then averaging those values.

In [ ]:
average_length_deputyship = pydbtools.read_sql_query(f"""
WITH deputyship_durations AS (
    SELECT 
        c.caserecnumber,
        MIN(c.orderdate) AS earliest_order_date,
        CASE 
            WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
            WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
            WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
            ELSE NULL
        END AS termination_date,
        DATE_DIFF('day', MIN(c.orderdate), 
            CASE 
                WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                ELSE NULL
            END) / 365.25 AS duration_in_years
    FROM opg_sirius_prod.persons p
    LEFT JOIN opg_sirius_prod.death_notifications dn 
        ON dn.person_id = p.id 
        AND dn.glueexporteddate = DATE('{end_date}')
    LEFT JOIN (
        SELECT 
            c.client_id,
            c.statusdate
        FROM opg_sirius_prod.cases c
        WHERE c.glueexporteddate = DATE('{end_date}')
            AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
    ) rc 
        ON rc.client_id = p.id
    INNER JOIN opg_sirius_prod.cases c 
        ON c.client_id = p.id
    WHERE c.glueexporteddate = DATE('{end_date}')
        AND c.type = 'order'
        AND p.type = 'actor_client'
    GROUP BY 
        c.caserecnumber, 
        p.dateofdeath, 
        p.clientstatus, 
        rc.statusdate, 
        dn.datenotified  -- ✅ Fixed column name
) 
SELECT AVG(duration_in_years) AS avg_deputyship_length
FROM deputyship_durations
WHERE termination_date IS NOT NULL
""")

average_length_deputyship


# Computes the average deputyship length per age group (0 to 106)
# Ensures no missing ages by:
- Using SEQUENCE(0, 106) to create a full list of ages
- Filling missing values using LAG (previous age) or LEAD (next age)
# Keeps the core logic intact for termination dates and durations

In [ ]:
average_length_deputyship_aged = pydbtools.read_sql_query(f"""
WITH deputyship_durations AS (
    SELECT 
        c.caserecnumber,
        YEAR(MIN(c.orderdate)) AS start_year,
        FLOOR(DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25) AS age_at_start,
        CASE 
            WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
            WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
            WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
            ELSE NULL
        END AS termination_date,
        DATE_DIFF('day', MIN(c.orderdate), 
            CASE 
                WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                ELSE NULL
            END) / 365.25 AS duration_in_years
    FROM opg_sirius_prod.persons p
    LEFT JOIN opg_sirius_prod.death_notifications dn 
        ON dn.person_id = p.id 
        AND dn.glueexporteddate = DATE('{end_date}')
    LEFT JOIN (
        SELECT 
            c.client_id,
            c.statusdate
        FROM opg_sirius_prod.cases c
        WHERE c.glueexporteddate = DATE('{end_date}')
            AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
    ) rc 
        ON rc.client_id = p.id
    INNER JOIN opg_sirius_prod.cases c 
        ON c.client_id = p.id
    WHERE c.glueexporteddate = DATE('{end_date}')
        AND c.type = 'order'
        AND p.type = 'actor_client'
    GROUP BY 
        c.caserecnumber, 
        p.dob, 
        p.dateofdeath, 
        p.clientstatus, 
        rc.statusdate, 
        dn.datenotified
), 

average_duration_per_age AS (
    SELECT 
        age_at_start AS age_in_years,
        AVG(duration_in_years) AS avg_deputyship_length
    FROM deputyship_durations
    WHERE termination_date IS NOT NULL
    GROUP BY age_at_start
), 

all_ages AS (
    SELECT age
    FROM UNNEST(SEQUENCE(0, 106)) AS t(age)  -- Ensuring all ages from 0 to 106 are covered
), 

filled_average_durations AS (
    SELECT 
        a.age AS age_in_years,
        COALESCE(ad.avg_deputyship_length, 
            LAG(ad.avg_deputyship_length) OVER (ORDER BY a.age), 
            LEAD(ad.avg_deputyship_length) OVER (ORDER BY a.age)
        ) AS avg_deputyship_length
    FROM all_ages a
    LEFT JOIN average_duration_per_age ad 
        ON a.age = ad.age_in_years
)

SELECT * FROM filled_average_durations
ORDER BY age_in_years
""")



print(f"Deputyship Average Length from the earliest made date and termination date: {average_length_deputyship_aged}")
average_length_deputyship_aged.to_csv('average_length_deputyship_aged.csv', index=True)

In [ ]:
forecasted_years = range(st_year+1, last_forecast_year)
forecast_actuals_dep_df = dep_df 
forecast_actuals_dep_df['forecasted_active'] = forecast_actuals_dep_df['num_active_caseloads']
for year in list(forecasted_years):
    df = pd.DataFrame()
    df1 = pd.DataFrame()
    for age in list(ages):
        df = actives_dict[year].reset_index()
        df1 = df.loc[df['year'] == year+1]
        forecast_actuals_dep_df.loc[(forecast_actuals_dep_df['year'] == year+1) & (
                                forecast_actuals_dep_df['age'] == 0), 
                                    'forecasted_active'] = df1.loc[(
            df1['age'] == 0)]['forecasted_active_caseloads']
        
forecast_actuals_dep_df        

In [ ]:
forecasted_years = range(st_year+1, last_forecast_year)
forecast_actuals_dep_df = dep_df 
#forecast_actuals_dep_df['forecasted_active'] = forecast_actuals_dep_df['num_active_caseloads']

forecast_actuals_dep_df.loc[(
    forecast_actuals_dep_df['year'] == last_forecast_year),
    'forecasted_active'] = 0


for year in list(forecasted_years):
    df = pd.DataFrame()
    df1 = pd.DataFrame()
    for age in list(ages):
        if year <= last_forecast_year:
            df = actives_dict[year].reset_index()
            df1 = df.loc[df['year'] == year+1]
            forecast_actuals_dep_df.loc[(forecast_actuals_dep_df['year'] == year+1) & (
                                    forecast_actuals_dep_df['age'] == 0), 
                                        'forecasted_active'] = df1.loc[(
                df1['age'] == 0)]['forecasted_active_caseloads']
        # else:
        #     .append({'year': year+1, 
        #                     'age': age, 
        #                     'forecasted_active_caseloads': 0,
        #                      'forecasted_active_caseloads': 0})

In [ ]:
forecast_actuals_dep_df.loc[(
    forecast_actuals_dep_df['year'] == last_forecast_year),
    'forecasted_active'] = 0
forecast_actuals_dep_df

In [ ]:
Deputyship_mortality_rate_num_new_orders_active = pydbtools.read_sql_query(f"""
WITH    
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE(2017, 2018)) AS t(year)
    ),
    ages AS (
        SELECT age_in_years
        FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
    ),
    all_years_ages AS (
        SELECT 
            y.year, 
            a.age_in_years
        FROM years y
        CROSS JOIN ages a
    ),
    processed_age_loss AS (
        SELECT
            YEAR(earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date
        FROM (
            SELECT
                eo.earliest_order_date,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date
            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('2024-12-31')
            LEFT JOIN (
                SELECT 
                    c.caserecnumber,
                    MIN(c.orderdate) AS earliest_order_date,
                    c.client_id
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('2024-12-31')
                  AND c.type = 'order'
                  AND c.orderdate >= DATE('2017-01-01')
                GROUP BY c.caserecnumber, c.client_id
            ) eo  
                ON eo.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    MIN(c.receiptdate) AS receiptdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('2024-12-31')
                  AND c.receiptdate IS NOT NULL
                GROUP BY c.client_id
            ) er 
                ON er.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.statusdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('2024-12-31')
                  AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
            ) rc 
                ON rc.client_id = p.id
            WHERE p.glueexporteddate = DATE('2024-12-31')
              AND p.type = 'actor_client'
        )
    ),
    agg_loss_count AS (
        SELECT 
            year,
            age_in_years,
            COUNT(termination_date) AS num_losses
        FROM processed_age_loss
        GROUP BY year, age_in_years
    ),
    total_yearly_losses AS (
        SELECT 
            year,
            SUM(num_losses) AS total_losses
        FROM agg_loss_count
        GROUP BY year
    ),
    final_agg AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(lc.num_losses, 0) AS num_losses
        FROM all_years_ages a
        LEFT JOIN agg_loss_count lc
            ON a.year = lc.year
            AND a.age_in_years = lc.age_in_years
    ),
    age_data AS (
        SELECT 
            c.caserecnumber,
            MIN(c.orderdate) AS orderdate,
            DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
        FROM opg_sirius_prod.persons p
        INNER JOIN opg_sirius_prod.cases c 
            ON c.client_id = p.id
            AND c.glueexporteddate = DATE('2024-12-31')
        WHERE c.glueexporteddate = DATE('2024-12-31')  
            AND c.type = 'order'
            AND c.orderdate >= DATE('2017-01-01')
        GROUP BY 
            c.caserecnumber, 
            p.dob
    ),
    processed_age_data AS (
        SELECT
            YEAR(orderdate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            orderdate
        FROM age_data
    ),
    all_years_ages_orders AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    aggregated_data AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS num_active_caseloads
        FROM processed_age_data
        GROUP BY year, age_in_years
    ),
    filled_data AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.num_active_caseloads, 0) AS num_active_caseloads
        FROM all_years_ages_orders a
        LEFT JOIN aggregated_data ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    ),
    age_data_new AS (
        SELECT 
        c.caserecnumber,
        MIN(c.orderdate) AS earliestorderdate,
        c.orderdate AS madedate,
        DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age,
        CASE
            WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
            WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
            WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
            ELSE NULL
        END AS termination_date
    FROM opg_sirius_prod.persons p
    LEFT JOIN opg_sirius_prod.death_notifications dn 
        ON dn.person_id = p.id
        AND dn.glueexporteddate = DATE('2024-12-31')
    LEFT JOIN (
            SELECT 
                c.client_id,
                c.statusdate
            FROM opg_sirius_prod.cases c
            WHERE c.glueexporteddate = DATE('2024-12-31')
                AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
        ) rc
            ON rc.client_id = p.id
    INNER JOIN opg_sirius_prod.cases c 
        ON c.client_id = p.id
    WHERE c.glueexporteddate = DATE('2024-12-31')
        AND c.type = 'order'
        AND c.orderstatus = 'ACTIVE'
        AND c.orderdate >= DATE('2017-01-01')
        AND p.type = 'actor_client'
    GROUP BY 
        c.caserecnumber, 
        p.dob,
        c.orderdate,
        p.dateofdeath,
        p.clientstatus,
        rc.statusdate,
        dn.datenotified
    ),
    processed_age_data_new AS (
        SELECT
            YEAR(madedate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            caserecnumber
        FROM age_data_new
        WHERE termination_date IS NULL
    ),
    all_years_ages_orders_new AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data_new) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    aggregated_data_new AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS new_orders
        FROM processed_age_data_new
        GROUP BY year, age_in_years
    ),
    filled_data_new AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.new_orders, 0) AS num_new_orders
        FROM all_years_ages_orders_new a
        LEFT JOIN aggregated_data_new ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    )    
    SELECT 
        a.year,
        a.age_in_years,
        a.num_losses,
        b.num_active_caseloads,
        d.num_new_orders,
        CAST((b.num_active_caseloads - d.num_new_orders) AS DOUBLE) AS num_closed,
        CAST(a.num_losses AS DOUBLE) / NULLIF((b.num_active_caseloads), 0) AS mortality_rate,
        CAST(a.num_losses AS DOUBLE) / NULLIF(c.total_losses, 0) AS mortality_rate_aged
    FROM final_agg a
    INNER JOIN filled_data b
        ON a.year = b.year AND a.age_in_years = b.age_in_years
    LEFT JOIN total_yearly_losses c
        ON a.year = c.year
    LEFT JOIN filled_data_new d
        ON a.year = d.year AND a.age_in_years = d.age_in_years
    ORDER BY 
        a.year ASC, 
        a.age_in_years ASC
""")

Deputyship_mortality_rate_num_new_orders_active


In [ ]:
# Total

Deputyship_total = pydbtools.read_sql_query(f"""
WITH    
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE(2017, 2018)) AS t(year)
    ),
    ages AS (
        SELECT age_in_years
        FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
    ),
    all_years_ages AS (
        SELECT 
            y.year, 
            a.age_in_years
        FROM years y
        CROSS JOIN ages a
    ),
    processed_age_loss AS (
        SELECT
            YEAR(earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date
        FROM (
            SELECT
                eo.earliest_order_date,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date
            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('2024-12-31')
            LEFT JOIN (
                SELECT 
                    c.caserecnumber,
                    MIN(c.orderdate) AS earliest_order_date,
                    c.client_id
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('2024-12-31')
                  AND c.type = 'order'
                  AND c.orderdate >= DATE('2017-01-01')
                GROUP BY c.caserecnumber, c.client_id
            ) eo  
                ON eo.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    MIN(c.receiptdate) AS receiptdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('2024-12-31')
                  AND c.receiptdate IS NOT NULL
                GROUP BY c.client_id
            ) er 
                ON er.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.statusdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('2024-12-31')
                  AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
            ) rc 
                ON rc.client_id = p.id
            WHERE p.glueexporteddate = DATE('2024-12-31')
              AND p.type = 'actor_client'
        )
    ),
    agg_loss_count AS (
        SELECT 
            year,
            age_in_years,
            termination_date
            --COUNT(termination_date) AS num_losses
        FROM processed_age_loss
        GROUP BY year, age_in_years, termination_date
    ),
    total_yearly_losses AS (
        SELECT 
            year,
            termination_date
            --SUM(COUNT(termination_date)) AS total_losses
        FROM agg_loss_count
        GROUP BY year, termination_date
    ),
    final_agg AS (
        SELECT 
            a.year,
            a.age_in_years,
            termination_date
            --COALESCE(lc.num_losses, 0) AS num_losses
        FROM all_years_ages a
        LEFT JOIN agg_loss_count lc
            ON a.year = lc.year
            AND a.age_in_years = lc.age_in_years
    ),
    age_data AS (
        SELECT 
            c.caserecnumber,
            MIN(c.orderdate) AS orderdate,
            DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
        FROM opg_sirius_prod.persons p
        INNER JOIN opg_sirius_prod.cases c 
            ON c.client_id = p.id
            AND c.glueexporteddate = DATE('2024-12-31')
        WHERE c.glueexporteddate = DATE('2024-12-31')  
            AND c.type = 'order'
            AND c.orderdate >= DATE('2017-01-01')
        GROUP BY 
            c.caserecnumber, 
            p.dob
    ),
    processed_age_data AS (
        SELECT
            YEAR(orderdate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            orderdate,
            caserecnumber
        FROM age_data
    ),
    all_years_ages_orders AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    aggregated_data AS (
        SELECT 
            year,
            age_in_years,
            caserecnumber
            --COUNT(*) AS num_active_caseloads
        FROM processed_age_data
        --GROUP BY year, age_in_years, caserecnumber
    ),
    filled_data AS (
        SELECT 
            a.year,
            a.age_in_years,
            caserecnumber
            --COALESCE(ag.num_active_caseloads, 0) AS num_active_caseloads
        FROM all_years_ages_orders a
        LEFT JOIN aggregated_data ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    ),
    age_data_new AS (
        SELECT 
        c.caserecnumber,
        MIN(c.orderdate) AS earliestorderdate,
        c.orderdate AS madedate,
        DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age,
        CASE
            WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
            WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
            WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
            ELSE NULL
        END AS termination_date,
        c.orderstatus,
        p.type,
        c.client_id,
        c.statusdate,
        c.orderclosurereason,
        dn.datenotified
    FROM opg_sirius_prod.persons p
    LEFT JOIN opg_sirius_prod.death_notifications dn 
        ON dn.person_id = p.id
        AND dn.glueexporteddate = DATE('2024-12-31')
    LEFT JOIN (
            SELECT 
                c.client_id,
                c.statusdate
            FROM opg_sirius_prod.cases c
            WHERE c.glueexporteddate = DATE('2024-12-31')
                AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
        ) rc
            ON rc.client_id = p.id
    INNER JOIN opg_sirius_prod.cases c 
        ON c.client_id = p.id
    WHERE c.glueexporteddate = DATE('2024-12-31')
        AND c.type = 'order'
        AND c.orderstatus = 'ACTIVE'
        AND c.orderdate >= DATE('2017-01-01')
        AND p.type = 'actor_client'
    GROUP BY 
        c.caserecnumber, 
        p.dob,
        c.orderdate,
        p.dateofdeath,
        p.clientstatus,
        rc.statusdate,
        c.statusdate,
        dn.datenotified,
        c.orderstatus,
        p.type,
        c.client_id,
        c.orderclosurereason
    ),
    processed_age_data_new AS (
        SELECT
            YEAR(madedate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date,
            caserecnumber,
            madedate,
            orderstatus,
            type,
            client_id,
            statusdate,
            orderclosurereason,
            datenotified
        FROM age_data_new
        WHERE termination_date IS NULL
    ),
    all_years_ages_orders_new AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data_new) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    aggregated_data_new AS (
        SELECT 
            year,
            age_in_years,
            caserecnumber,
            termination_date,
            madedate,
            orderstatus,
            type,
            client_id,
            statusdate,
            orderclosurereason,
            datenotified
        FROM processed_age_data_new
    ),
    filled_data_new AS (
        SELECT ag.*
        FROM all_years_ages_orders_new a
        LEFT JOIN aggregated_data_new ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    )    
    SELECT 
        a.year,
        a.age_in_years,
        d.caserecnumber,
        d.termination_date,
        d.madedate,
        d.orderstatus,
        d.type,
        d.client_id,
        d.statusdate,
        d.orderclosurereason,
        d.datenotified
    FROM final_agg a
    INNER JOIN filled_data b
        ON a.year = b.year AND a.age_in_years = b.age_in_years
    LEFT JOIN filled_data_new d
        ON a.year = d.year AND a.age_in_years = d.age_in_years
    LEFT JOIN total_yearly_losses c
        ON a.year = c.year
    ORDER BY 
        a.year ASC, 
        a.age_in_years ASC
""")

Deputyship_total


In [ ]:
Deputyship_mortality_rate_num_new_orders_active = pydbtools.read_sql_query(f"""
WITH    
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE(2017, 2018)) AS t(year)
    ),
    ages AS (
        SELECT age_in_years
        FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
    ),
    all_years_ages AS (
        SELECT 
            y.year, 
            a.age_in_years
        FROM years y
        CROSS JOIN ages a
    ),
    processed_age_loss AS (
        SELECT
            YEAR(earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date
        FROM (
            SELECT
                eo.earliest_order_date,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date
            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('2024-12-31')
            LEFT JOIN (
                SELECT 
                    c.caserecnumber,
                    MIN(c.orderdate) AS earliest_order_date,
                    c.client_id
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('2024-12-31')
                  AND c.type = 'order'
                  AND c.orderdate >= DATE('2017-01-01')
                GROUP BY c.caserecnumber, c.client_id
            ) eo  
                ON eo.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    MIN(c.receiptdate) AS receiptdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('2024-12-31')
                  AND c.receiptdate IS NOT NULL
                GROUP BY c.client_id
            ) er 
                ON er.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.statusdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('2024-12-31')
                  AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
            ) rc 
                ON rc.client_id = p.id
            WHERE p.glueexporteddate = DATE('2024-12-31')
              AND p.type = 'actor_client'
        )
    ),
    agg_loss_count AS (
        SELECT 
            year,
            age_in_years,
            COUNT(termination_date) AS num_losses
        FROM processed_age_loss
        GROUP BY year, age_in_years
    ),
    total_yearly_losses AS (
        SELECT 
            year,
            SUM(num_losses) AS total_losses
        FROM agg_loss_count
        GROUP BY year
    ),
    final_agg AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(lc.num_losses, 0) AS num_losses
        FROM all_years_ages a
        LEFT JOIN agg_loss_count lc
            ON a.year = lc.year
            AND a.age_in_years = lc.age_in_years
    ),
    age_data AS (
        SELECT 
            c.caserecnumber,
            MIN(c.orderdate) AS orderdate,
            DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
        FROM opg_sirius_prod.persons p
        INNER JOIN opg_sirius_prod.cases c 
            ON c.client_id = p.id
            AND c.glueexporteddate = DATE('2024-12-31')
        WHERE c.glueexporteddate = DATE('2024-12-31')  
            AND c.type = 'order'
            AND c.orderdate >= DATE('2017-01-01')
        GROUP BY 
            c.caserecnumber, 
            p.dob
    ),
    processed_age_data AS (
        SELECT
            YEAR(orderdate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            orderdate
        FROM age_data
    ),
    all_years_ages_orders AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    aggregated_data AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS num_active_caseloads
        FROM processed_age_data
        GROUP BY year, age_in_years
    ),
    filled_data AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.num_active_caseloads, 0) AS num_active_caseloads
        FROM all_years_ages_orders a
        LEFT JOIN aggregated_data ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    ),
    age_data_new AS (
        SELECT 
        c.caserecnumber,
        MIN(c.orderdate) AS earliestorderdate,
        c.orderdate AS madedate,
        DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age,
        CASE
            WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
            WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
            WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
            ELSE NULL
        END AS termination_date
    FROM opg_sirius_prod.persons p
    LEFT JOIN opg_sirius_prod.death_notifications dn 
        ON dn.person_id = p.id
        AND dn.glueexporteddate = DATE('2024-12-31')
    LEFT JOIN (
            SELECT 
                c.client_id,
                c.statusdate
            FROM opg_sirius_prod.cases c
            WHERE c.glueexporteddate = DATE('2024-12-31')
                AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
        ) rc
            ON rc.client_id = p.id
    INNER JOIN opg_sirius_prod.cases c 
        ON c.client_id = p.id
    WHERE c.glueexporteddate = DATE('2024-12-31')
        AND c.type = 'order'
        AND c.orderstatus = 'ACTIVE'
        AND c.orderdate >= DATE('2017-01-01')
        AND p.type = 'actor_client'
    GROUP BY 
        c.caserecnumber, 
        p.dob,
        c.orderdate,
        p.dateofdeath,
        p.clientstatus,
        rc.statusdate,
        dn.datenotified
    ),
    processed_age_data_new AS (
        SELECT
            YEAR(madedate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            caserecnumber
        FROM age_data_new
        WHERE termination_date IS NULL
    ),
    all_years_ages_orders_new AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data_new) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    aggregated_data_new AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS new_orders
        FROM processed_age_data_new
        GROUP BY year, age_in_years
    ),
    filled_data_new AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.new_orders, 0) AS num_new_orders
        FROM all_years_ages_orders_new a
        LEFT JOIN aggregated_data_new ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    )    
    SELECT 
        a.year,
        a.age_in_years,
        a.num_losses,
        b.num_active_caseloads,
        d.num_new_orders,
        CAST((b.num_active_caseloads - d.num_new_orders) AS DOUBLE) AS num_closed,
        CAST(a.num_losses AS DOUBLE) / NULLIF((b.num_active_caseloads), 0) AS mortality_rate,
        CAST(a.num_losses AS DOUBLE) / NULLIF(c.total_losses, 0) AS mortality_rate_aged
    FROM final_agg a
    INNER JOIN filled_data b
        ON a.year = b.year AND a.age_in_years = b.age_in_years
    LEFT JOIN total_yearly_losses c
        ON a.year = c.year
    LEFT JOIN filled_data_new d
        ON a.year = d.year AND a.age_in_years = d.age_in_years
    ORDER BY 
        a.year ASC, 
        a.age_in_years ASC
""")

Deputyship_mortality_rate_num_new_orders_active


In [ ]:
Deputyship_mortality_rate_num_new_orders_active = pydbtools.read_sql_query(f"""
WITH 
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE({st_year}, {st_year})) AS t(year)
    ),
    ages AS (
        SELECT age_in_years
        FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
    ),
    all_years_ages AS (
        SELECT 
            y.year, 
            a.age_in_years
        FROM years y
        CROSS JOIN ages a
    ),
    processed_age_loss AS (
        SELECT
            YEAR(earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date
        FROM (
            SELECT
                eo.earliest_order_date,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date
            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('{end_date}')
            LEFT JOIN (
                SELECT 
                    c.caserecnumber,
                    MIN(c.orderdate) AS earliest_order_date,
                    c.client_id
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.type = 'order'
                  AND c.orderdate >= DATE('{start_date}')
                GROUP BY c.caserecnumber, c.client_id
            ) eo  
                ON eo.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    MIN(c.receiptdate) AS receiptdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.receiptdate IS NOT NULL
                GROUP BY c.client_id
            ) er 
                ON er.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.statusdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
            ) rc 
                ON rc.client_id = p.id
            WHERE p.glueexporteddate = DATE('{end_date}')
              AND p.type = 'actor_client'
        )
    ),
    agg_loss_count AS (
        SELECT 
            year,
            age_in_years,
            COUNT(termination_date) AS num_losses
        FROM processed_age_loss
        GROUP BY year, age_in_years
    ),
    
    total_yearly_losses AS (
        SELECT 
            year,
            SUM(num_losses) AS total_losses
        FROM agg_loss_count
        GROUP BY year
    ),
    final_agg AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(lc.num_losses, 0) AS num_losses
        FROM all_years_ages a
        LEFT JOIN agg_loss_count lc
            ON a.year = lc.year
            AND a.age_in_years = lc.age_in_years
    ),
    age_data AS (
        SELECT 
            c.caserecnumber,
            MIN(c.orderdate) AS orderdate,
            DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
        FROM opg_sirius_prod.persons p
        INNER JOIN opg_sirius_prod.cases c 
            ON c.client_id = p.id
            AND c.glueexporteddate = DATE('{str(end_date)}')
        WHERE c.glueexporteddate = DATE('{str(end_date)}')  
            AND c.type = 'order'
            AND c.orderdate >= DATE('{start_date}')
        GROUP BY 
            c.caserecnumber, 
            p.dob
    ),
    processed_age_data AS (
        SELECT
            YEAR(orderdate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            orderdate
        FROM age_data
    ),
    all_years_ages_orders AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    aggregated_data AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS num_orders
        FROM processed_age_data
        GROUP BY year, age_in_years
    ),
    filled_data AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.num_orders, (
                SELECT AVG(num_orders)
                FROM aggregated_data
                WHERE aggregated_data.year = a.year
            )) AS num_orders
        FROM all_years_ages_orders a
        LEFT JOIN aggregated_data ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    ),
    processed_age_active_caseload_exclud_new_orders(
        SELECT
            YEAR(earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            COUNT(court_number) AS num_active_exclud_new_orders 
        FROM (
            SELECT
                eo.earliest_order_date,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date
            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('{end_date}')
            LEFT JOIN (
                SELECT 
                    c.caserecnumber,
                    MIN(c.orderdate) AS earliest_order_date,
                    c.client_id
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.type = 'order'
                  AND c.orderdate >= DATE('{start_date}')
                GROUP BY c.caserecnumber, c.client_id
            ) eo  
                ON eo.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.caserecnumber AS court_number
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.orderdate > eo.earliest_order_made_date 
                  AND eo.termination_date IS NOT NULL  
            ) ac 
                ON ac.caserecnumber = eo.caserecnumber
            WHERE p.glueexporteddate = DATE('{end_date}')
                AND c.type = 'order'
                AND p.type = 'actor_client'
                AND c.orderdate >= DATE('{start_date}')
                AND c.statusdate = 'ACTIVE'
                AND year(c.orderdate) > eo.earliest_order_made_date         
        )
    )
    SELECT 
        a.year,
        a.age_in_years,
        a.num_losses,
        b.num_orders,
        e.num_active_exclud_new_orders,
        b.num_orders + e.num_active_exclud_new_orders AS num_active,
        CAST(a.num_losses AS DOUBLE) / NULLIF(c.total_losses, 0) AS mortality_rate,
    FROM final_agg a
    INNER JOIN filled_data b
        ON a.year = b.year AND a.age_in_years = b.age_in_years
    INNER JOIN processed_age_active_caseload_exclud_new_orders e
        ON a.year = e.year AND a.age_in_years = e.age_in_years 
    LEFT JOIN total_yearly_losses f
        ON a.year = f.year
    ORDER BY 
        a.year ASC, 
        a.age_in_years ASC
""")

Deputyship_mortality_rate_num_new_orders_active

In [ ]:


Total_annual_new_orders_active_aged = pydbtools.read_sql_query(f"""
WITH  
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE({st_year}, {en_year})) AS t(year)
    ),
    ages AS (
        SELECT age_in_years
        FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
    ),
    all_years_ages AS (
        SELECT 
            y.year, 
            a.age_in_years
        FROM years y
        CROSS JOIN ages a
    ),
    earliest_order_made AS (
        SELECT 
            c.caserecnumber AS court_number,
            c.client_id,
            MIN(c.orderdate) AS earliest_order_made_date,
            DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
        FROM opg_sirius_prod.persons p
        INNER JOIN opg_sirius_prod.cases c 
            ON c.client_id = p.id
        WHERE c.glueexporteddate = DATE('{str(end_date)}')  
            AND c.orderdate >= DATE('{start_date}')
            AND c.orderdate <= DATE('{end_date}')
        GROUP BY 
            c.caserecnumber, 
            c.client_id,
            p.dob
    ),
    processed_age_loss AS (
        SELECT
            eo.court_number,
            YEAR(eo.earliest_order_made_date) AS year,
            CASE 
                WHEN FLOOR(eo.raw_age) < 0 THEN 0
                ELSE ROUND(eo.raw_age)
            END AS age_in_years,
            eo.earliest_order_made_date,
            -- Correctly derive termination_date
            CASE
                WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                ELSE NULL
            END AS termination_date
        FROM opg_sirius_prod.persons p
        LEFT JOIN opg_sirius_prod.death_notifications dn 
            ON dn.person_id = p.id
            AND dn.glueexporteddate = DATE('{end_date}')
        LEFT JOIN earliest_order_made eo  
            ON eo.client_id = p.id
        LEFT JOIN (
            SELECT 
                c.client_id,
                c.statusdate
            FROM opg_sirius_prod.cases c
            WHERE c.glueexporteddate = DATE('{end_date}')
              AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
        ) rc 
            ON rc.client_id = p.id
        WHERE p.glueexporteddate = DATE('{end_date}')
          AND p.type = 'actor_client'
    ),
    court_records_analysis AS (
        SELECT
            eo.court_number,
            eo.year,
            eo.age_in_years,
            eo.earliest_order_made_date,
            eo.termination_date,
            COUNT(*) AS record_count,
            MAX(CASE 
                WHEN c.orderstatus = 'ACTIVE' 
                     AND c.orderdate = eo.earliest_order_made_date 
                     AND eo.termination_date IS NULL THEN 1 ELSE 0 
            END) AS active_flag,
            MAX(CASE 
                WHEN c.orderstatus = 'CLOSED' 
                     AND c.orderdate > eo.earliest_order_made_date 
                     AND eo.termination_date IS NULL THEN 1 ELSE 0 
            END) AS closed_flag
        FROM opg_sirius_prod.cases c
        LEFT JOIN processed_age_loss eo
            ON c.caserecnumber = eo.court_number
        GROUP BY 
            eo.court_number, 
            eo.year, 
            eo.age_in_years, 
            eo.earliest_order_made_date, 
            eo.termination_date
    ),
    updated_caseload AS (
        SELECT 
            *,
            CASE 
                WHEN record_count > 1 
                     AND active_flag = 1 
                     AND closed_flag = 1 THEN 1 
                ELSE 0 
            END AS num_active_caseload_update,
            CASE 
                WHEN record_count = 1 
                     AND active_flag = 1 
                     AND termination_date IS NULL THEN 1 
                ELSE 0 
            END AS num_new_orders_update,
            CASE 
                WHEN record_count = 1 
                     AND closed_flag = 1 
                     AND termination_date IS NOT NULL THEN 1 
                ELSE 0 
            END AS num_losses_update
        FROM court_records_analysis
    ),
    yearly_updates AS (
        SELECT 
            u.year,
            y.year AS active_year,
            u.age_in_years,
            u.num_active_caseload_update,
            u.num_new_orders_update,
            u.num_losses_update
        FROM updated_caseload u
        JOIN years y 
            ON y.year BETWEEN u.year AND {en_year}
    )
SELECT
    active_year AS year,
    age_in_years,
    SUM(num_active_caseload_update) AS num_active_caseload,
    SUM(num_new_orders_update) AS num_new_orders,
    SUM(num_losses_update) AS num_losses
FROM yearly_updates
GROUP BY 
    active_year,
    age_in_years
ORDER BY 
    active_year ASC,
    age_in_years ASC
""")
Total_annual_new_orders_active_aged

In [ ]:
int(max(Total_annual_new_orders_active_aged['num_active_caseload']))

In [ ]:
# Total_annual_new_orders_active_aged = pydbtools.read_sql_query(f"""
# WITH  
#     years AS (
#         SELECT year
#         FROM UNNEST(SEQUENCE(2017, 2024)) AS t(year)
#     ),
#     ages AS (
#         SELECT age_in_years
#         FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
#     ),
#     all_years_ages AS (
#         SELECT 
#             y.year, 
#             a.age_in_years
#         FROM years y
#         CROSS JOIN ages a
#     ),
#     earliest_order_made AS (
#         SELECT 
#             c.caserecnumber,
#             c.client_id,
#             MIN(c.orderdate) AS earliest_order_made_date,
#             DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
#         FROM opg_sirius_prod.persons p
#         INNER JOIN opg_sirius_prod.cases c 
#             ON c.client_id = p.id
#         WHERE c.glueexporteddate = DATE('{str(end_date)}')  
#             AND c.orderdate >= DATE('{start_date}')
#             AND c.orderdate <= DATE('{end_date}')
#         GROUP BY 
#             c.caserecnumber, 
#             c.client_id,
#             p.dob
#     ),
#     processed_age_loss AS (
#         SELECT
#             YEAR(eo.earliest_order_made_date) AS year,
#             CASE 
#                 WHEN FLOOR(eo.raw_age) < 0 THEN 0
#                 ELSE ROUND(eo.raw_age)
#             END AS age_in_years,
#             eo.caserecnumber,
#             eo.earliest_order_made_date,
#             -- Correctly derive termination_date
#             CASE
#                 WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                 WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                 WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                 ELSE NULL
#             END AS termination_date
#         FROM opg_sirius_prod.persons p
#         LEFT JOIN opg_sirius_prod.death_notifications dn 
#             ON dn.person_id = p.id
#             AND dn.glueexporteddate = DATE('{end_date}')
#         LEFT JOIN earliest_order_made eo  
#             ON eo.client_id = p.id
#         LEFT JOIN (
#             SELECT 
#                 c.client_id,
#                 c.statusdate
#             FROM opg_sirius_prod.cases c
#             WHERE c.glueexporteddate = DATE('{end_date}')
#               AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#         ) rc 
#             ON rc.client_id = p.id
#         WHERE p.glueexporteddate = DATE('{end_date}')
#           AND p.type = 'actor_client'
#     ),
#     agg_loss_count AS (
#         SELECT 
#             year,
#             age_in_years,
#             COUNT(termination_date) AS num_losses
#         FROM processed_age_loss
#         GROUP BY year, age_in_years
#     ),
#     total_yearly_losses AS (
#         SELECT 
#             year,
#             SUM(num_losses) AS total_losses
#         FROM agg_loss_count
#         GROUP BY year
#     ),
#     court_records_analysis AS (
#         SELECT
#             COUNT(*) AS record_count,
#             eo.year,
#             eo.age_in_years,
#             eo.termination_date,
#             MAX(CASE WHEN c.orderstatus = 'ACTIVE' AND c.orderdate = eo.earliest_order_made_date AND eo.termination_date IS NULL THEN 1 ELSE 0 END) AS active_flag,
#             MAX(CASE WHEN c.orderstatus = 'CLOSED' AND c.orderdate > eo.earliest_order_made_date AND eo.termination_date IS NULL THEN 1 ELSE 0 END) AS closed_flag
#         FROM opg_sirius_prod.cases c
#         LEFT JOIN processed_age_loss eo
#             ON c.caserecnumber = eo.caserecnumber
#         GROUP BY eo.year, eo.age_in_years, eo.termination_date
#     ),
#     updated_caseload AS (
#         SELECT 
#             *,
#             CASE 
#                 WHEN record_count > 1 
#                      AND active_flag = 1 
#                      AND closed_flag = 1 
#                      AND year BETWEEN 2017 AND 2024 THEN 1 
#                 ELSE 0 
#             END AS num_active_caseload_update,
#             CASE 
#                 WHEN record_count = 1 
#                      AND active_flag = 1 
#                      AND termination_date IS NULL THEN 1 
#                 ELSE 0 
#             END AS num_new_orders_update,
#             CASE 
#                 WHEN record_count = 1 
#                      AND closed_flag = 1 
#                      AND termination_date IS NOT NULL THEN 1 
#                 ELSE 0 
#             END AS num_losses_update
#         FROM court_records_analysis
#     )
# SELECT
#     year,
#     age_in_years,
#     SUM(num_active_caseload_update) AS num_active_caseload,
#     SUM(num_new_orders_update) AS num_new_orders,
#     SUM(num_losses_update) AS num_losses
# FROM updated_caseload
# GROUP BY year,
#     age_in_years
# ORDER BY year ASC,
#     age_in_years ASC
# """)
# Total_annual_new_orders_active_aged

In [ ]:
# Total_annual_new_orders_active_aged = pydbtools.read_sql_query(f"""
# WITH  
#     years AS (
#         SELECT year
#         FROM UNNEST(SEQUENCE(2017, 2024)) AS t(year)
#     ),
#     ages AS (
#         SELECT age_in_years
#         FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
#     ),
#     all_years_ages AS (
#         SELECT 
#             y.year, 
#             a.age_in_years
#         FROM years y
#         CROSS JOIN ages a
#     ),
#     earliest_order_made AS (
#         SELECT 
#             c.caserecnumber,
#             client_id,
#             MIN(c.orderdate) AS earliest_order_made_date
#         FROM opg_sirius_prod.cases c 
#         WHERE c.glueexporteddate = DATE('{end_date}')
#         AND c.type = 'order'
#         AND c.orderdate >= DATE('{start_date}')
#         AND c.orderdate <= DATE('{end_date}')
#         GROUP BY c.caserecnumber, c.client_id
#     ),
#     earliest_order_made AS (
#         SELECT 
#             c.caserecnumber,
#             client_id,
#             MIN(c.orderdate) AS earliest_order_made_date,
#             DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
#         FROM opg_sirius_prod.persons p
#         INNER JOIN opg_sirius_prod.cases c 
#             ON c.client_id = p.id
#         WHERE c.glueexporteddate = DATE('{str(end_date)}')  
#             AND c.orderdate >= DATE('{start_date}')
#             AND c.orderdate <= DATE('{end_date}')
#         GROUP BY 
#             c.caserecnumber, 
#             c.client_id,
#             p.dob
#         ),
#     processed_age_loss AS (
#         SELECT
#             YEAR(earliest_order_made_date) AS year,
#             CASE 
#                 WHEN FLOOR(raw_age) < 0 THEN 0
#                 ELSE ROUND(raw_age)
#             END AS age_in_years,
#             caserecnumber,
#             earliest_order_made_date,
#             termination_date
#         FROM (
#             SELECT
#                 eo.earliest_order_made_date,
#                 eo.caserecnumber,
#                 DATE_DIFF('day', p.dob, eo.earliest_order_made_date) / 365.25 AS raw_age,
#                 CASE
#                     WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
#                     WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
#                     WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
#                     ELSE NULL
#                 END AS termination_date
#             FROM opg_sirius_prod.persons p
#             LEFT JOIN opg_sirius_prod.death_notifications dn 
#                 ON dn.person_id = p.id
#                 AND dn.glueexporteddate = DATE('{end_date}')
#             LEFT JOIN (
#                 SELECT 
#                     caserecnumber,
#                     client_id,
#                     earliest_order_made_date
#                 FROM earliest_order_made    
#             ) eo  
#                 ON eo.client_id = p.id
#             LEFT JOIN (
#                 SELECT 
#                     c.client_id,
#                     c.statusdate
#                 FROM opg_sirius_prod.cases c
#                 WHERE c.glueexporteddate = DATE('{end_date}')
#                   AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
#             ) rc 
#                 ON rc.client_id = p.id
#             WHERE p.glueexporteddate = DATE('{end_date}')
#               AND p.type = 'actor_client'
#         )
#     ),
#     agg_loss_count AS (
#         SELECT 
#             year,
#             age_in_years,
#             COUNT(termination_date) AS num_losses
#         FROM processed_age_loss
#         GROUP BY year, age_in_years
#     ),
#     total_yearly_losses AS (
#         SELECT 
#             year,
#             SUM(num_losses) AS total_losses
#         FROM agg_loss_count
#         GROUP BY year
#     ),
#     court_records_analysis AS (
#         SELECT
#             c.caserecnumber,
#             COUNT(*) AS record_count,
#             d.earliest_order_made_date,
#             MAX(CASE WHEN c.orderstatus = 'ACTIVE' AND c.orderdate = d.earliest_order_made_date AND d.termination_date IS NULL THEN 1 ELSE 0 END) AS active_flag,
#             MAX(CASE WHEN c.orderstatus = 'CLOSED' AND c.orderdate > d.earliest_order_made_date AND d.termination_date IS NULL THEN 1 ELSE 0 END) AS closed_flag
#         FROM opg_sirius_prod.cases c
#         LEFT JOIN processed_age_loss d
#             ON c.caserecnumber = d.caserecnumber
#         GROUP BY c.caserecnumber,
#         d.earliest_order_made_date
#     ),
#     updated_caseload AS (
#         SELECT 
#             *,
#             CASE 
#                 WHEN record_count > 1 
#                      AND active_flag = 1 
#                      AND closed_flag = 1 
#                      AND YEAR(earliest_order_made_date) BETWEEN 2017 AND 2024 THEN 1 
#                 ELSE 0 
#             END AS num_active_caseload_update,
#             CASE 
#                 WHEN record_count = 1 
#                      AND active_flag = 1 
#                      AND termination_date IS NULL THEN 1 
#                 ELSE 0 
#             END AS num_new_orders_update,
#             CASE 
#                 WHEN record_count = 1 
#                      AND closed_flag = 1 
#                      AND termination_date IS NOT NULL THEN 1 
#                 ELSE 0 
#             END AS num_losses_update
#         FROM court_records_analysis
#     )
# SELECT
#     year,
#     SUM(num_active_caseload_update) AS num_active_caseload,
#     SUM(num_new_orders_update) AS num_new_orders,
#     SUM(num_losses_update) AS num_losses
# FROM updated_caseload
# GROUP BY year
# ORDER BY year ASC
# """)


# Total_annual_new_orders_active_aged

In [ ]:
# # alternatively, instead of 'receiptdate' of the order, you could use the earliest date that the order was given a supervision-level in OPG
# # i.e. basically the date where OPG actually kind of 'take control' of an order from CoP

# # greg said: (That spike in November 2023 is genuine – we received a huge dump of older orders.)
# # thats the spike in the receiptdate data
# # but supervision-level date will not show that peak, because thats the date when OPG actually get around to doing their bit 
# # - i.e. they cant just work a huge spike of 2000 received orders from OPG, that work gets spread over the next few months
# # and, again, this is new CLIENTS/ new CASERECNUMBERS (Court numbers), not new orders
# # count the new orders: df2.groupby(['month_year_earliest_receipt_date'])['client_id'].nunique().reset_index(name='count')  (WHERE c.glueexporteddate = DATE('2024-08-15'))
# # (WHERE c.glueexporteddate = DATE('2024-08-15') OR {target_date} )

# # age_data: Computes raw ages and identifies order dates.
# # processed_age_data: Rounds ages to integers and handles negative ages by setting them to 0.
# # all_years_ages: Generates all combinations of years and ages (0–106).
# # aggregated_data: Counts the number of orders for each age in each year.
# # filled_data: Joins aggregated_data with all_years_ages and imputes missing num_orders using the average for that year.
# # Final Selection: Outputs the required columns in ascending order of year and age.

# # Replaced generator: Used SEQUENCE(0, 106) and UNNEST to generate age values from 0 to 106. This works in Athena.
# # Cross-Join all_years_ages: Combines distinct years with all possible age values.
# # Aggregate and Join: Ensures missing ages are included for every year, and missing num_orders values are replaced with averages.

# Total_annual_new_orders_made_aged = pydbtools.read_sql_query(f"""
#     WITH 
#         age_data AS (
#             SELECT 
#                 c.caserecnumber,
#                 MIN(c.orderdate) AS orderdate,
#                 DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
#             FROM opg_sirius_prod.persons p
#             INNER JOIN opg_sirius_prod.cases c 
#                 ON c.client_id = p.id
#                 AND c.glueexporteddate = DATE('{str(end_date)}')
#             WHERE c.glueexporteddate = DATE('{str(end_date)}')  
#                 AND c.type = 'order'
#                 AND c.orderdate > DATE('2016-12-31')
#             GROUP BY 
#                 c.caserecnumber, 
#                 p.dob
#         ),
#         processed_age_data AS (
#             SELECT
#                 YEAR(orderdate) AS year,
#                 CASE 
#                     WHEN FLOOR(raw_age) < 0 THEN 0
#                     ELSE ROUND(raw_age)
#                 END AS age_in_years,
#                 orderdate
#             FROM age_data
#         ),
#         all_years_ages AS (
#             SELECT 
#                 y.year, 
#                 a.age AS age_in_years
#             FROM 
#                 (SELECT DISTINCT year FROM processed_age_data) y
#             CROSS JOIN 
#                 (SELECT CAST(n AS INT) AS age 
#                  FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
#         ),
#         aggregated_data AS (
#             SELECT 
#                 year,
#                 age_in_years,
#                 COUNT(*) AS num_orders
#             FROM processed_age_data
#             GROUP BY year, age_in_years
#         ),
#         filled_data AS (
#             SELECT 
#                 a.year,
#                 a.age_in_years,
#                 COALESCE(ag.num_orders, (
#                     SELECT AVG(num_orders)
#                     FROM aggregated_data
#                     WHERE aggregated_data.year = a.year
#                 )) AS num_orders
#             FROM all_years_ages a
#             LEFT JOIN aggregated_data ag 
#                 ON a.year = ag.year AND a.age_in_years = ag.age_in_years
#         )
#     SELECT 
#         year,
#         age_in_years,
#         num_orders
#     FROM filled_data
#     ORDER BY year ASC, age_in_years ASC
# """)

# #print(Total_annual_new_orders['num_orders'].sum()) #.groupby(['year']).sum()
# Total_annual_new_orders_made_aged

In [ ]:
Deputyship_annual_new_orders_made_aged = Total_annual_new_orders_made_aged
Deputyship_annual_new_orders_made_aged.to_csv('Deputyship_annual_new_orders_made_aged.csv', index=False)

In [ ]:
Deputyship_mortality_rate_num_new_orders = pydbtools.read_sql_query(f"""
    WITH 
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE(2017, 2024)) AS t(year)
    ),
    ages AS (
        SELECT age_in_years
        FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
    ),
    all_years_ages AS (
        SELECT 
            y.year, 
            a.age_in_years
        FROM years y
        CROSS JOIN ages a
    ),
    processed_age_loss AS (
        SELECT
            YEAR(earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date
        FROM (
            SELECT
                eo.earliest_order_date,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date
            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('{end_date}')
            LEFT JOIN (
                SELECT 
                    c.caserecnumber,
                    MIN(c.orderdate) AS earliest_order_date,
                    c.client_id
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.type = 'order'
                  AND c.orderdate >= DATE('{start_date}')
                GROUP BY c.caserecnumber, c.client_id
            ) eo  
                ON eo.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    MIN(c.receiptdate) AS receiptdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.receiptdate IS NOT NULL
                GROUP BY c.client_id
            ) er 
                ON er.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.statusdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
            ) rc 
                ON rc.client_id = p.id
            WHERE p.glueexporteddate = DATE('{end_date}')
              AND p.type = 'actor_client'
        )
    ),
    agg_loss_count AS (
        SELECT 
            year,
            age_in_years,
            COUNT(termination_date) AS num_losses
        FROM processed_age_loss
        GROUP BY year, age_in_years
    ),
    
    total_yearly_losses AS (
        SELECT 
            year,
            SUM(num_losses) AS total_losses
        FROM agg_loss_count
        GROUP BY year
    ),
    final_agg AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(lc.num_losses, 0) AS num_losses
        FROM all_years_ages a
        LEFT JOIN agg_loss_count lc
            ON a.year = lc.year
            AND a.age_in_years = lc.age_in_years
    ),
    age_data AS (
        SELECT 
            c.caserecnumber,
            MIN(c.orderdate) AS orderdate,
            DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
        FROM opg_sirius_prod.persons p
        INNER JOIN opg_sirius_prod.cases c 
            ON c.client_id = p.id
            AND c.glueexporteddate = DATE('{str(end_date)}')
        WHERE c.glueexporteddate = DATE('{str(end_date)}')  
            AND c.type = 'order'
            AND c.orderdate >= DATE('{start_date}')
        GROUP BY 
            c.caserecnumber, 
            p.dob
    ),
    processed_age_data AS (
        SELECT
            YEAR(orderdate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            orderdate
        FROM age_data
    ),
    all_years_ages_orders AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    aggregated_data AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS num_orders
        FROM processed_age_data
        GROUP BY year, age_in_years
    ),
    filled_data AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.num_orders, (
                SELECT AVG(num_orders)
                FROM aggregated_data
                WHERE aggregated_data.year = a.year
            )) AS num_orders
        FROM all_years_ages_orders a
        LEFT JOIN aggregated_data ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    )    
    SELECT 
        a.year,
        a.age_in_years,
        a.num_losses,
        b.num_orders,
        CAST(a.num_losses AS DOUBLE) / NULLIF(c.total_losses, 0) AS mortality_rate
    FROM final_agg a
    INNER JOIN filled_data b
        ON a.year = b.year AND a.age_in_years = b.age_in_years
    LEFT JOIN total_yearly_losses c
        ON a.year = c.year
    ORDER BY 
        a.year ASC, 
        a.age_in_years ASC
""")

Deputyship_mortality_rate_num_new_orders

In [ ]:
Deputyship_mortality_rate_num_new_orders.to_csv('Deputyship_mortality_rate_num_new_orders.csv', index=False)

In [ ]:
Deputyship_losses_num_new_orders = pydbtools.read_sql_query(f"""
    WITH 
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE(2017, 2024)) AS t(year)
    ),
    
    ages AS (
        SELECT age_in_years
        FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
    ),
    
    all_years_ages AS (
        SELECT 
            y.year, 
            a.age_in_years
        FROM years y
        CROSS JOIN ages a
    ),
    
    processed_age_loss AS (
        SELECT
            YEAR(earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date
        FROM (
            SELECT
                eo.earliest_order_date,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date
            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('{end_date}')
            LEFT JOIN (
                SELECT 
                    c.caserecnumber,
                    MIN(c.orderdate) AS earliest_order_date,
                    c.client_id
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.type = 'order'
                  AND c.orderdate >= DATE('{start_date}')
                GROUP BY c.caserecnumber, c.client_id
            ) eo  
                ON eo.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    MIN(c.receiptdate) AS receiptdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.receiptdate IS NOT NULL
                GROUP BY c.client_id
            ) er 
                ON er.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.statusdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
            ) rc 
                ON rc.client_id = p.id
            WHERE p.glueexporteddate = DATE('{end_date}')
              AND p.type = 'actor_client'
        )
    ),
    
    agg_loss_count AS (
        SELECT 
            year,
            age_in_years,
            COUNT(termination_date) AS num_losses
        FROM processed_age_loss
        GROUP BY year, age_in_years
    ),
    
    final_agg AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(lc.num_losses, 0) AS num_losses
        FROM all_years_ages a
        LEFT JOIN agg_loss_count lc
            ON a.year = lc.year
            AND a.age_in_years = lc.age_in_years
    ),

    
    age_data AS (
        SELECT 
            c.caserecnumber,
            MIN(c.orderdate) AS orderdate,
            DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
        FROM opg_sirius_prod.persons p
        INNER JOIN opg_sirius_prod.cases c 
            ON c.client_id = p.id
            AND c.glueexporteddate = DATE('{str(end_date)}')
        WHERE c.glueexporteddate = DATE('{str(end_date)}')  
            AND c.type = 'order'
            AND c.orderdate >= DATE('{start_date}')
        GROUP BY 
            c.caserecnumber, 
            p.dob
    ),
    processed_age_data AS (
        SELECT
            YEAR(orderdate) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            orderdate
        FROM age_data
    ),
    all_years_ages_orders AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_data) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
            FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    aggregated_data AS (
        SELECT 
            year,
            age_in_years,
            COUNT(*) AS num_orders
        FROM processed_age_data
        GROUP BY year, age_in_years
    ),
    filled_data AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.num_orders, (
                SELECT AVG(num_orders)
                FROM aggregated_data
                WHERE aggregated_data.year = a.year
            )) AS num_orders
        FROM all_years_ages_orders a
        LEFT JOIN aggregated_data ag 
            ON a.year = ag.year AND a.age_in_years = ag.age_in_years
    )
        
    SELECT 
        a.year,
        a.age_in_years,
        a.num_losses,
        b.num_orders
    FROM final_agg a
    INNER JOIN filled_data b
        ON a.year = b.year AND a.age_in_years = b.age_in_years
    ORDER BY 
        a.year ASC, 
        a.age_in_years ASC""")

Deputyship_losses_num_new_orders

In [ ]:
Deputyship_losses_num_new_orders.to_csv('Deputyship_losses_num_new_orders.csv', index=False)

In [ ]:
Deputyship_mortality_rate = pydbtools.read_sql_query(f"""
    WITH 
    years AS (
        SELECT year
        FROM UNNEST(SEQUENCE(2017, 2024)) AS t(year)
    ),
    
    ages AS (
        SELECT age_in_years
        FROM UNNEST(SEQUENCE(0, 106)) AS t(age_in_years)
    ),
    
    all_years_ages AS (
        SELECT 
            y.year, 
            a.age_in_years
        FROM years y
        CROSS JOIN ages a
    ),
    
    processed_age_loss AS (
        SELECT
            YEAR(earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date
        FROM (
            SELECT
                eo.earliest_order_date,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date
            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('{end_date}')
            LEFT JOIN (
                SELECT 
                    c.caserecnumber,
                    MIN(c.orderdate) AS earliest_order_date,
                    c.client_id
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.type = 'order'
                  AND c.orderdate > DATE('2021-12-31')
                GROUP BY c.caserecnumber, c.client_id
            ) eo  
                ON eo.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    MIN(c.receiptdate) AS receiptdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.receiptdate IS NOT NULL
                GROUP BY c.client_id
            ) er 
                ON er.client_id = p.id
            LEFT JOIN (
                SELECT 
                    c.client_id,
                    c.statusdate
                FROM opg_sirius_prod.cases c
                WHERE c.glueexporteddate = DATE('{end_date}')
                  AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
            ) rc 
                ON rc.client_id = p.id
            WHERE p.glueexporteddate = DATE('{end_date}')
              AND p.type = 'actor_client'
        )
    ),
    
    agg_loss_count AS (
        SELECT 
            year,
            age_in_years,
            COUNT(termination_date) AS num_losses
        FROM processed_age_loss
        GROUP BY year, age_in_years
    ),
    
    final_agg AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(lc.num_losses, 0) AS num_losses
        FROM all_years_ages a
        LEFT JOIN agg_loss_count lc
            ON a.year = lc.year
            AND a.age_in_years = lc.age_in_years
    )
    
    SELECT *
    FROM final_agg
    ORDER BY year, age_in_years
""")

Deputyship_mortality_rate

In [ ]:
Deputyship_mortality_rate = pydbtools.read_sql_query(f"""
    WITH 
    earliest_orderdate AS (
        SELECT 
            c.caserecnumber,
            MIN(c.orderdate) AS earliest_order_date,
            c.client_id
        FROM opg_sirius_prod.cases c
        WHERE c.glueexporteddate = DATE('{end_date}')
          AND c.type = 'order'
          AND c.orderdate > DATE('2021-12-31')
        GROUP BY c.caserecnumber, c.client_id
    ),
    
    earliest_receiptdate AS (
        SELECT 
            MIN(c.receiptdate) AS receiptdate,
            c.client_id
        FROM opg_sirius_prod.cases c
        WHERE c.glueexporteddate = DATE('{end_date}')
          AND c.receiptdate IS NOT NULL
        GROUP BY c.client_id
    ),    
        
    regain AS (
        SELECT
            c.client_id,
            c.statusdate
        FROM opg_sirius_prod.cases c
        WHERE c.glueexporteddate = DATE('{end_date}')
          AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
    ),
    
    main_deputyship_loss AS (
        SELECT
            eo.earliest_order_date,
            c.caserecnumber AS court_number,
            DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
            p.dateofdeath AS client_date_of_death,
            CAST(er.receiptdate AS DATE) AS earliest_receipt_date,
            p.id AS client_id,
            p.clientstatus AS client_status,
            CASE
                WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                ELSE NULL
            END AS termination_date,
            c.id AS order_id,
            c.casesubtype AS order_type,
            c.orderstatus AS order_status,
            c.orderdate AS order_made_date,
            c.orderissuedate AS order_issue_date,
            c.orderexpirydate AS order_expiry_date,
            c.orderclosurereason AS order_closure_reason
        FROM opg_sirius_prod.persons p
        LEFT JOIN opg_sirius_prod.death_notifications dn 
            ON dn.person_id = p.id
            AND dn.glueexporteddate = DATE('{end_date}')
        LEFT JOIN earliest_orderdate eo  
            ON eo.client_id = p.id
        LEFT JOIN earliest_receiptdate er 
            ON er.client_id = p.id
        LEFT JOIN regain rc 
            ON rc.client_id = p.id                
        LEFT JOIN opg_sirius_prod.cases c 
            ON p.id = c.client_id
            AND c.glueexporteddate = DATE('{end_date}')
            AND LOWER(c.type) = 'order'
        WHERE p.glueexporteddate = DATE('{end_date}')
          AND p.type = 'actor_client'
        ORDER BY c.caserecnumber, c.orderdate
    ),
    
    processed_age_loss AS (
        SELECT
            YEAR(earliest_order_date) AS year,
            CASE 
                WHEN FLOOR(raw_age) < 0 THEN 0
                ELSE ROUND(raw_age)
            END AS age_in_years,
            termination_date
        FROM main_deputyship_loss
    ),
    
    agg_loss_count AS (
        SELECT 
            year,
            age_in_years,
            COUNT(termination_date) AS num_loss
        FROM processed_age_loss
        GROUP BY year, age_in_years
    ),
    
    all_years_ages_loss AS (
        SELECT 
            y.year, 
            a.age AS age_in_years
        FROM 
            (SELECT DISTINCT year FROM processed_age_loss) y
        CROSS JOIN 
            (SELECT CAST(n AS INT) AS age 
             FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
    ),
    
    agg_age AS (
        SELECT 
            a.year,
            a.age_in_years,
            COALESCE(ag.num_loss, (
                SELECT AVG(num_loss)
                FROM agg_loss_count
                WHERE agg_loss_count.year = a.year
            )) AS num_orders
        FROM all_years_ages_loss a
        LEFT JOIN agg_loss_count ag 
            ON a.year = ag.year 
            AND a.age_in_years = ag.age_in_years
    ),
    
    agg_losses_aged AS (
        SELECT 
            YEAR(m.earliest_order_date) AS year,
            ROUND(m.raw_age) AS age_in_years,
            COUNT(m.termination_date) AS num_loss
        FROM main_deputyship_loss m
        GROUP BY YEAR(m.earliest_order_date), ROUND(m.raw_age)
    )                
    
    SELECT 
        b.year,
        b.age_in_years,
        COALESCE(t.num_loss, (
            SELECT AVG(num_loss)
            FROM agg_losses_aged
            WHERE agg_losses_aged.year = b.year
        )) AS num_losses
    FROM all_years_ages_loss b
    LEFT JOIN agg_losses_aged t 
        ON b.year = t.year 
        AND b.age_in_years = t.age_in_years
""")

Deputyship_mortality_rate

In [ ]:
Deputyship_mortality_rate = pydbtools.read_sql_query(f""" 
    WITH 
        earliest_orderdate AS (
            SELECT 
                c.caserecnumber,
                MIN(c.orderdate) as earliest_order_date,
                c.client_id
            FROM opg_sirius_prod.cases c
            WHERE c.glueexporteddate = DATE('{str(end_date)}')
                AND c.type = 'order'
                AND c.orderdate > DATE('2021-12-31')
            GROUP BY 
                c.caserecnumber,
                c.client_id
        ),
        
        earliest_receiptdate AS (
            SELECT 
                MIN(c.receiptdate) as receiptdate,
                c.client_id
            FROM opg_sirius_prod.cases c
            WHERE c.glueexporteddate = DATE('{str(end_date)}')
                AND c.receiptdate IS NOT NULL
            GROUP BY c.client_id
        ),    
        
        regain AS (
            SELECT
                c.client_id,
                c.statusdate
            FROM opg_sirius_prod.cases c
            WHERE c.glueexporteddate = DATE('{str(end_date)}')
                AND c.orderclosurereason = 'CLIENT REGAINED CAPACITY'
        ),
        
        main_deputyship_loss AS (
            SELECT
                eo.earliest_order_date,
                c.caserecnumber AS court_number,
                DATE_DIFF('day', p.dob, eo.earliest_order_date) / 365.25 AS raw_age,
                p.dateofdeath AS client_date_of_death,
                CAST(er.receiptdate AS DATE) AS earliest_receipt_date,
                p.id AS client_id,
                p.clientstatus AS client_status,
                CASE
                    WHEN p.dateofdeath IS NOT NULL THEN p.dateofdeath
                    WHEN p.clientstatus != 'ACTIVE' AND rc.statusdate IS NOT NULL THEN rc.statusdate
                    WHEN p.dateofdeath IS NULL AND dn.datenotified IS NOT NULL THEN dn.datenotified
                    ELSE NULL
                END AS termination_date,
                c.id as order_id,
                c.casesubtype as order_type,
                c.orderstatus as order_status,
                c.orderdate as order_made_date,
                c.orderissuedate as order_issue_date,
                c.orderexpirydate as order_expiry_date,
                c.orderclosurereason as order_closure_reason

            FROM opg_sirius_prod.persons p
            LEFT JOIN opg_sirius_prod.death_notifications dn 
                ON dn.person_id = p.id
                AND dn.glueexporteddate = DATE('{str(end_date)}')
            LEFT JOIN earliest_orderdate eo  
                ON eo.client_id = p.id
                AND dn.glueexporteddate = DATE('{str(end_date)}')
            LEFT JOIN earliest_receiptdate er 
                ON er.client_id = p.id
            LEFT JOIN regain rc 
                ON rc.client_id = p.id                
            LEFT JOIN opg_sirius_prod.cases c 
                ON p.id = c.client_id
                AND c.glueexporteddate = DATE('{str(end_date)}')
                AND lower(c.type) = 'order'
            LEFT JOIN (
                SELECT mr.order_id, MAX(mr.createddate) mostrecent
                FROM opg_sirius_prod.supervision_level_log mr
                WHERE mr.glueexporteddate = DATE('{str(end_date)}')
                    AND mr.supervisionlevel is not null
                GROUP BY mr.order_id
            ) mostrecents 
                ON c.id = mostrecents.order_id 
            LEFT JOIN opg_sirius_prod.supervision_level_log level 
                ON mostrecents.order_id = level.order_id
                AND mostrecents.mostrecent = level.createddate
                AND level.glueexporteddate = DATE('{str(end_date)}')
            WHERE p.glueexporteddate = DATE('{str(end_date)}')
            AND p.type = 'actor_client'
            ORDER BY
                c.caserecnumber, 
                c.orderdate ASC
        ),
        
        processed_age_loss AS (
            SELECT
                YEAR(earliest_order_date) AS year,
                CASE 
                    WHEN FLOOR(raw_age) < 0 THEN 0
                    ELSE ROUND(raw_age)
                END AS age_in_years,
                termination_date
            FROM main_deputyship_loss
        ),
        
        all_years_ages_loss AS (
            SELECT 
                y.year, 
                a.age AS age_in_years,
                termination_date
            FROM 
                (SELECT DISTINCT year FROM processed_age_loss) y
            CROSS JOIN 
                (SELECT CAST(n AS INT) AS age 
                 FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
        ),
        
        agg_loss_count AS (
            SELECT 
                year,
                age_in_years,
                COUNT(m.termination_date) AS num_loss
            FROM all_years_ages_loss
            GROUP BY 
                year, 
                age_in_years
        ),
        
        agg_age AS (
            SELECT 
                a.year,
                a.age_in_years,
                COALESCE(ag.num_loss, (
                    SELECT AVG(num_loss)
                    FROM agg_loss_count
                    WHERE agg_loss_count.year = a.year
                )) AS num_orders
            FROM all_years_ages_loss a
            LEFT JOIN agg_loss_count ag 
                ON a.year = ag.year 
                AND a.age_in_years = ag.age_in_years
        ),
        
        agg_losses_aged AS (
            SELECT 
                fag.year,
                fag.age_in_years,
                COUNT(m.termination_date) AS num_loss
            FROM main_deputyship_tbl m
            LEFT JOIN all_years_ages fag
                ON YEAR(m.earliest_order_date) = fag.year
                AND m.raw_age = fag.age_in_years
        )                
        
        SELECT 
            b.year,
            b.age_in_years,
            COALESCE(t.num_loss, (
                SELECT AVG(num_loss)
                FROM agg_losses_aged
                WHERE agg_losses_aged.year = b.year
            )) AS num_losses
        FROM all_years_ages_loss b
        LEFT JOIN agg_losses_aged t 
            ON b.year = t.year 
            AND b.age_in_years = t.age_in_years
        """)

        # age_data AS (
        #     SELECT 
        #         o.caserecnumber,
        #         o.client_id,
        #         o.earliest_order_date,
        #         DATE_DIFF('day', p.dob, o.earliest_order_date) / 365.25 AS raw_age
        #     FROM opg_sirius_prod.persons p
        #     INNER JOIN earliest_orderdate o
        #         ON o.client_id = p.id
        #         AND p.glueexporteddate = DATE('{str(end_date)}')
        #     GROUP BY
        #         o.earliest_order_date,
        #         o.caserecnumber, 
        #         o.client_id,
        #         p.dob
        # ),
Deputyship_mortality_rate

In [ ]:
# Deputyship_annual_new_orders_made_aged_mortality = pydbtools.read_sql_query(f""" 
#     WITH 
#         age_data AS (
#             SELECT 
#                 c.caserecnumber,
#                 MIN(c.orderdate) AS orderdate,
#                 DATE_DIFF('day', p.dob, MIN(c.orderdate)) / 365.25 AS raw_age
#             FROM opg_sirius_prod.persons p
#             INNER JOIN opg_sirius_prod.cases c 
#                 ON c.client_id = p.id
#                 AND c.glueexporteddate = DATE('{str(end_date)}')
#             WHERE c.glueexporteddate = DATE('{str(end_date)}')  
#                 AND c.type = 'order'
#                 AND c.orderdate > DATE('2016-12-31')
#             GROUP BY 
#                 c.caserecnumber, 
#                 p.dob
#         ),
#         processed_age_data AS (
#             SELECT
#                 YEAR(orderdate) AS year,
#                 CASE 
#                     WHEN FLOOR(raw_age) < 0 THEN 0
#                     ELSE ROUND(raw_age)
#                 END AS age_in_years,
#                 orderdate
#             FROM age_data
#         ),
#         all_years_ages AS (
#             SELECT 
#                 y.year, 
#                 a.age AS age_in_years
#             FROM 
#                 (SELECT DISTINCT year FROM processed_age_data) y
#             CROSS JOIN 
#                 (SELECT CAST(n AS INT) AS age 
#                  FROM UNNEST(SEQUENCE(0, 106)) AS t(n)) a
#         ),
#         aggregated_data AS (
#             SELECT 
#                 year,
#                 age_in_years,
#                 COUNT(*) AS num_orders
#             FROM processed_age_data
#             GROUP BY year, age_in_years
#         ),
#         filled_data AS (
#             SELECT 
#                 a.year,
#                 a.age_in_years,
#                 COALESCE(ag.num_orders, (
#                     SELECT AVG(num_orders)
#                     FROM aggregated_data
#                     WHERE aggregated_data.year = a.year
#                 )) AS num_orders
#             FROM all_years_ages a
#             LEFT JOIN aggregated_data ag 
#                 ON a.year = ag.year AND a.age_in_years = ag.age_in_years
#         ),
#         termination_data AS (
#             SELECT 
#                 YEAR(p.dateofdeath) AS year_of_termination, 
#                 f.age_in_years, 
#                 COUNT(*) AS termination_count
#             FROM opg_sirius_prod.persons p 
#             LEFT JOIN aggregated_data ag 
#                 ON a.year = ag.year AND a.age_in_years = ag.age_in_years
#                 ON YEAR(p.dateofdeath) = f.year
#                 AND p.glueexporteddate = DATE('{str(end_date)}')
#             WHERE p.glueexporteddate = DATE('{str(end_date)}')
#                 AND p.type = 'actor_client'
#             GROUP BY YEAR(p.dateofdeath), f.age_in_years
#         )
#     SELECT 
#         f.year,
#         f.age_in_years,
#         f.num_orders,
#         t.termination_count
#     FROM filled_data f
#     LEFT JOIN termination_data t
#         ON f.year = t.year_of_termination 
#         AND f.age_in_years = t.age_in_years
#     ORDER BY f.year ASC, f.age_in_years ASC
# """)
# Deputyship_annual_new_orders_made_aged_mortality

# Deputyship_annual_new_orders_made_aged_mortality.to_csv('Deputyship_annual_new_orders_made_aged_mortality.csv', index=False)

In [ ]:
# rate of loss (death)
# Does CASRAC record orders made by CoP even if the person has died? If so, by looking at the receipt date instead of the order made date, 
# should there be minimal differences between receipts sent to OPG and those recorded in CoP, making them equivalent?
# If there are differences in the above, could they be caused by the loss of individuals who have died? What is the rate of loss (death), 
# and how has it affected OPG receipts?

## counts per month/year of dateofdeath for actor_clients in sirius
## the rate supervision clients are dying and moving out of 'active'
loss_rate_receipt_monthly = pydbtools.read_sql_query(f"""
    select 
        YEAR(p.dateofdeath) as year,
        MONTH(p.dateofdeath) as month,
        COUNT(*) as count
    from opg_sirius_prod.persons p
    where p.glueexporteddate = DATE('{str(end_date)}')
        and p.type = 'actor_client'
    group by
        YEAR(p.dateofdeath),
        MONTH(p.dateofdeath)
    order by 
        YEAR(p.dateofdeath) DESC,
        MONTH(p.dateofdeath) DESC
    """)

print(loss_rate_receipt_monthly)


loss_rate_receipt = pydbtools.read_sql_query(f"""
    select 
        YEAR(p.dateofdeath) as year,
        COUNT(*) as count
    from opg_sirius_prod.persons p
    where p.glueexporteddate = DATE('{str(final_date)}')
        and p.type = 'actor_client'
    group by
        YEAR(p.dateofdeath)
    order by 
        YEAR(p.dateofdeath) DESC
    """)

print(loss_rate_receipt)

death_loss_rate_receipt = loss_rate_receipt[loss_rate_receipt['year'] >= 2017]

# Plotting annual death count
plt.figure(figsize=(10, 6))
plt.bar(death_loss_rate_receipt['year'], death_loss_rate_receipt['count'], color='red')
plt.title('Annual Death Count of Supervision Clients')
plt.xlabel('Year')
plt.ylabel('Number of Deaths')
plt.show()


In [ ]:
yearly_age = pydbtools.read_sql_query(f"""
    with client_age as (
        SELECT
            p.id,
            (date_diff('day', p.dob, c.orderdate) / 365.25) age_in_years,
            c.orderdate
        FROM opg_sirius_prod.persons p
        INNER JOIN opg_sirius_prod.cases c on c.client_id = p.id
            and c.glueexporteddate = DATE('{str(end_date)}') 
        WHERE p.glueexporteddate = DATE('{str(end_date)}')
            and p.type = 'actor_client'
    )
    SELECT 
        YEAR(ca.orderdate) AS order_year,
        -- MONTH(ca.orderdate) AS order_month,
        ca.age_in_years AS age
    FROM 
        client_age ca
    WHERE YEAR(ca.orderdate) > 2016
    GROUP BY 
        YEAR(ca.orderdate), 
        -- MONTH(ca.orderdate),
        ca.age_in_years
    ORDER BY 
        order_year ASC,
        -- order_month ASC,
        age ASC
        """)

print(yearly_age)
yearly_age

# The focus is on analysing trends and distributions related to the age of protected persons under supervision, especially in the context of new orders and mortality rates. 
1. Monthly Trend in Average Age of Protected Persons for New Orders
This part calculates the average age of clients at the time a new order is made, grouped by month and year.


# The monthly trend in the average age of protected persons for whom new orders have been made.

It should be possible to produce this directly from Sirius based on receipt dates. 
This will also be revealing particularly post pandemic given the issue with delays in OPG receiving new orders and the suggestion that this results in many cases not being received by OPG as individuals sadly die before receipt.


In [ ]:
monthly_trend_avg_age = pydbtools.read_sql_query(f"""
    with client_age as (
        SELECT
            p.id,
            (date_diff('day', p.dob, c.orderdate) / 365.25) age_in_years,
            c.orderdate
        FROM opg_sirius_prod.persons p
        INNER JOIN opg_sirius_prod.cases c on c.client_id = p.id
            and c.glueexporteddate = DATE('{str(end_date)}')
        WHERE p.glueexporteddate = DATE('{str(end_date)}')
            and p.type = 'actor_client'
    )

    SELECT 
        YEAR(ca.orderdate) AS order_year,
        MONTH(ca.orderdate) AS order_month,
        DAY(ca.orderdate) AS order_day,
        AVG(ca.age_in_years) AS average_age,
        MAX(ca.age_in_years) AS max_age,
        MIN(ca.age_in_years) AS min_age
    FROM 
        client_age ca
    WHERE YEAR(ca.orderdate) > 2016
    GROUP BY 
        YEAR(ca.orderdate), 
        MONTH(ca.orderdate),
        DAY(ca.orderdate)
    ORDER BY 
        YEAR(ca.orderdate) ASC, 
        MONTH(ca.orderdate) ASC,
        DAY(ca.orderdate) ASC,
        MIN(ca.age_in_years),
        MAX(ca.age_in_years),
        AVG(ca.age_in_years)
        """)

print(monthly_trend_avg_age)

# Convert year and month to a datetime object  
#monthly_trend_avg_age['date'] = pd.to_datetime(monthly_trend_avg_age[['order_year', 'order_month']].assign(DAY=1))
for i in range(len(monthly_trend_avg_age)):
    year = monthly_trend_avg_age.loc[i, 'order_year']
    month = monthly_trend_avg_age.loc[i, 'order_month']
    monthly_trend_avg_age.loc[i, 'date']= pd.to_datetime(f"{year}-{month}-1").strftime('%b-%y')

# Plotting
plt.figure(figsize=(25, 12))
plt.plot(monthly_trend_avg_age['date'], monthly_trend_avg_age['average_age'], marker='o')
# Define the lockdown date
lockdown_date = pd.to_datetime('2020-03-01') #'Mar-20' #
lockdown_date_formatted = pd.to_datetime(str(lockdown_date)[:10]).strftime('%b-%y')
plt.axvline(x=lockdown_date_formatted, color='r', linestyle='--', label='Lockdown Start (Mar-20)')
plt.title('Monthly Trend in Average Age of Protected Persons for New Orders')
plt.xlabel('Date')
plt.ylabel('Average Age')
plt.grid(True)

# Customise x-axis labels to show every 4 months
xticks = monthly_trend_avg_age['date'][::6]  # Select every 4th date
plt.xticks(xticks, rotation=85)  # Rotate for better readability

plt.tight_layout()
plt.show()

2. Comparison of Age Distribution for New Orders Between 2017 and 2024
This section compares the age distribution of new orders between 2017 and 2024 using histograms.

To enhance the analysis and visualization for comparing the age distributions between 2017 and 2024 to achieve the following:

Calculate the Actual Counts in Each Age Bin: This will allow us to see the distribution of clients in different age groups.

Conduct a Statistical Test (e.g., Chi-Square Test): This will help to determine if the differences between the distributions are statistically significant.

Visualize the Age Distributions with Counts: Plot histograms to visualize the actual number of clients in each age bin for the years 2017 and 2024.

Counts in Each Bin: We used pd.cut() to categorize ages into bins and value_counts() to get the number of clients in each bin for 2017 and 2024. This information is plotted to show the distribution of clients across age groups.

Statistical Test: The Chi-Square test (chi2_contingency) is used to determine if the differences between the 2017 and 2024 distributions are statistically significant. The result includes the test statistic and p-value.

Visualization:

A bar chart is used to display the actual number of clients in each age bin for both years.
Histograms show the relative frequency of clients across different ages, similar to your original approach, but now with statistical context.
This expanded analysis will provide you with a more comprehensive understanding of the changes in the age distribution of protected persons between 2017 and 2024.

In [ ]:

age_distribution_2017_24 = f"""
    SELECT
        p.id,
        (date_diff('day', p.dob, c.orderdate) / 365.25) age,
        c.orderdate
    FROM opg_sirius_prod.persons p
    INNER JOIN opg_sirius_prod.cases c on c.client_id = p.id
        and c.glueexporteddate = DATE('{str(end_date)}')
    WHERE p.glueexporteddate = DATE('{str(end_date)}')
        and p.type = 'actor_client'
        and YEAR(c.orderdate) > 2016
"""
df = pydbtools.read_sql_query(age_distribution_2017_24)
df['orderdate'] = pd.to_datetime(df['orderdate'])
df['year'] = df['orderdate'].dt.year
bins = np.linspace(0,106,21)
years = df['year'].unique()
plt.figure(figsize=(6, 10))
years = sorted(df['year'].unique())
for i, year in enumerate(years, 1):
    plt.subplot(len(years), 1, i)
    df[df['year'] == year]['age'].plot(kind='hist', bins=bins, edgecolor='black',density=True)
    plt.title(f'Age Distribution in {year}')
    plt.xlabel('Age')
    plt.ylabel('Frequency')
    plt.xlim([21,106])
plt.tight_layout()
plt.show()


bins = np.linspace(0, 106, 21)
plt.figure(figsize=(25, 12))

for year in years:
    plt.hist(df[df['year'] == year], bins=bins, alpha=0.5, label=f'{year}', density=True)

plt.title('Age Distribution of New Orders in 2017 and 2024')
plt.xlabel('Age')
plt.ylabel('Relative Frequency')
plt.legend()
plt.show()


from scipy.stats import chi2_contingency

# Create bins and calculate counts
bins = np.linspace(0, 106, 21)
df['age_bin'] = pd.cut(df['age'], bins=bins)

# Get counts for 2017 and 2024
counts_2017 = df[df['year'] == 2017]['age_bin'].value_counts().sort_index()
counts_2024 = df[df['year'] == 2024]['age_bin'].value_counts().sort_index()

# Create a DataFrame for comparison
age_dist_df = pd.DataFrame({'2017': counts_2017, '2024': counts_2024}).fillna(0)

# Plot the actual counts in each bin
plt.figure(figsize=(25, 13))
age_dist_df.plot(kind='bar', edgecolor='black', alpha=0.7)
plt.title('Age Distribution Comparison (2017 vs 2024)')
plt.xlabel('Age Bins')
plt.ylabel('Number of Clients')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Chi-Square Test for Statistical Significance
chi2, p, dof, expected = chi2_contingency(age_dist_df.T)

print(f"Chi-Square Test Statistic: {chi2}")
print(f"P-Value: {p}")

if p < 0.05:
    print("The difference between the age distributions in 2017 and 2024 is statistically significant.")
else:
    print("The difference between the age distributions in 2017 and 2024 is not statistically significant.")

# Plot relative frequencies
oldest_year = df['year'].min()
newest_year = df['year'].max()
plt.figure(figsize=(10, 6))
df[df['year'] == oldest_year]['age'].plot(kind='hist', bins=bins, density=True, 
                                          alpha=0.5, edgecolor='black', color='blue', label=f'{oldest_year}')
df[df['year'] == newest_year]['age'].plot(kind='hist', bins=bins, density=True, 
                                          alpha=0.5, edgecolor='black', color='red', label=f'{newest_year}')
plt.title(f'Age Distribution (Relative Frequency) for {oldest_year} and {newest_year}')
plt.xlabel('Age')
plt.ylabel('Relative Frequency')
plt.legend()
plt.show()

3. Mortality Rate Analysis
This analysis focuses on the rate of supervision clients dying and moving out of active supervision.

In [ ]:
# rate of loss (death)
# Does CASRAC record orders made by CoP even if the person has died? If so, by looking at the receipt date instead of the order made date, 
# should there be minimal differences between receipts sent to OPG and those recorded in CoP, making them equivalent?
# If there are differences in the above, could they be caused by the loss of individuals who have died? What is the rate of loss (death), 
# and how has it affected OPG receipts?

## counts per month/year of dateofdeath for actor_clients in sirius
## the rate supervision clients are dying and moving out of 'active'
loss_rate_receipt_monthly = pydbtools.read_sql_query(f"""
    select 
        YEAR(p.dateofdeath) as year,
        MONTH(p.dateofdeath) as month,
        COUNT(*) as count
    from opg_sirius_prod.persons p
    where p.glueexporteddate = DATE('{str(end_date)}')
        and p.type = 'actor_client'
    group by
        YEAR(p.dateofdeath),
        MONTH(p.dateofdeath)
    order by 
        YEAR(p.dateofdeath) DESC,
        MONTH(p.dateofdeath) DESC
    """)

print(loss_rate_receipt_monthly)


loss_rate_receipt = pydbtools.read_sql_query(f"""
    select 
        YEAR(p.dateofdeath) as year,
        COUNT(*) as count
    from opg_sirius_prod.persons p
    where p.glueexporteddate = DATE('{str(final_date)}')
        and p.type = 'actor_client'
    group by
        YEAR(p.dateofdeath)
    order by 
        YEAR(p.dateofdeath) DESC
    """)

print(loss_rate_receipt)

death_loss_rate_receipt = loss_rate_receipt[loss_rate_receipt['year'] >= 2017]

# Plotting annual death count
plt.figure(figsize=(10, 6))
plt.bar(death_loss_rate_receipt['year'], death_loss_rate_receipt['count'], color='red')
plt.title('Annual Death Count of Supervision Clients')
plt.xlabel('Year')
plt.ylabel('Number of Deaths')
plt.show()


4. Total Annual New Orders
This section calculates and visualizes the number of new orders received per year.

In [ ]:
# alternatively, instead of 'receiptdate' of the order, you could use the earliest date that the order was given a supervision-level in OPG
# i.e. basically the date where OPG actually kind of 'take control' of an order from CoP

# greg said: (That spike in November 2023 is genuine – we received a huge dump of older orders.)
# thats the spike in the receiptdate data
# but supervision-level date will not show that peak, because thats the date when OPG actually get around to doing their bit 
# - i.e. they cant just work a huge spike of 2000 received orders from OPG, that work gets spread over the next few months
# and, again, this is new CLIENTS/ new CASERECNUMBERS (Court numbers), not new orders
# count the new orders: df2.groupby(['month_year_earliest_receipt_date'])['client_id'].nunique().reset_index(name='count')  (WHERE c.glueexporteddate = DATE('2024-08-15'))
# (WHERE c.glueexporteddate = DATE('2024-08-15') OR {target_date} )

Total_annual_new_orders = pydbtools.read_sql_query(f"""
    with 
        earliest_receiptdate AS (
            SELECT 
                MIN(c.receiptdate) as receiptdate,
                c.caserecnumber
            FROM opg_sirius_prod.cases c 
            WHERE c.glueexporteddate = DATE('{str(final_date)}') 
                AND c.type = 'order'
            GROUP BY c.caserecnumber
        )
    SELECT 
        year(c.receiptdate) as year,
        count(*) as num_orders
    FROM earliest_receiptdate c
    GROUP BY year(c.receiptdate) 
    order by year(c.receiptdate) desc
    """)

#print(Total_annual_new_orders['num_orders'].sum()) #.groupby(['year']).sum()
#DATE('2024-08-28')   DATE({final_date.strftime('%Y-%m-%d')}) 
print(Total_annual_new_orders)

total_new_orders = Total_annual_new_orders[Total_annual_new_orders['year'] >= 2017]
total_new_orders = total_new_orders[total_new_orders['year'] < 2024]

# Plotting new orders per year
plt.figure(figsize=(10, 6))
plt.plot(np.array(total_new_orders['year']), 
         np.array(total_new_orders['num_orders']), 
         marker='o')
plt.title('Total Annual New Orders')
plt.xlabel('Year')
plt.ylabel('Number of Orders')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# alternatively, instead of 'receiptdate' of the order, you could use the earliest date that the order was given a supervision-level in OPG
# i.e. basically the date where OPG actually kind of 'take control' of an order from CoP

# greg said: (That spike in November 2023 is genuine – we received a huge dump of older orders.)
# thats the spike in the receiptdate data
# but supervision-level date will not show that peak, because thats the date when OPG actually get around to doing their bit 
# - i.e. they cant just work a huge spike of 2000 received orders from OPG, that work gets spread over the next few months
# and, again, this is new CLIENTS/ new CASERECNUMBERS (Court numbers), not new orders
# count the new orders: df2.groupby(['month_year_earliest_receipt_date'])['client_id'].nunique().reset_index(name='count')  (WHERE c.glueexporteddate = DATE('2024-08-15'))
# (WHERE c.glueexporteddate = DATE('2024-08-15') OR {target_date} )

Total_annual_new_orders = pydbtools.read_sql_query(f"""
    with 
        earliest_receiptdate AS (
            SELECT 
                MIN(c.receiptdate) as receiptdate,
                c.caserecnumber
            FROM opg_sirius_prod.cases c 
            WHERE c.glueexporteddate = DATE('2024-08-15')  
                AND c.type = 'order'
            GROUP BY c.caserecnumber
        )
    SELECT 
        year(c.receiptdate) as year,
        count(*) as num_orders
    FROM earliest_receiptdate c
    GROUP BY year(c.receiptdate) 
    order by year(c.receiptdate) desc
    """)

#print(Total_annual_new_orders['num_orders'].sum()) #.groupby(['year']).sum()
Total_annual_new_orders

In [ ]:
# It still shows a smaller step change, but we’d expect it to – there was a pandemic! 
# If we take an arbitrary average of the 12 months between 1/5/2023 and 30/4/2024, it’s 960 – 7 higher than
# the pre-pandemic level we’ve ‘stepped down’ from. 
# To me, it looks like there was the expected disruption and slow recovery from the pandemic but CoP 
# have now recovered and are supplying orders at a pre-pandemic rate. 
# The move from Casrec to COPCRIS has disrupted the regular flow of work over the past few weeks though.
# that 12 months has picked, does include the insane spike from the dump from CoP
# (WHERE c.glueexporteddate = DATE('2024-08-15') OR {target_date} )

arbitrary_avg_monthly_new_orders_receipt = pydbtools.read_sql_query(f"""
    WITH earliest_receiptdate AS (
        SELECT 
            MIN(c.receiptdate) AS receiptdate,
            c.caserecnumber
        FROM 
            opg_sirius_prod.cases c 
        WHERE 
            c.glueexporteddate = DATE('2024-08-15')  
            AND c.type = 'order'
        GROUP BY 
            c.caserecnumber
    ),
    monthly_counts AS (
        SELECT 
            EXTRACT(YEAR FROM e.receiptdate) AS year, 
            EXTRACT(MONTH FROM e.receiptdate) AS month, 
            COUNT(*) AS count_per_month
        FROM 
            earliest_receiptdate e
        GROUP BY 
            EXTRACT(YEAR FROM e.receiptdate), 
            EXTRACT(MONTH FROM e.receiptdate)
    )
    SELECT 
        mc1.year, 
        mc1.month, 
        AVG(mc2.count_per_month) AS avg_count_last_12_months
    FROM 
        monthly_counts mc1
    JOIN 
        monthly_counts mc2 
    ON 
        (mc2.year * 12 + mc2.month) BETWEEN (mc1.year * 12 + mc1.month) - 11 AND (mc1.year * 12 + mc1.month)
    GROUP BY 
        mc1.year, 
        mc1.month
    ORDER BY 
        mc1.year DESC, 
        mc1.month DESC
    """) 

# but that would give, for each month, the average count of receiptdate orders per month for the previous 12 months
arbitrary_avg_monthly_new_orders_receipt

In [ ]:
# Plotting the line chart
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta, datetime

arbitrary_avg_monthly_new_orders_receipt = arbitrary_avg_monthly_new_orders_receipt.sort_values(by=['year', 'month'])
arbitrary_avg_monthly_new_orders_filtered_receipt = arbitrary_avg_monthly_new_orders_receipt[
    (arbitrary_avg_monthly_new_orders_receipt['year'] >= 2017) & 
    (arbitrary_avg_monthly_new_orders_receipt['year'] <= 2025)
]

for i in range(len(arbitrary_avg_monthly_new_orders_filtered_receipt)):
    year = arbitrary_avg_monthly_new_orders_filtered_receipt.loc[i, 'year']
    month = arbitrary_avg_monthly_new_orders_filtered_receipt.loc[i, 'month']
    arbitrary_avg_monthly_new_orders_filtered_receipt.loc[i, 'formatted_date'] = dt.date(year, month, 1).strftime('%b-%y')
        
arbitrary_avg_monthly_new_orders_filtered_receipt = arbitrary_avg_monthly_new_orders_filtered_receipt.rename(columns={'formatted_date': 'month_year'})
arbitrary_avg_monthly_new_orders_filtered_receipt


#[(result_df1['client_status'] == 'ACTIVE')]
#arbitrary_avg_monthly_new_orders_filtered = arbitrary_avg_monthly_new_orders_filtered.groupby(['month_year_made_earliest'])['client_id'].nunique().reset_index(name='count')
#result_df1 = result_df1.groupby(['month_year_made'])['client_id'].nunique().reset_index(name='count')

plt.figure(figsize=(35, 15))
plt.plot(arbitrary_avg_monthly_new_orders_filtered_receipt['month_year'], 
         arbitrary_avg_monthly_new_orders_filtered_receipt['avg_count_last_12_months'], 
         marker='o')
plt.title('Unique Client Count by Month and Year')
plt.xlabel('Month and Year')
plt.ylabel('Unique Client Count')
plt.grid(True)

# Set the locator for the x-axis to show labels quarterly
#ax = plt.gca()
#ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display the result
print(arbitrary_avg_monthly_new_orders_filtered_receipt)

In [ ]:
# since it wasnt a spike in orders being made by CoP, it was a spike in them being received by OPG?
# but that does show a trend downwards right
# because using orderdate, not receiptdate
# thats not that surprising, the last bunch of months will be very unreliable for the orderdate version
# 4 months or so for a random case I just looked up for this year


arbitrary_avg_monthly_new_orders_made = pydbtools.read_sql_query(f"""
    WITH earliest_orderdate AS (
        SELECT 
            MIN(c.orderdate) AS orderdate,
            c.caserecnumber
        FROM 
            opg_sirius_prod.cases c 
        WHERE 
            c.glueexporteddate = DATE('2024-08-15')  
            AND c.type = 'order'
        GROUP BY 
            c.caserecnumber
    ),
    monthly_counts AS (
        SELECT 
            EXTRACT(YEAR FROM e.orderdate) AS year, 
            EXTRACT(MONTH FROM e.orderdate) AS month, 
            COUNT(*) AS count_per_month
        FROM 
            earliest_orderdate e
        GROUP BY 
            EXTRACT(YEAR FROM e.orderdate), 
            EXTRACT(MONTH FROM e.orderdate)
    )
    SELECT 
        mc1.year, 
        mc1.month, 
        AVG(mc2.count_per_month) AS avg_count_last_12_months
    FROM 
        monthly_counts mc1
    JOIN 
        monthly_counts mc2 
    ON 
        (mc2.year * 12 + mc2.month) BETWEEN (mc1.year * 12 + mc1.month) - 11 AND (mc1.year * 12 + mc1.month)
    GROUP BY 
        mc1.year, 
        mc1.month
    ORDER BY 
        mc1.year DESC, 
        mc1.month DESC
    """) 

arbitrary_avg_monthly_new_orders_made

In [ ]:
# Plotting the line chart
import datetime as dt
from datetime import timedelta, datetime


arbitrary_avg_monthly_new_orders_made = arbitrary_avg_monthly_new_orders_made.sort_values(by=['year', 'month'])
arbitrary_avg_monthly_new_orders_filtered_made = arbitrary_avg_monthly_new_orders_made[
    (arbitrary_avg_monthly_new_orders_made['year'] >= 2017) & 
    (arbitrary_avg_monthly_new_orders_made['year'] <= 2025)
]

for i in range(len(arbitrary_avg_monthly_new_orders_filtered_made)):
    year = arbitrary_avg_monthly_new_orders_filtered_made.loc[i, 'year']
    month = arbitrary_avg_monthly_new_orders_filtered_made.loc[i, 'month']
    arbitrary_avg_monthly_new_orders_filtered_made.loc[i, 'formatted_date'] = dt.date(year, month, 1).strftime('%b-%y')
        
arbitrary_avg_monthly_new_orders_filtered_made = arbitrary_avg_monthly_new_orders_filtered_made.rename(columns={'formatted_date': 'month_year'})
arbitrary_avg_monthly_new_orders_filtered_made



#[(result_df1['client_status'] == 'ACTIVE')]
#arbitrary_avg_monthly_new_orders_filtered = arbitrary_avg_monthly_new_orders_filtered.groupby(['month_year_made_earliest'])['client_id'].nunique().reset_index(name='count')
#result_df1 = result_df1.groupby(['month_year_made'])['client_id'].nunique().reset_index(name='count')

plt.figure(figsize=(35, 15))
plt.plot(arbitrary_avg_monthly_new_orders_filtered_made['month_year'], 
         arbitrary_avg_monthly_new_orders_filtered_made['avg_count_last_12_months'], 
         marker='o')
plt.title('Unique Client Count by Month and Year')
plt.xlabel('Month and Year')
plt.ylabel('Unique Client Count')
plt.grid(True)

# Set the locator for the x-axis to show labels quarterly
#ax = plt.gca()
#ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display the result
print(arbitrary_avg_monthly_new_orders_filtered_made)

In [ ]:
x1 = arbitrary_avg_monthly_new_orders_filtered_made['month_year']
x2 = arbitrary_avg_monthly_new_orders_filtered_receipt['month_year']
y1 = arbitrary_avg_monthly_new_orders_filtered_made['avg_count_last_12_months']
y2 = arbitrary_avg_monthly_new_orders_filtered_receipt['avg_count_last_12_months']

df_deputyship_trend = {"month_year_made": x1, 
        "avg_count_last_12_months_made": y1,
                      "month_year_receipt": x2, 
        "avg_count_last_12_months_receipt": y2} 
df_deputyship_trend = pd.DataFrame(df_deputyship_trend)
df_deputyship_trend

In [ ]:
# Plotting the data
x1 = arbitrary_avg_monthly_new_orders_filtered_made['month_year']
x2 = arbitrary_avg_monthly_new_orders_filtered_receipt['month_year']
y1 = arbitrary_avg_monthly_new_orders_filtered_made['avg_count_last_12_months']
y2 = arbitrary_avg_monthly_new_orders_filtered_receipt['avg_count_last_12_months']

plt.figure(figsize=(35, 15))
plt.plot(x1, y1, marker='o', label='Number of the new deputyship orders based on the made date')
plt.plot(x2, y2, marker='o', label='Number of the new deputyship orders based on the receipt date')
plt.title('Unique Client Count by Month and Year')
# Adding legend
plt.legend()
plt.xlabel('Month and Year')
plt.ylabel('Unique Client Count')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

x1 = arbitrary_avg_monthly_new_orders_filtered_made['month_year']
x2 = arbitrary_avg_monthly_new_orders_filtered_receipt['month_year']
y1 = arbitrary_avg_monthly_new_orders_filtered_made['avg_count_last_12_months']
y2 = arbitrary_avg_monthly_new_orders_filtered_receipt['avg_count_last_12_months']

df_deputyship_trend = {"month_year_made": x1, 
        "avg_count_last_12_months_made": y1,
                      "month_year_receipt": x2, 
        "avg_count_last_12_months_receipt": y2} 
df_deputyship_trend = pd.DataFrame(df_deputyship_trend)
df_deputyship_trend

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df_deputyship_trend is already created as per your code
#df_deputyship_trend['month_year_made'] = pd.to_datetime(df_deputyship_trend['month_year_made'])
#df_deputyship_trend['month_year_receipt'] = pd.to_datetime(df_deputyship_trend['month_year_receipt'])

# Calculate percentage change
df_deputyship_trend['pct_change_made'] = df_deputyship_trend['avg_count_last_12_months_made'].pct_change() * 100
df_deputyship_trend['pct_change_receipt'] = df_deputyship_trend['avg_count_last_12_months_receipt'].pct_change() * 100

# Define the lockdown date
lockdown_date = 'Mar-20' #pd.to_datetime('2020-03-01')

# Split the data into before and after lockdown
before_lockdown_made = df_deputyship_trend[df_deputyship_trend['month_year_made'] < lockdown_date]
after_lockdown_made = df_deputyship_trend[df_deputyship_trend['month_year_made'] >= lockdown_date]

before_lockdown_receipt = df_deputyship_trend[df_deputyship_trend['month_year_receipt'] < lockdown_date]
after_lockdown_receipt = df_deputyship_trend[df_deputyship_trend['month_year_receipt'] >= lockdown_date]

# Calculate average percentage change before and after lockdown
avg_pct_change_before_made = before_lockdown_made['pct_change_made'].mean()
avg_pct_change_after_made = after_lockdown_made['pct_change_made'].mean()

avg_pct_change_before_receipt = before_lockdown_receipt['pct_change_receipt'].mean()
avg_pct_change_after_receipt = after_lockdown_receipt['pct_change_receipt'].mean()

# Print the results
print(f"Average percentage change in 'made' before lockdown: {avg_pct_change_before_made:.2f}%")
print(f"Average percentage change in 'made' after lockdown: {avg_pct_change_after_made:.2f}%")
print(f"Average percentage change in 'made' before compare to after lockdown: {(avg_pct_change_after_made / (avg_pct_change_before_made + avg_pct_change_after_made)):.2f}%")
print(f"Average percentage change in 'receipt' before lockdown: {avg_pct_change_before_receipt:.2f}%")
print(f"Average percentage change in 'receipt' after lockdown: {avg_pct_change_after_receipt:.2f}%")
print(f"Average percentage change in 'receipt' before compare to after lockdown: {(avg_pct_change_after_receipt / (avg_pct_change_after_receipt + avg_pct_change_after_receipt)):.2f}%")

# Plotting the data
plt.figure(figsize=(35, 15))
plt.plot(df_deputyship_trend['month_year_made'], df_deputyship_trend['avg_count_last_12_months_made'], marker='o', label='Number of the new deputyship orders based on the made date')
plt.plot(df_deputyship_trend['month_year_receipt'], df_deputyship_trend['avg_count_last_12_months_receipt'], marker='o', label='Number of the new deputyship orders based on the receipt date')
plt.title('Unique Client Count by Month and Year')
plt.legend()
plt.xlabel('Month and Year')
plt.ylabel('Unique Client Count')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df_deputyship_trend is already created and datetime columns are parsed
df_deputyship_trend['month_year_made'] = pd.to_datetime(df_deputyship_trend['month_year_made'], format='%b-%y')
df_deputyship_trend['month_year_receipt'] = pd.to_datetime(df_deputyship_trend['month_year_receipt'], format='%b-%y')

# Define the lockdown date
lockdown_date = pd.to_datetime('2020-03-01') #'Mar-20' #

# Split the data into before and after lockdown
before_lockdown_made = df_deputyship_trend[df_deputyship_trend['month_year_made'] < lockdown_date]
after_lockdown_made = df_deputyship_trend[df_deputyship_trend['month_year_made'] >= lockdown_date]

before_lockdown_receipt = df_deputyship_trend[df_deputyship_trend['month_year_receipt'] < lockdown_date]
after_lockdown_receipt = df_deputyship_trend[df_deputyship_trend['month_year_receipt'] >= lockdown_date]

# Calculate the total or average number of orders before and after lockdown
total_before_made = before_lockdown_made['avg_count_last_12_months_made'].sum()
total_after_made = after_lockdown_made['avg_count_last_12_months_made'].sum()

total_before_receipt = before_lockdown_receipt['avg_count_last_12_months_receipt'].sum()
total_after_receipt = after_lockdown_receipt['avg_count_last_12_months_receipt'].sum()

# Calculate the percentage change
pct_change_made = (total_after_made / (total_after_made + total_before_made)) * 100
pct_change_receipt = (total_after_receipt / (total_after_receipt + total_before_receipt)) * 100

# Print the results
print(f"Percentage change in 'made' after lockdown compared to before: {pct_change_made:.2f}%")
print(f"Percentage change in 'receipt' after lockdown compared to before: {pct_change_receipt:.2f}%")

# Plotting the data
plt.figure(figsize=(35, 15))
plt.plot(df_deputyship_trend['month_year_made'], df_deputyship_trend['avg_count_last_12_months_made'], marker='o', label='Number of new deputyship orders (made date)')
plt.plot(df_deputyship_trend['month_year_receipt'], df_deputyship_trend['avg_count_last_12_months_receipt'], marker='o', label='Number of new deputyship orders (receipt date)')
plt.axvline(x=lockdown_date, color='r', linestyle='--', label='Lockdown Start (Mar-20)')
plt.title('Unique Client Count by Month and Year')
plt.legend()
plt.xlabel('Month and Year')
plt.ylabel('Unique Client Count')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Read the Excel file
file_path = 'supervision_clients_caseflow.xlsx'
df = pd.read_excel(file_path, sheet_name='clients')

In [ ]:
import pandas as pd
#SELECT c.caserecnumber, MIN(c.orderdate) AS OldestMadeDate, MIN(c.receiptdate) AS OldestRcvdDate FROM opg_sirius_prod.cases c WHERE c.glueexporteddate = current_date AND c.type = 'order' GROUP BY c.caserecnumber

# Filter the DataFrame based on the conditions
#filtered_df = df[(df['glueexporteddate'] == pd.Timestamp('today').normalize()) & (df['type'] == 'order')]
filtered_df = df
filtered_df['order_made_date'] = pd.to_datetime(filtered_df['order_made_date'], errors='coerce')
filtered_df['order_issue_date'] = pd.to_datetime(filtered_df['order_issue_date'], errors='coerce')
filtered_df['earliest_order_made_date'] = pd.to_datetime(filtered_df['earliest_order_made_date'], errors='coerce')
filtered_df['earliest_receipt_date'] = pd.to_datetime(filtered_df['earliest_receipt_date'], errors='coerce')

snapshot_start = '2017-01-01'
snapshot_end = filtered_df['earliest_receipt_date'].max()
#snapshot_end = str(snapshot_end)
snapshot_end = str(target_date)
snapshot_end = snapshot_end[:10]

filtered_df = filtered_df.loc[(filtered_df['earliest_receipt_date'] >= pd.to_datetime(snapshot_start))]
filtered_df = filtered_df.loc[(filtered_df['earliest_order_made_date'] <= pd.to_datetime(snapshot_end))]


filtered_df

In [ ]:
dff=filtered_df.loc[(filtered_df['earliest_receipt_date'] >= pd.to_datetime(snapshot_start))]
dff['earliest_receipt_date']

In [ ]:
# Plotting the line chart
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

filtered_df1 = filtered_df
filtered_df1['month_year_made_earliest'] = filtered_df1['earliest_order_made_date'].dt.strftime('%b-%y')
result_df1 = filtered_df1 #[(result_df1['client_status'] == 'ACTIVE')]
result_df1 = result_df1.groupby(['month_year_made_earliest'])['client_id'].nunique().reset_index(name='count')
#result_df1 = result_df1.groupby(['month_year_made'])['client_id'].nunique().reset_index(name='count')

plt.figure(figsize=(35, 15))
plt.plot(result_df1['month_year_made_earliest'], result_df1['count'], marker='o')
plt.title('Unique Client Count by Month and Year')
plt.xlabel('Month and Year')
plt.ylabel('Unique Client Count')
plt.grid(True)

# Set the locator for the x-axis to show labels quarterly
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display the result
print(result_df1)


In [ ]:
filtered_df1['month_year_made_earliest']

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

# Ensure 'month_year_made_earliest' is in datetime format
result_df1['month_year_made_earliest'] = pd.to_datetime(result_df1['month_year_made_earliest'])

plt.figure(figsize=(35, 15))
plt.plot(result_df1['month_year_made_earliest'], result_df1['count'], marker='o')
plt.title('Unique Client Count by Month and Year')
plt.xlabel('Month and Year')
plt.ylabel('Unique Client Count')
plt.grid(True)

# Set the locator for the x-axis to show labels monthly
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

# Set x-axis limits
#ax.set_xlim(pd.Timestamp('2017-01-01'), pd.Timestamp('2024-08-31'))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display the result
print(result_df1)


In [ ]:


filtered_df2 = filtered_df
filtered_df2['month_year_earliest_receipt_date'] = filtered_df2['earliest_receipt_date'].dt.strftime('%b-%y')
result_df2 = filtered_df2 #[(result_df1['client_status'] == 'ACTIVE')]
result_df2 = result_df2.groupby(['month_year_earliest_receipt_date'])['client_id'].nunique().reset_index(name='count')
#result_df1 = result_df1.groupby(['month_year_made'])['client_id'].nunique().reset_index(name='count')

# Display the result
print(result_df2)

# Plotting the line chart
plt.figure(figsize=(35, 15))
plt.plot(result_df2['month_year_earliest_receipt_date'], result_df2['count'], marker='o')
plt.title('Unique Client Count by Month and Year')
plt.xlabel('Month and Year')
plt.ylabel('Unique Client Count')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
# Group by 'caserecnumber' and calculate the minimum dates
# result_df3 = filtered_df
# result_df3 = filtered_df3.groupby('caserecnumber').agg(
#     OldestMadeDate=('orderdate', 'min'),
#     OldestRcvdDate=('receiptdate', 'min')
# ).reset_index()

# Display the result
print(result_df2)

In [ ]:
with 
    earliest_receiptdate AS (
        SELECT 
            MIN(c.receiptdate) as receiptdate,
            c.caserecnumber
        FROM opg_sirius_prod.cases c
        WHERE c.glueexporteddate = DATE('2024-08-15')
            AND c.type = 'order'
        GROUP BY c.caserecnumber
    )
SELECT 
    year(c.receiptdate) as year,
    count(*) as num_orders
FROM earliest_receiptdate c
GROUP BY year(c.receiptdate) 
order by year(c.receiptdate) desc

In [ ]:
WITH monthly_counts AS (
    SELECT 
        EXTRACT(YEAR FROM c.receiptdate) AS year, 
        EXTRACT(MONTH FROM c.receiptdate) AS month, 
        COUNT(*) AS count_per_month
    FROM 
        opg_sirius_prod.cases c
    WHERE
        c.glueexporteddate = DATE('2024-08-15')
            and c.type = 'order'
    GROUP BY 
        EXTRACT(YEAR FROM c.receiptdate), 
        EXTRACT(MONTH FROM c.receiptdate)
)
 
SELECT 
    mc1.year, 
    mc1.month, 
    AVG(mc2.count_per_month) AS avg_count_last_12_months
FROM 
    monthly_counts mc1
JOIN 
    monthly_counts mc2 
ON 
    (mc2.year * 12 + mc2.month) BETWEEN (mc1.year * 12 + mc1.month) - 11 AND (mc1.year * 12 + mc1.month)
GROUP BY 
    mc1.year, 
    mc1.month
ORDER BY 
    mc1.year DESC, 
    mc1.month DESC